# Import

In [2]:
!pip install numpy scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install scipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!pip install networkx==2.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import csv
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import numpy as np
import scipy


from networkx import edge_betweenness_centrality as betweenness
from networkx import find_cliques
from google.colab import files
from scipy.cluster.hierarchy import ward, dendrogram, linkage, fcluster
from scipy.cluster import hierarchy
from networkx.algorithms import community
from community import community_louvain
from collections import defaultdict
from scipy.linalg import eigh
from sklearn.cluster import KMeans


# Load Data

In [12]:
def load_graph_from_csv(filename):
  G = nx.Graph()

  with open(filename, 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
      # Take the reciprocal of weight because weight is the number of interactions between two people
      node1, node2, weight = row[0], row[1], 1/int(row[4])
      G.add_edge(node1, node2, weight=weight)
  
  return G

In [7]:
filename = 'asoiaf-all-edges.csv'
G = load_graph_from_csv(filename)

In [19]:
# Create a dictionary of correspondence between matrix indices and nodes in the graph
nodes = list(G.nodes())
index_to_node = {}

for i, node in enumerate(nodes):
    index_to_node[i] = node

# Fast Greedy Algorithm

In [ ]:
def fast_greedy(G, n):
    """
    Detect communities in a given graph using a fast clustering algorithm
    
    G: networkx.Graph
    n: stop when there is n communities
        
    Return: List[List[str]]
        A list of communities in the graph, each community is a list of node identifiers (strings)
    """
    # Divide all nodes into separate communities
    communities = []
    for node in G.nodes():
        communities.append([node])
    
    # Find the largest edge weight between all nodes
    while True:
        max_edge = (None, None, 0)  # Initialize the maximum edge weight
        for u, v, weight in G.edges.data('weight'):
            if weight > max_edge[2]:
                # print(u,v,weight)
                max_edge = (u, v, weight)
        
        # If no node can be found to merge, exit the loop
        if max_edge[2] == 0:
            break
        
        # Merge the community of the node corresponding to the maximum edge weight
        u_community = None
        v_community = None
        for c in communities:
            if max_edge[0] in c:
                u_community = c
            if max_edge[1] in c:
                v_community = c
   
        if u_community != v_community:
            u_community.extend(v_community)
            communities.remove(v_community)
        G.add_edge(max_edge[0], max_edge[1], weight=0)

        if(len(communities)<=n):
          break
        
            
    return communities

In [ ]:
communities_list = fast_greedy(G, 100)
with open("fast_greedy.txt", "w") as f:
  f.write(str(communities_list))

# Girvan Newman

In [ ]:
def girvan_newman(G):    
    # Compute the best partition
    comp = nx.algorithms.community.girvan_newman(G, most_valuable_edge=most_central_edge)
    
    # Return the partition as a dictionary
    k = 10
    limited = itertools.takewhile(lambda c: len(c) <= k, comp)

    communities_list = []
    for communities in limited:
      communities_list.append(communities)

    return communities_list

In [ ]:
def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    return max(centrality, key=centrality.get)

In [ ]:
communities_list = girvan_newman(G)
communities_list
with open("girvan_newman.txt", "w") as f:
  f.write(str(communities_list))

# Louvain

In [ ]:
# Computing the Community Structure of a Powered Graph
partition = community_louvain.best_partition(G)

# Print the community each node is in
for node, community in partition.items():
    print("Node", node, "belongs to community", community)

# Create a dictionary, the default value is an empty list
communities = defaultdict(list)

# Traversing Nodes and Communities
for node, community in partition.items():
    # Add the node to the list corresponding to the community
    communities[community].append(node)


with open("louvain.txt", "w") as f:
  f.write(str(communities))

# Bron-Kerbosch

In [ ]:
# Get a list of all nodes in the graph
vertices = list(G.nodes())

# Initialize the community list
communities = []
desired_size = 1

def bron_kerbosch(r, p, x):
    # If both p and x are empty, then r is a community
    if not p and not x:
      if len(r) >= desired_size:
          # Check if the community already exists in the list
          if r not in communities:
              communities.append(r)
              return
    # Iterate over all nodes
    for v in p.copy():
        # Find the list of neighbors of v
        neighbors = list(G.neighbors(v))
        # Compute a new set of nodes for v
        new_p = p.intersection(neighbors)
        new_x = x.intersection(neighbors)
        # Recursively call the Bron-Kerbosch algorithm
        bron_kerbosch(r.union(set([v])), new_p, new_x)
        p.remove(v)
        x.add(v)

# Call the Bron-Kerbosch algorithm
bron_kerbosch(set(), set(vertices), set())

# Export community
for community in communities:
    print(community)

In [32]:
with open("bron_kerbosch.txt", "w") as f:
  f.write(str(communities))

# Hierarchical clustering

In [24]:
# Initialize each point as a separate community
communities = [[node] for node in G.nodes()]

# Repeat until only 10 communities remain
while len(communities) > 10:
  # Initialize the most similar two communities
  most_similar_communities = (0, 1)
  highest_similarity = -1

  # Iterate over all community pairs
  for i, community1 in enumerate(communities):
      for j, community2 in enumerate(communities):
          # skip the same community
          if i == j:
              continue
          # Calculate the similarity between communities
          similarity = 0
          for node1 in community1:
              for node2 in community2:
                  if G.has_edge(node1, node2):
                      similarity += G[node1][node2]['weight']
          # If the similarity is the highest, update the two most similar communities
          if similarity > highest_similarity:
              most_similar_communities = (i, j)
              highest_similarity = similarity
  # Merge the two most similar communities
  communities[most_similar_communities[0]] += communities[most_similar_communities[1]]
  communities.pop(most_similar_communities[1])

In [25]:
with open("hierarchical_clustering.txt", "w") as f:
  f.write(str(communities))

# Spectral clustering

In [24]:
# Build an adjacency matrix of a weighted graph
A = nx.adjacency_matrix(G)

# Perform eigendecomposition on the adjacency matrix to get the eigenvector of each point
_, V = eigh(A.toarray())

# Project eigenvectors into a low-dimensional space
X = V[:, :2]

# Group points into communities using K-Means clustering algorithm in low-dimensional space
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)

<ipython-input-24-eaa3838e5b22>:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


KMeans(n_clusters=10)

In [25]:
communities = {}
for i, label in enumerate(kmeans.labels_):
    node = index_to_node[i]
    if label not in communities:
        communities[label] = []
    communities[label].append(node)

{9: ['Addam-Marbrand', 'Daenerys-Targaryen', 'Theon-Greyjoy', 'Balon-Greyjoy', 'Barristan-Selmy', 'Brienne-of-Tarth', 'Jon-Arryn', 'Ramsay-Snow', 'Alayaya', 'Cortnay-Penrose', 'Lancel-Lannister', 'Chiswyck', 'Edric-Storm', 'Arys-Oakheart', 'Doran-Martell', 'Ellaria-Sand', 'Benjen-Stark', 'Mordane', 'Rodrik-Cassel', 'Vayon-Poole', 'Harys-Swyft', 'Baelor-I-Targaryen', 'Tanda-Stokeworth', 'Cleos-Frey', 'Lysa-Arryn', 'Garlan-Tyrell', 'Garth-Tyrell', 'High-Septon-(Tyrions)', 'Maegor-I-Targaryen', 'Mark-Mullendore', 'Preston-Greenfield', 'Raynard', 'Vylarr', 'High-Septon-(fat_one)', 'Hallyne'], 0: ['Brynden-Tully', 'Lyle-Crakehall', 'Varys', 'Aegon-Frey-(son-of-Stevron)', 'Stevron-Frey', 'Harren-Hoare', 'Rhaenys-Targaryen', 'Visenya-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Franklyn-Flowers', 'Haldon', 'Harry-Strickland', 'Lemore', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Rolly-Duckfield', 'Viserys-Targaryen', 'Yandry', 'Ysilla', 'Aegon-V-Targaryen', 'Alleras', 'Daeron-II-Targaryen

In [26]:
with open("spectral_clustering.txt", "w") as f:
  f.write(str(communities))

# Post-process
Variable Names:

fast_greedy

girvan_newman

louvain

bron_kerbosch

hierarchical_clustering

spectral_clustering

## Load Results

In [58]:
fast_greedy = [['Harry-Strickland', 'Myles-Toyne'], ['Rolly-Duckfield'], ['Aegon-V-Targaryen'], ['Dareon', 'Lanna-(Happy-Port)'], ['Gormond-Goodbrother'], ['Rus'], ['Megga-Tyrell'], ['Alys-Karstark'], ['Alysane-Mormont'], ['Tickler'], ['Ardrian-Celtigar'], ['Obara-Sand'], ['Cedra'], ['Mero', 'Grazdan-mo-Eraz'], ['Brusco'], ['Denyo-Terys', 'Ternesio-Terys'], ['Kindly-Man'], ['Meralyn'], ['Ravella-Swann'], ['Umma'], ['Waif'], ['Yorko-Terys'], ['Azor-Ahai'], ['Bael-the-Bard'], ['Balman-Byrch'], ['Bannen'], ['Reznak-mo-Reznak'], ['Mariya-Darry'], ['Wick-Whittlestick'], ['Leaf'], ['Walder-Frey-(son-of-Jammos)'], ['Creighton-Longbough', 'Brella', 'Galazza-Galare', 'Brea', 'Arron', 'Archibald-Yronwood', 'Anya-Waynwood', 'Andrey-Dalt', 'Amabel', 'Aggar', 'Gariss', 'Gelmarr', 'Gynir', 'Aeron-Greyjoy', 'Grigg', 'Jarl', 'Varamyr', 'Harma', 'Holly', 'Nute', 'Garth-(Wolfs-Den)', 'Damon-Dance-for-Me', 'Mezzara', 'Pylos', 'Maggy', 'Jason-Mallister', 'Vardis-Egen', 'Marillion', 'Lollys-Stokeworth', 'Shagwell', 'Cley-Cerwyn', 'Brynden-Rivers', 'Blue-Bard', 'Benfred-Tallhart', 'Benethon-Scales', 'Khrazz', 'Groleo', 'Ben-Plumm', 'Dywen', 'Osmund-Kettleblack', 'Jocelyn-Swyft', 'Rorge', 'Lommy-Greenhands', 'Arwyn-Oakheart', 'Arstan', 'Emrick', 'Hareth-(Moles-Town)', 'Mors-Umber', 'Gerris-Drinkwater', 'Caggo', 'Tattered-Prince', 'Horton-Redfort', 'Harrold-Hardyng', 'Benedar-Belmore', 'Gilwood-Hunter', 'Lem', 'Sylva-Santagar', 'Garin-(orphan)', 'Vargo-Hoat', 'Pyg', 'Robett-Glover', 'Gregor-Clegane', 'Justin-Massey', 'Harwin', 'Alyn', 'Armen', 'Elinor-Tyrell', 'Alla-Tyrell', 'Axell-Florent', 'Salladhor-Saan', 'Alester-Florent', 'Lamprey', 'Alerie-Hightower', 'Bran-Stark', 'Chayle', 'Hali', 'Hallis-Mollen', 'Albett', 'Halder', 'Daario-Naharis', 'Ramsay-Snow', 'Donella-Hornwood', 'Meldred-Merlyn', 'Gorold-Goodbrother', 'Dunstan-Drumm', 'Rodrik-Harlaw', 'Baelor-Blacktyde', 'Asha-Greyjoy', 'Arnolf-Karstark', 'Hosteen-Frey', 'Aenys-Frey', 'Donal-Noye', 'Mance-Rayder', 'Arya-Stark', 'Jeor-Mormont', 'Blane', 'Jorah-Mormont', 'Lark', 'Denys-Mallister', 'Aemon-Targaryen-(Maester-Aemon)', 'Cotter-Pyke', 'Rhaegar-Targaryen', 'Arthur-Dayne', 'Rhaenys-Targaryen', 'Eddard-Stark', 'Catelyn-Stark', 'Balon-Greyjoy', 'Varys', 'Tywin-Lannister', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Brienne-of-Tarth', 'Elia-Martell', 'Elmar-Frey', 'Emmon-Frey', 'Genna-Lannister', 'Jon-Umber-(Greatjon)', 'Marq-Piper', 'Martyn-Lannister', 'Renly-Baratheon', 'Jaime-Lannister', 'Aemon-Targaryen-(Dragonknight)', 'Amory-Lorch', 'Clement-Piper', 'Dick-Crabb', 'Galbart-Glover', 'Garrett-Paege', 'Harys-Swyft', 'Hugo-Vance', 'Jon-Snow', 'Joy-Hill', 'Lewys-Piper', 'Melara-Hetherspoon', 'Addam-Marbrand', 'Brynden-Tully', 'Cersei-Lannister', 'Gyles-Rosby', 'Jalabhar-Xho', 'Joffrey-Baratheon', 'Kevan-Lannister', 'Oberyn-Martell', 'Daven-Lannister', 'Edwyn-Frey', 'Nestor-Royce', 'Ryman-Frey', 'Aerys-II-Targaryen', 'Alayaya', 'Arys-Oakheart', 'Benjen-Stark', 'Chiswyck', 'Daenerys-Targaryen', 'Doran-Martell', 'Garlan-Tyrell', 'Garth-Tyrell', 'High-Septon-(Tyrions)', 'Maegor-I-Targaryen', 'Mark-Mullendore', 'Mordane', 'Preston-Greenfield', 'Raynard', 'Tanda-Stokeworth', 'Vylarr', 'Aurane-Waters', 'Mace-Tyrell', 'Sansa-Stark', 'Tyrion-Lannister', 'Olenna-Redwyne', 'Rafford', 'Rennifer-Longwaters', 'Rodrik-Cassel', 'Shadrich', 'Shitmouth', 'Stafford-Lannister', 'Tysha', 'Wyl-(guard)', 'Wylis-Manderly', 'Zollo', 'Lambert-Turnberry', 'Tallad', 'Beric-Dondarrion', 'Hallyne', 'Horas-Redwyne', 'Qyburn', 'Selyse-Florent', 'Shae', 'Shireen-Baratheon', 'Yoren', 'Ronnet-Connington', 'Stannis-Baratheon', 'Willem-Lannister', 'Anders-Yronwood', 'Areo-Hotah', 'Arianne-Martell', 'Daemon-Sand', 'Tyene-Sand', 'Aegon-I-Targaryen', 'Baelor-I-Targaryen', 'Barristan-Selmy', 'Cortnay-Penrose', 'Galyeon-of-Cuy', 'Kasporio', 'Pia', 'Taena-of-Myr', 'Theon-Greyjoy', 'Lancel-Lannister', 'Mandon-Moore', 'Margaery-Tyrell', 'Robb-Stark', 'Timett', 'Brenett', 'Colemon', 'Donnel-Waynwood', 'Eon-Hunter', 'Hyle-Hunt', 'Lothar-Frey', 'Mathis-Rowan', 'Moreo-Tumitis', 'Mychel-Redfort', 'Robert-Arryn', 'Robin-Ryger', 'Utherydes-Wayn', 'Brandon-Stark', 'Dacey-Mormont', 'Davos-Seaworth', 'Hullen', 'Martyn-Rivers', 'Myrcella-Baratheon', 'Robin-Flint', 'Stiv', 'Sybell-Spicer', 'Torrhen-Karstark', 'Tristifer-IV-Mudd', 'Wendel-Manderly', 'Nan', 'Torrhen-Stark', 'Visenya-Targaryen', 'Benerro', 'Bloodbeard', 'Cletus-Yronwood', 'Daeron-II-Targaryen', 'Ezzara', 'Lynesse-Hightower', 'Lysono-Maar', 'Marselen', 'Miklaz', 'Moqorro', 'Spotted-Cat', 'Cleos-Frey', 'Daryn-Hornwood', 'Edmure-Tully', 'Gendry', 'Gerold-Hightower', 'Howland-Reed', 'Maron-Greyjoy', 'Mycah', 'Wylla', 'Allard-Seaworth', 'Alliser-Thorne', 'Barbrey-Dustin', 'Bowen-Marsh', 'Bronn', 'Bryce-Caron', 'Clydas', 'Corliss-Penny', 'Dontos-Hollard', 'Jory-Cassel', 'Khorane-Sathmantes', 'Lyanna-Mormont', 'Marlon-Manderly', 'Marya-Seaworth', 'Monford-Velaryon', 'Orton-Merryweather', 'Osney-Kettleblack', 'Randyll-Tarly', 'Richard-Horpe', 'Roose-Ryswell', 'Sigorn', 'Steffon-Baratheon', 'Theomore', 'William-Foxglove', 'Franklyn-Flowers', 'Viserys-Targaryen', 'Ashara-Dayne', 'Black-Balaq', 'Rhaella-Targaryen', 'Doreah', 'Eddison-Tollett', 'Hobb', 'Maekar-I-Targaryen', 'Owen', 'Wynton-Stout', 'Grenn', 'Satin', 'Cellador', 'Pypar', 'Janos-Slynt', 'Gilly', 'Mully', 'Othell-Yarwyck', 'Watt', 'Coldhands', 'Jojen-Reed', 'Meera-Reed', 'Small-Paul', 'Craster', 'Melisandre', 'Samwell-Tarly', 'Allar-Deem', 'Aegon-Frey-(son-of-Stevron)', 'Stevron-Frey', 'Walder-Frey', 'Danwell-Frey', 'Pycelle', 'Roose-Bolton', 'Ottyn-Wythers', 'Arson', 'Big-Boil', 'Brandon-Norrey', 'Dick-Follard', 'Garth-Greyfeather', 'Gendel', 'Godry-Farring', 'Gorne', 'Jack-Bulwer', 'Mag-Mar-Tun-Doh-Weg', 'Matthar', 'Narbert-Grandison', 'Soren-Shieldbreaker', 'Ulmer', 'Weeper', 'Zei', 'Robert-Baratheon', 'Kegs', 'Aerys-I-Targaryen', 'Drogo', 'Edric-Storm', 'Hoster-Tully', 'Jeyne-Poole', 'Meryn-Trant', 'Val', 'Vayon-Poole', 'Willam-Dustin', 'Bedwyck', 'Clubfoot-Karl', 'Leo-Tyrell', 'Rast', 'Bellegere-Otherys', 'Desmond', 'Harra', 'Koss', 'Petyr-Baelish', 'Praed', 'Woth', 'Jared-Frey', 'Beth-Cassel', 'Captain-Myraham-Daughter', 'Drennan', 'Frenya', 'Grunt', 'Murenmure', 'Palla', 'Squint', 'Urzen', 'Victarion-Greyjoy', 'Walda-Frey-(daughter-of-Merrett)', 'Werlag', 'Wyman-Manderly', 'Hodor', 'Rhaegar-Frey', 'Grimtongue', 'Hagen', 'Red-Oarsman', 'Rolfe-the-Dwarf', 'Sybelle-Glover', 'Torgon-Greyiron', 'Tycho-Nestoris', 'Wex-Pyke', 'Dagmer', 'Harras-Harlaw', 'Longwater-Pyke', 'Ralf-Stonehouse', 'Wulfe', 'Elbert-Arryn', 'Jon-Arryn', 'Owen-Merryweather', 'Lewyn-Martell', 'Aggo', 'Boros-Blount', 'Grazhar', 'Jhogo', 'Larraq', 'Sandor-Clegane', 'Steelskin', 'Widower', 'Xaro-Xhoan-Daxos', 'Rickard-Stark', 'Rickon-Stark', 'Lysa-Arryn', 'Robar-Royce', 'Tommen-Baratheon', 'Vyman', 'Walton', 'Oswell-Whent', 'Alys-Arryn', 'Hugh', 'Walder-Frey-(son-of-Merrett)', 'Belwas', 'Quaro', 'Jhiqui', 'Quentyn-Martell', 'Skahaz-mo-Kandaq', 'Prendahl-na-Ghezn', 'Sallor', 'Jommo', 'Mirri-Maz-Duur', 'Missandei', 'Haldon', 'Illyrio-Mopatis', 'Pono', 'Rakharo', 'Haggo', 'Dancy', 'Falyse-Stokeworth', 'Shagga', 'Poxy-Tym', 'Reek', 'Robert-Strong', 'Dorcas', 'Senelle', 'Guncer-Sunglass', 'Imry-Florent', 'Leona-Woolfield', 'Matthos-Seaworth', 'Porridge', 'Balon-Swann', 'Cayn', 'Colen-of-Greenpools', 'Devan-Seaworth', 'Emmon-Cuy', 'Lorent-Caswell', 'Cressen', 'Patchface', 'Florian-the-Fool', 'Gretchel', 'Kella', 'Maddy', 'Myranda-Royce', 'Polliver', 'Tomard', 'Yohn-Royce', 'Alleras', 'Marwyn', 'Mollander', 'Roone', 'Pate-(novice)', 'Dalla', 'Ygritte', 'Jacelyn-Bywater', 'Dunsen', 'Hot-Pie', 'Ilyn-Payne', 'Weese', 'Cutjack', 'Greenbeard', 'Merrit', 'Willow-Heddle', 'Helman-Tallhart', 'Sharna', 'Loras-Tyrell', 'Rickard-Karstark', 'Anguy', 'Aron-Santagar', 'Biter', 'Jaqen-Hghar', 'Syrio-Forel', 'Cleon', 'Kedry', 'Old-Bill-Bone', 'Trystane-Martell', 'Ellaria-Sand', 'Franklyn-Fowler', 'Mellei', 'Morra', 'Ricasso', 'Timoth', 'Nymeria-Sand', 'Sarella-Sand', 'Husband', 'Thoros-of-Myr', 'Tansy', 'Tom-of-Sevenstreams', 'Grisel', 'Lyonel-Corbray', 'Merrett-Frey', 'Gerold-Dayne', 'Arthor-Karstark', 'Hother-Umber', 'Luwin', 'Maege-Mormont', 'Emmett', 'Leathers', 'Spare-Boot', 'Kraznys-mo-Nakloz', 'Irri', 'Grey-Worm', 'Hizdahr-zo-Loraq', 'Jonothor-Darry', 'Jaremy-Rykker', 'Waymar-Royce', 'Rowan', 'Squirrel', 'Halys-Hornwood', 'Lorren', 'Qarl-the-Maid', 'Tristifer-Botley', 'Lucas-Codd', 'Sigfryd-Harlaw', 'Paxter-Redwyne', 'Daeron-I-Targaryen', 'Orell', 'Stonesnake', 'Penny', 'Belaquo', 'Qezza', 'Puckens', 'Byam-Flint', 'Tormund', 'Camarron', 'Tybero-Istarion', 'Malegorn', 'Patrek-of-Kings-Mountain', 'Leobald-Tallhart', 'Stygg', 'Jafer-Flowers', 'Chett', 'Qhorin-Halfhand', 'Ragwyle', 'Thoren-Smallwood', 'Joseth', 'Hairy-Hal', 'Dryn', 'Torghen-Flint', 'Symon-Silver-Tongue', 'Eustace-Brune', 'Lothor-Brune', 'Willas-Tyrell', 'Forley-Prester', 'Tytos-Blackwood', 'Lyn-Corbray', 'Nage', 'Eldon-Estermont', 'Steffon-Varner', 'Lucifer-Long', 'Walder-Rivers', 'Karyl-Vance', 'Jon-Fossoway', 'Gyles-Grafton', 'Jonos-Bracken', 'Perwyn-Frey', 'Shadd', 'Leonette-Fossoway', 'Jaehaerys-I-Targaryen', 'Softfoot', 'Tarber', 'Draqaz', 'Jeyne-Westerling', 'Gawen-Westerling', 'Ogo', 'Heward', 'Harl', 'Rugen', 'Wun-Weg-Wun-Dar-Wun', 'Torwold-Browntooth', 'Shortear', 'Lyle-Crakehall', 'Moelle', 'Bonifer-Hasty', 'Hoster-Blackwood', 'High-Septon-(fat_one)', 'Amerei-Frey', 'Leo-Lefford', 'Morgo', 'Shrouded-Lord', 'Chiggen', 'Willis-Wode', 'Joanna-Lannister', 'Jon-Connington', 'Nyessos-Vhassar', 'Ballabar', 'Podrick-Payne', 'Cedric-Payne', 'Chella', 'Desmond-Grell', 'Raynald-Westerling', 'Rollam-Westerling', 'Roslin-Frey', 'Petyr-Frey', 'Harren-Hoare', 'Barsena', 'Oznak-zo-Pahl', 'Symon-Stripeback', 'Yurkhaz-zo-Yunzak', 'Jacks', 'Porther', 'Raymun-Darry', 'Tregar', 'Varly', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Lemore', 'Ysilla', 'Yandry', 'Lyanna-Stark', 'Gared', 'Jarman-Buckwell', 'Dalbridge', 'Errok', 'Styr', 'Fulk', 'Hake', 'Hop-Robin', 'Tom-Barleycorn', 'Dirk', 'Quhuru-Mo', 'Dobber', 'Mad-Huntsman', 'Pate-(Old)', 'Talea', 'Ben-Bones', 'Captain-Myraham', 'Euron-Greyjoy', 'Humfrey-Hewett', 'Quellon-Greyjoy', 'Sawane-Botley', 'Kyra', 'Alebelly', 'Hayhead', 'Mikken', 'Osha', 'Roger-Ryswell', 'Skinner', 'Luton', 'Willow-Witch-eye', 'Emmond', 'Greydon-Goodbrother', 'Clayton-Suggs', 'Hotho-Harlaw', 'Denys-Darklyn', 'Rossart', 'Smiling-Knight', 'Simon-Toyne', 'Clarence-Crabb', 'Elder-Brother', 'Meribald', 'Jeyne-Heddle', 'Selwyn-Tarth', 'Nurse', 'Qotho', 'Andrew-Estermont', 'Gerald-Gower', 'Bartimus', 'High-Sparrow', 'Quill', 'Elys-Waynwood', 'Joramun', 'Caleotte', 'Mebble', 'Weasel', 'Timeon', 'Symond-Templeton', 'Oswell-Kettleblack', 'Meris', 'Denzo-Dhan', 'Dick-Straw', 'Hugh-Hungerford', 'Manfrey-Martell', 'Rory', 'Pate-(whipping-boy)', 'Cromm', 'Frenken', 'Osfryd-Kettleblack', 'Hero', 'Goghor-the-Giant', 'Sweets', 'Rodrik-Sparr', 'Brus-Buckler', 'Munda', 'Jon-Umber-(Smalljon)', 'Narbert', 'Ulf-son-of-Umar', 'Olyvar-Frey'], ['Illifer'], ['Guyard-Morrigen'], ['Bump'], ['Butterbumps'], ['Masha-Heddle'], ['Mya-Stone'], ['Moon-Boy'], ['Scolera'], ['Unella'], ['Chataya'], ['Ghael'], ['Cohollo'], ['Conn'], ['Coratt'], ['Kurz'], ['Eroeh'], ['Pyat-Pree'], ['Qavo-Nogarys'], ['Quaithe'], ['Rhaegel-Targaryen'], ['Ebben'], ['Sour-Alyn', 'Yellow-Dick'], ['Godric-Borrell'], ['Dolf'], ['Rhaego'], ['Joss'], ['Tobho-Mott'], ['Farlen'], ['Jonquil'], ['Fogo'], ['Will-(prologue)'], ['Murch'], ['Gerold-Grafton', 'Terrance-Lynderly'], ['Gerrick-Kingsblood', 'Raymun-Redbeard'], ['Kojja-Mo'], ['Gormon-Tyrell', 'Walgrave'], ['Todder'], ['Gunthor-son-of-Gurn'], ['Haggon'], ['Rattleshirt'], ['Harwood-Stout'], ['Marghaz-zo-Loraq'], ['Hobber-Redwyne'], ['Jack-Be-Lucky'], ['Josmyn-Peckledon'], ['Urswyck'], ['Jeyne-Farman'], ['Othor'], ['Ryk'], ['Toregg'], ['Jyck'], ['Morrec'], ['Kerwin'], ['Kurleket'], ['Malaquo-Maegyr'], ['Mallador-Locke'], ['Mord'], ['Yezzan-zo-Qaggaz'], ['Rosey'], ['Xhondo'], ['Ralf-(Limper)'], ['Torbert'], ['Rolph-Spicer'], ['Theobald'], ['Scar'], ['Talbert-Serry'], ['Thistle'], ['Tion-Frey']]

In [59]:
girvan_newman = ({'Biter', 'Meribald', 'Karyl-Vance', 'Selwyn-Tarth', 'Elder-Brother', 'Senelle', 'Matthos-Seaworth', 'Gilwood-Hunter', 'Selyse-Florent', 'Kevan-Lannister', 'Lyanna-Mormont', 'Edmure-Tully', 'Mord', 'Petyr-Baelish', 'Guyard-Morrigen', 'Corliss-Penny', 'Alerie-Hightower', 'Colemon', 'Genna-Lannister', 'Garrett-Paege', 'Cressen', 'Janos-Slynt', 'Maggy', 'Kella', 'Shadrich', 'Tybero-Istarion', 'Mya-Stone', 'Butterbumps', 'Jonothor-Darry', 'Elys-Waynwood', 'Alayaya', 'Tyrion-Lannister', 'Steffon-Baratheon', 'Allard-Seaworth', 'Walder-Frey', 'Conn', 'Brienne-of-Tarth', 'Jeyne-Westerling', 'Ellaria-Sand', 'William-Foxglove', 'Lothor-Brune', 'Merrett-Frey', 'Leo-Lefford', 'Smiling-Knight', 'Marya-Seaworth', 'Falyse-Stokeworth', 'High-Sparrow', 'Robert-Strong', 'Coratt', 'Lewys-Piper', 'Mathis-Rowan', 'Lyonel-Corbray', 'Rennifer-Longwaters', 'Tywin-Lannister', 'Robert-Baratheon', 'Terrance-Lynderly', 'Arwyn-Oakheart', 'Daven-Lannister', 'Lyanna-Stark', 'Willas-Tyrell', 'Theomore', 'Vayon-Poole', 'Mordane', 'Rhaella-Targaryen', 'Allar-Deem', 'Hyle-Hunt', 'Ryman-Frey', 'Ulf-son-of-Umar', 'Caleotte', 'Brenett', 'Kasporio', 'Roslin-Frey', 'Rossart', 'Lollys-Stokeworth', 'Anya-Waynwood', 'Puckens', 'Oswell-Kettleblack', 'Amory-Lorch', 'Yohn-Royce', 'Balman-Byrch', 'Jonos-Bracken', 'Nestor-Royce', 'Jon-Arryn', 'Hobber-Redwyne', 'Pate-(whipping-boy)', 'Steffon-Varner', 'Catelyn-Stark', 'Kurleket', 'Barbrey-Dustin', 'Hoster-Tully', 'Alla-Tyrell', 'Davos-Seaworth', 'Dick-Crabb', 'Daeron-I-Targaryen', 'Galbart-Glover', 'Bonifer-Hasty', 'Pycelle', 'Andrew-Estermont', 'Scolera', 'Yoren', 'Chiswyck', 'Roose-Bolton', 'Shitmouth', 'Ardrian-Celtigar', 'Aron-Santagar', 'Lorent-Caswell', 'Orton-Merryweather', 'Morrec', 'Jaehaerys-I-Targaryen', 'Varys', 'Dunsen', 'Bronn', 'Porridge', 'Alys-Arryn', 'Elbert-Arryn', 'Ben-Plumm', 'Axell-Florent', 'Chella', 'Taena-of-Myr', 'Symon-Silver-Tongue', 'Garth-Tyrell', 'Jyck', 'Lyn-Corbray', 'Rafford', 'Clarence-Crabb', 'Lothar-Frey', 'Marillion', 'Cortnay-Penrose', 'Edwyn-Frey', 'Paxter-Redwyne', 'Emmon-Cuy', 'Shae', 'Grisel', 'Bartimus', 'Godric-Borrell', 'Simon-Toyne', 'Timeon', 'Joffrey-Baratheon', 'Forley-Prester', 'Jeyne-Heddle', 'Wylis-Manderly', 'Lambert-Turnberry', 'Zollo', 'Timett', 'Gerold-Hightower', 'Shortear', 'Aegon-Frey-(son-of-Stevron)', 'Symond-Templeton', 'High-Septon-(Tyrions)', 'Elia-Martell', 'Jacelyn-Bywater', 'Robert-Arryn', 'Loras-Tyrell', 'Shireen-Baratheon', 'Eldon-Estermont', 'Eon-Hunter', 'Walton', 'Stannis-Baratheon', 'Brynden-Tully', 'Shrouded-Lord', 'Oberyn-Martell', 'Urswyck', 'Denys-Darklyn', 'Podrick-Payne', 'Baelor-I-Targaryen', 'Melara-Hetherspoon', 'Ballabar', 'Mark-Mullendore', 'Barristan-Selmy', 'Blue-Bard', 'Jeyne-Farman', 'Vardis-Egen', 'Mace-Tyrell', 'Cersei-Lannister', 'Robett-Glover', 'Jocelyn-Swyft', 'Maegor-I-Targaryen', 'Joy-Hill', 'Jaqen-Hghar', 'Helman-Tallhart', 'Wyman-Manderly', 'Sansa-Stark', 'Benedar-Belmore', 'Elinor-Tyrell', 'Preston-Greenfield', 'Tytos-Blackwood', 'Cedric-Payne', 'Illifer', 'Ronnet-Connington', 'Hugo-Vance', 'Ilyn-Payne', 'Tysha', 'Vylarr', 'Tomard', 'Lewyn-Martell', 'Horas-Redwyne', 'Azor-Ahai', 'Mychel-Redfort', 'Pylos', 'Amerei-Frey', 'Chataya', 'Owen-Merryweather', 'Balon-Swann', 'Doran-Martell', 'Shagga', 'Garlan-Tyrell', 'Moreo-Tumitis', 'Randyll-Tarly', 'Horton-Redfort', 'Marlon-Manderly', 'Arthur-Dayne', 'Heward', 'Harys-Swyft', 'Oswell-Whent', 'Jalabhar-Xho', 'Jon-Fossoway', 'Unella', 'Tickler', 'Harrold-Hardyng', 'Jeyne-Poole', 'Megga-Tyrell', 'Rugen', 'Olenna-Redwyne', 'Eustace-Brune', 'Rhaegar-Targaryen', 'Polliver', 'Donnel-Waynwood', 'Jory-Cassel', 'Jaime-Lannister', 'Boros-Blount', 'Dolf', 'Meryn-Trant', 'Torbert', 'Tanda-Stokeworth', 'Gerold-Grafton', 'Emmon-Frey', 'Patchface', 'Nage', 'Galyeon-of-Cuy', 'Sandor-Clegane', 'Bryce-Caron', 'Florian-the-Fool', 'Gerald-Gower', 'Monford-Velaryon', 'Creighton-Longbough', 'Stevron-Frey', 'Josmyn-Peckledon', 'Weese', 'Raynard', 'Sybell-Spicer', 'Hugh', 'Garth-(Wolfs-Den)', 'Lysa-Arryn', 'Elmar-Frey', 'Petyr-Frey', 'Imry-Florent', 'Rickard-Karstark', 'Moelle', 'Tansy', 'Robin-Ryger', 'Gyles-Rosby', 'Margaery-Tyrell', 'Alester-Florent', 'Dorcas', 'Moon-Boy', 'Viserys-Targaryen', 'Widower', 'Marq-Piper', 'Mycah', 'Clement-Piper', 'Masha-Heddle', 'Aemon-Targaryen-(Dragonknight)', 'Ashara-Dayne', 'Mandon-Moore', 'Renly-Baratheon', 'Hallyne', 'Robar-Royce', 'Rorge', 'Lyle-Crakehall', 'Daemon-Sand', 'Dancy', 'Willis-Wode', 'Gregor-Clegane', 'Shagwell', 'Narbert', 'Vyman', 'Arys-Oakheart', 'Myranda-Royce', 'Addam-Marbrand', 'Maddy', 'Leonette-Fossoway', 'Aurane-Waters', 'Aerys-II-Targaryen', 'Larraq', 'Lamprey', 'Joanna-Lannister', 'Morgo', 'Myrcella-Baratheon', 'Osney-Kettleblack', 'Jonquil', 'Osmund-Kettleblack', 'Hoster-Blackwood', 'Dontos-Hollard', 'Gyles-Grafton', 'Tallad', 'Lancel-Lannister', 'Cayn', 'Pyg', 'Chiggen', 'Osfryd-Kettleblack', 'Colen-of-Greenpools', 'Leona-Woolfield', 'Brella', 'Roose-Ryswell', 'Gretchel', 'Walder-Rivers', 'Tommen-Baratheon', 'Gunthor-son-of-Gurn', 'Vargo-Hoat', 'Pia', 'Wyl-(guard)', 'Aerys-I-Targaryen', 'Qyburn', 'Frenken', 'Areo-Hotah', 'Edric-Storm', 'Gawen-Westerling'}, {'Benjen-Stark', 'Poxy-Tym', 'Rickard-Stark', 'Wendel-Manderly', 'Stiv', 'Robin-Flint', 'Osha', 'Donella-Hornwood', 'Martyn-Lannister', 'Hodor', 'Hother-Umber', 'Maege-Mormont', 'Nan', 'Robb-Stark', 'Hali', 'Walder-Frey-(son-of-Jammos)', 'Leobald-Tallhart', 'Arthor-Karstark', 'Luwin', 'Jon-Umber-(Greatjon)', 'Shadd', 'Rollam-Westerling', 'Aegon-I-Targaryen', 'Desmond-Grell', 'Perwyn-Frey', 'Raynald-Westerling', 'Arnolf-Karstark', 'Mors-Umber', 'Leaf', 'Howland-Reed', 'Olyvar-Frey', 'Beth-Cassel', 'Rodrik-Cassel', 'Meera-Reed', 'Benfred-Tallhart', 'Brandon-Stark', 'Tion-Frey', 'Rhaenys-Targaryen', 'Utherydes-Wayn', 'Harren-Hoare', 'Halys-Hornwood', 'Bran-Stark', 'Joseth', 'Rickon-Stark', 'Dacey-Mormont', 'Willem-Lannister', 'Torrhen-Karstark', 'Hullen', 'Daryn-Hornwood', 'Visenya-Targaryen', 'Martyn-Rivers', 'Torrhen-Stark', 'Brynden-Rivers', 'Hallis-Mollen', 'Walder-Frey-(son-of-Merrett)', 'Jon-Umber-(Smalljon)', 'Cley-Cerwyn', 'Tristifer-IV-Mudd', 'Rolph-Spicer', 'Stafford-Lannister'}, {'Daeron-II-Targaryen', 'Qavo-Nogarys', 'Yandry', 'Tattered-Prince', 'Daario-Naharis', 'Rakharo', 'Irri', 'Symon-Stripeback', 'Illyrio-Mopatis', 'Miklaz', 'Franklyn-Flowers', 'Grey-Worm', 'Nyessos-Vhassar', 'Denzo-Dhan', 'Quaithe', 'Skahaz-mo-Kandaq', 'Black-Balaq', 'Cohollo', 'Ezzara', 'Rhaego', 'Missandei', 'Aggo', 'Marghaz-zo-Loraq', 'Caggo', 'Quentyn-Martell', 'Barsena', 'Lysono-Maar', 'Ghael', 'Groleo', 'Mero', 'Dick-Straw', 'Jommo', 'Gerris-Drinkwater', 'Pyat-Pree', 'Old-Bill-Bone', 'Hizdahr-zo-Loraq', 'Spotted-Cat', 'Arstan', 'Belwas', 'Goghor-the-Giant', 'Lynesse-Hightower', 'Jon-Connington', 'Pono', 'Archibald-Yronwood', 'Draqaz', 'Drogo', 'Haldon', 'Ogo', 'Quaro', 'Haggo', 'Prendahl-na-Ghezn', 'Fogo', 'Benerro', 'Daenerys-Targaryen', 'Galazza-Galare', 'Lucifer-Long', 'Ysilla', 'Camarron', 'Khrazz', 'Trystane-Martell', 'Myles-Toyne', 'Sallor', 'Doreah', 'Meris', 'Kedry', 'Steelskin', 'Reznak-mo-Reznak', 'Rhaegel-Targaryen', 'Malaquo-Maegyr', 'Hero', 'Marselen', 'Mirri-Maz-Duur', 'Jhogo', 'Kraznys-mo-Nakloz', 'Qezza', 'Cletus-Yronwood', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Yurkhaz-zo-Yunzak', 'Xaro-Xhoan-Daxos', 'Grazdan-mo-Eraz', 'Jhiqui', 'Eroeh', 'Anders-Yronwood', 'Oznak-zo-Pahl', 'Bloodbeard', 'Qotho', 'Cleon', 'Hugh-Hungerford', 'Rolly-Duckfield', 'Harry-Strickland', 'Mezzara', 'Belaquo', 'Grazhar', 'Lemore', 'Aegon-Targaryen-(son-of-Rhaegar)'}, {'Tobho-Mott', 'Joss', 'Tregar', 'High-Septon-(fat_one)', 'Syrio-Forel', 'Raymun-Darry', 'Jacks', 'Willam-Dustin', 'Wylla', 'Varly', 'Eddard-Stark', 'Porther', 'Cleos-Frey', 'Maron-Greyjoy'}, {'Donal-Noye', 'Mallador-Locke', 'Scar', 'Marwyn', 'Cellador', 'Bannen', 'Gormon-Tyrell', 'Wun-Weg-Wun-Dar-Wun', 'Mag-Mar-Tun-Doh-Weg', 'Armen', 'Wick-Whittlestick', 'Errok', 'Brus-Buckler', 'Tormund', 'Khorane-Sathmantes', 'Kojja-Mo', 'Ebben', 'Alebelly', 'Leathers', 'Grigg', 'Grenn', 'Ygritte', 'Walgrave', 'Spare-Boot', 'Clubfoot-Karl', 'Chett', 'Roone', 'Halder', 'Gared', 'Hop-Robin', 'Lanna-(Happy-Port)', 'Jeor-Mormont', 'Pypar', 'Varamyr', 'Rosey', 'Gorne', 'Waymar-Royce', 'Guncer-Sunglass', 'Jon-Snow', 'Chayle', 'Frenya', 'Owen', 'Coldhands', 'Bael-the-Bard', 'Aemon-Targaryen-(Maester-Aemon)', 'Softfoot', 'Quhuru-Mo', 'Nurse', 'Malegorn', 'Val', 'Bowen-Marsh', 'Ragwyle', 'Arson', 'Small-Paul', 'Garth-Greyfeather', 'Albett', 'Jorah-Mormont', 'Mollander', 'Hairy-Hal', 'Jafer-Flowers', 'Hake', 'Mully', 'Toregg', 'Stonesnake', 'Mikken', 'Denys-Mallister', 'Weeper', 'Raymun-Redbeard', 'Tom-Barleycorn', 'Wynton-Stout', 'Aegon-V-Targaryen', 'Xhondo', 'Leo-Tyrell', 'Cotter-Pyke', 'Melisandre', 'Matthar', 'Jarl', 'Holly', 'Zei', 'Brandon-Norrey', 'Ottyn-Wythers', 'Thoren-Smallwood', 'Orell', 'Will-(prologue)', 'Haggon', 'Sigorn', 'Dareon', 'Dryn', 'Munda', 'Maekar-I-Targaryen', 'Kegs', 'Dick-Follard', 'Pate-(novice)', 'Ulmer', 'Qhorin-Halfhand', 'Othor', 'Gilly', 'Ryk', 'Soren-Shieldbreaker', 'Hareth-(Moles-Town)', 'Dalbridge', 'Jack-Bulwer', 'Eddison-Tollett', 'Penny', 'Blane', 'Sweets', 'Dirk', 'Big-Boil', 'Emmett', 'Samwell-Tarly', 'Lark', 'Yezzan-zo-Qaggaz', 'Torghen-Flint', 'Dywen', 'Benethon-Scales', 'Hobb', 'Patrek-of-Kings-Mountain', 'Alliser-Thorne', 'Gerrick-Kingsblood', 'Quill', 'Rattleshirt', 'Bedwyck', 'Todder', 'Narbert-Grandison', 'Jaremy-Rykker', 'Hayhead', 'Craster', 'Harma', 'Byam-Flint', 'Watt', 'Mance-Rayder', 'Fulk', 'Gendel', 'Clydas', 'Arron', 'Alleras', 'Alys-Karstark', 'Emrick', 'Theobald', 'Jojen-Reed', 'Devan-Seaworth', 'Rowan', 'Othell-Yarwyck', 'Joramun', 'Bump', 'Satin', 'Rast', 'Dalla', 'Rory', 'Styr', 'Salladhor-Saan', 'Jarman-Buckwell', 'Thistle', 'Squirrel'}, {'Squint', 'Hosteen-Frey', 'Luton', 'Palla', 'Urzen', 'Jason-Mallister', 'Ramsay-Snow', 'Captain-Myraham', 'Damon-Dance-for-Me', 'Willow-Witch-eye', 'Werlag', 'Lorren', 'Walda-Frey-(daughter-of-Merrett)', 'Yellow-Dick', 'Skinner', 'Sour-Alyn', 'Aenys-Frey', 'Kyra', 'Ben-Bones', 'Theon-Greyjoy', 'Captain-Myraham-Daughter', 'Jared-Frey', 'Rhaegar-Frey', 'Harwood-Stout', 'Farlen', 'Stygg', 'Reek', 'Grunt', 'Harl', 'Roger-Ryswell', 'Danwell-Frey', 'Drennan'}, {'Pate-(Old)', 'Koss', 'Brusco', 'Ravella-Swann', 'Ternesio-Terys', 'Brea', 'Arya-Stark', 'Amabel', 'Waif', 'Denyo-Terys', 'Mad-Huntsman', 'Praed', 'Bellegere-Otherys', 'Dobber', 'Desmond', 'Yorko-Terys', 'Talea', 'Umma', 'Meralyn', 'Kindly-Man', 'Harra'}, {'Baelor-Blacktyde', 'Tristifer-Botley', 'Dagmer', 'Tycho-Nestoris', 'Alysane-Mormont', 'Sybelle-Glover', 'Gariss', 'Qarl-the-Maid', 'Nute', 'Aeron-Greyjoy', 'Sawane-Botley', 'Lucas-Codd', 'Wex-Pyke', 'Murenmure', 'Cromm', 'Red-Oarsman', 'Ralf-Stonehouse', 'Murch', 'Greydon-Goodbrother', 'Humfrey-Hewett', 'Rodrik-Harlaw', 'Torgon-Greyiron', 'Rodrik-Sparr', 'Euron-Greyjoy', 'Gynir', 'Rus', 'Rolfe-the-Dwarf', 'Balon-Greyjoy', 'Justin-Massey', 'Harras-Harlaw', 'Clayton-Suggs', 'Gormond-Goodbrother', 'Emmond', 'Hagen', 'Dunstan-Drumm', 'Grimtongue', 'Gelmarr', 'Richard-Horpe', 'Wulfe', 'Godry-Farring', 'Longwater-Pyke', 'Talbert-Serry', 'Quellon-Greyjoy', 'Aggar', 'Torwold-Browntooth', 'Victarion-Greyjoy', 'Sigfryd-Harlaw', 'Asha-Greyjoy', 'Ralf-(Limper)', 'Moqorro', 'Kerwin', 'Gorold-Goodbrother', 'Meldred-Merlyn', 'Hotho-Harlaw'}, {'Willow-Heddle', 'Lommy-Greenhands', 'Gendry', 'Kurz', 'Hot-Pie', 'Mebble', 'Sharna', 'Tom-of-Sevenstreams', 'Cutjack', 'Thoros-of-Myr', 'Harwin', 'Jack-Be-Lucky', 'Woth', 'Tarber', 'Anguy', 'Merrit', 'Husband', 'Weasel', 'Greenbeard', 'Lem', 'Beric-Dondarrion', 'Mariya-Darry', 'Alyn'}, {'Sarella-Sand', 'Nymeria-Sand', 'Garin-(orphan)', 'Franklyn-Fowler', 'Andrey-Dalt', 'Mellei', 'Tyene-Sand', 'Manfrey-Martell', 'Gerold-Dayne', 'Timoth', 'Obara-Sand', 'Morra', 'Sylva-Santagar', 'Arianne-Martell', 'Ricasso', 'Cedra'})

In [60]:
louvain = {4: ['Addam-Marbrand', 'Cersei-Lannister', 'Gyles-Rosby', 'Jalabhar-Xho', 'Kevan-Lannister', 'Tyrion-Lannister', 'Varys', 'Alayaya', 'Bronn', 'Dancy', 'Alerie-Hightower', 'Mace-Tyrell', 'Margaery-Tyrell', 'Alla-Tyrell', 'Elinor-Tyrell', 'Megga-Tyrell', 'Osney-Kettleblack', 'Lancel-Lannister', 'Arwyn-Oakheart', 'Randyll-Tarly', 'Tommen-Baratheon', 'Aurane-Waters', 'Harys-Swyft', 'Orton-Merryweather', 'Paxter-Redwyne', 'Balman-Byrch', 'Falyse-Stokeworth', 'Boros-Blount', 'Osmund-Kettleblack', 'Ben-Plumm', 'Pycelle', 'Kasporio', 'Penny', 'Tybero-Istarion', 'Blue-Bard', 'Lambert-Turnberry', 'Qyburn', 'Tallad', 'Tanda-Stokeworth', 'Brella', 'Shae', 'Chiggen', 'Jacelyn-Bywater', 'Lollys-Stokeworth', 'Shagga', 'Timett', 'Willis-Wode', 'Butterbumps', 'Olenna-Redwyne', 'Mathis-Rowan', 'Dorcas', 'Garth-Tyrell', 'High-Septon-(Tyrions)', 'High-Sparrow', 'Jocelyn-Swyft', 'Maegor-I-Targaryen', 'Maggy', 'Mark-Mullendore', 'Melara-Hetherspoon', 'Moelle', 'Moon-Boy', 'Osfryd-Kettleblack', 'Preston-Greenfield', 'Raynard', 'Robert-Strong', 'Scolera', 'Senelle', 'Taena-of-Myr', 'Unella', 'Vylarr', 'Chataya', 'Chella', 'Conn', 'Coratt', 'Dolf', 'Galyeon-of-Cuy', 'Gunthor-son-of-Gurn', 'Jaehaerys-I-Targaryen', 'Josmyn-Peckledon', 'Pia', 'Ronnet-Connington', 'Jeyne-Farman', 'Jon-Fossoway', 'Nurse', 'Jyck', 'Morrec', 'Kurleket', 'Leo-Lefford', 'Symon-Silver-Tongue', 'Mord', 'Morgo', 'Yezzan-zo-Qaggaz', 'Pate-(whipping-boy)', 'Sweets', 'Puckens', 'Shortear', 'Torbert', 'Scar', 'Ulf-son-of-Umar', 'Shrouded-Lord'], 1: ['Brynden-Tully', 'Jaime-Lannister', 'Lyle-Crakehall', 'Tywin-Lannister', 'Aegon-Frey-(son-of-Stevron)', 'Catelyn-Stark', 'Stevron-Frey', 'Walder-Frey', 'Elia-Martell', 'Aemon-Targaryen-(Dragonknight)', 'Brienne-of-Tarth', 'Amerei-Frey', 'Vargo-Hoat', 'Elmar-Frey', 'Rorge', 'Ashara-Dayne', 'Bonifer-Hasty', 'Brenett', 'Clarence-Crabb', 'Creighton-Longbough', 'Edmure-Tully', 'Elder-Brother', 'Emmon-Cuy', 'Hyle-Hunt', 'Illifer', 'Jeyne-Heddle', 'Lysa-Arryn', 'Meribald', 'Narbert', 'Pyg', 'Robar-Royce', 'Selwyn-Tarth', 'Shagwell', 'Timeon', 'Vyman', 'Walton', 'Vardis-Egen', 'Daven-Lannister', 'Edwyn-Frey', 'Emmon-Frey', 'Nestor-Royce', 'Ryman-Frey', 'Donnel-Waynwood', 'Eon-Hunter', 'Jason-Mallister', 'Jeyne-Westerling', 'Lothar-Frey', 'Marq-Piper', 'Masha-Heddle', 'Moreo-Tumitis', 'Mychel-Redfort', 'Robin-Ryger', 'Roslin-Frey', 'Tytos-Blackwood', 'Clement-Piper', 'Karyl-Vance', 'Shadrich', 'Robett-Glover', 'Forley-Prester', 'Walder-Rivers', 'Genna-Lannister', 'Garrett-Paege', 'Gawen-Westerling', 'Hoster-Blackwood', 'Hugo-Vance', 'Jonos-Bracken', 'Joy-Hill', 'Lewys-Piper', 'Rennifer-Longwaters', 'Shitmouth', 'Urswyck', 'Wylis-Manderly', 'Zollo', 'Joanna-Lannister', 'Willem-Lannister', 'Merrett-Frey', 'Martyn-Lannister', 'Nage', 'Olyvar-Frey', 'Petyr-Frey', 'Rugen', 'Tion-Frey'], 6: ['Joffrey-Baratheon', 'Aegon-I-Targaryen', 'Eddard-Stark', 'Harren-Hoare', 'Nan', 'Rhaenys-Targaryen', 'Torrhen-Stark', 'Visenya-Targaryen', 'Robert-Baratheon', 'Arya-Stark', 'Aerys-I-Targaryen', 'Harwin', 'Amabel', 'Harra', 'Amory-Lorch', 'Chiswyck', 'Dunsen', 'Gendry', 'Gregor-Clegane', 'Hot-Pie', 'Ilyn-Payne', 'Meryn-Trant', 'Polliver', 'Rafford', 'Sandor-Clegane', 'Tickler', 'Yoren', 'Anguy', 'Beric-Dondarrion', 'Greenbeard', 'Husband', 'Lem', 'Sharna', 'Tom-of-Sevenstreams', 'Balon-Swann', 'Aron-Santagar', 'Bellegere-Otherys', 'Biter', 'Brea', 'Brusco', 'Denyo-Terys', 'Desmond', 'Dobber', 'Jaqen-Hghar', 'Kindly-Man', 'Koss', 'Lommy-Greenhands', 'Mad-Huntsman', 'Meralyn', 'Mycah', 'Pate-(Old)', 'Praed', 'Ravella-Swann', 'Syrio-Forel', 'Talea', 'Ternesio-Terys', 'Thoros-of-Myr', 'Umma', 'Vayon-Poole', 'Waif', 'Weasel', 'Weese', 'Woth', 'Yorko-Terys', 'Loras-Tyrell', 'Mandon-Moore', 'Mariya-Darry', 'Cleos-Frey', 'Willow-Heddle', 'Hoster-Tully', 'Tansy', 'Cutjack', 'Kurz', 'Tarber', 'Val', 'Daryn-Hornwood', 'Torrhen-Karstark', 'High-Septon-(fat_one)', 'Jacks', 'Joss', 'Maron-Greyjoy', 'Porther', 'Raymun-Darry', 'Tobho-Mott', 'Tregar', 'Varly', 'Willam-Dustin', 'Wylla', 'Merrit', 'Hallyne', 'Hobber-Redwyne', 'Horas-Redwyne', 'Mebble', 'Jack-Be-Lucky'], 3: ['Oberyn-Martell', 'Anders-Yronwood', 'Andrey-Dalt', 'Arianne-Martell', 'Garin-(orphan)', 'Myrcella-Baratheon', 'Sylva-Santagar', 'Tyene-Sand', 'Areo-Hotah', 'Arys-Oakheart', 'Caleotte', 'Doran-Martell', 'Ellaria-Sand', 'Obara-Sand', 'Cedra', 'Franklyn-Fowler', 'Gerold-Dayne', 'Mellei', 'Morra', 'Ricasso', 'Timoth', 'Daemon-Sand', 'Nymeria-Sand', 'Manfrey-Martell', 'Sarella-Sand'], 5: ['Robb-Stark', 'Aenys-Frey', 'Hosteen-Frey', 'Theon-Greyjoy', 'Wyman-Manderly', 'Murenmure', 'Brandon-Stark', 'Ramsay-Snow', 'Alebelly', 'Bran-Stark', 'Hayhead', 'Mikken', 'Roose-Bolton', 'Arnolf-Karstark', 'Arthor-Karstark', 'Mors-Umber', 'Rickard-Karstark', 'Jeyne-Poole', 'Rickon-Stark', 'Rodrik-Cassel', 'Galbart-Glover', 'Barbrey-Dustin', 'Roger-Ryswell', 'Ben-Bones', 'Benfred-Tallhart', 'Helman-Tallhart', 'Leobald-Tallhart', 'Stygg', 'Luwin', 'Beth-Cassel', 'Chayle', 'Cley-Cerwyn', 'Donella-Hornwood', 'Hali', 'Hallis-Mollen', 'Hodor', 'Howland-Reed', 'Hullen', 'Jon-Umber-(Greatjon)', 'Joseth', 'Leaf', 'Osha', 'Poxy-Tym', 'Reek', 'Rickard-Stark', 'Stafford-Lannister', 'Stiv', 'Walder-Frey-(son-of-Jammos)', 'Walder-Frey-(son-of-Merrett)', 'Lyanna-Stark', 'Captain-Myraham', 'Captain-Myraham-Daughter', 'Dacey-Mormont', 'Desmond-Grell', 'Maege-Mormont', 'Raynald-Westerling', 'Utherydes-Wayn', 'Wendel-Manderly', 'Damon-Dance-for-Me', 'Grunt', 'Skinner', 'Sour-Alyn', 'Danwell-Frey', 'Jared-Frey', 'Drennan', 'Squint', 'Farlen', 'Frenya', 'Holly', 'Halys-Hornwood', 'Harwood-Stout', 'Rowan', 'Hother-Umber', 'Rhaegar-Frey', 'Squirrel', 'Jon-Umber-(Smalljon)', 'Robin-Flint', 'Kyra', 'Luton', 'Martyn-Rivers', 'Palla', 'Perwyn-Frey', 'Yellow-Dick', 'Rollam-Westerling', 'Rolph-Spicer', 'Tristifer-IV-Mudd', 'Walda-Frey-(daughter-of-Merrett)', 'Shadd', 'Werlag', 'Willow-Witch-eye'], 9: ['Daenerys-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Franklyn-Flowers', 'Haldon', 'Harry-Strickland', 'Jon-Connington', 'Lemore', 'Rhaegar-Targaryen', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Rolly-Duckfield', 'Viserys-Targaryen', 'Yandry', 'Ysilla', 'Daeron-II-Targaryen', 'Barristan-Selmy', 'Aggo', 'Belwas', 'Daario-Naharis', 'Drogo', 'Grey-Worm', 'Jhogo', 'Jorah-Mormont', 'Quaro', 'Rakharo', 'Quentyn-Martell', 'Archibald-Yronwood', 'Cletus-Yronwood', 'Gerris-Drinkwater', 'Tattered-Prince', 'Arstan', 'Illyrio-Mopatis', 'Jhiqui', 'Kraznys-mo-Nakloz', 'Mero', 'Bloodbeard', 'Galazza-Galare', 'Grazhar', 'Groleo', 'Hizdahr-zo-Loraq', 'Khrazz', 'Larraq', 'Missandei', 'Reznak-mo-Reznak', 'Skahaz-mo-Kandaq', 'Steelskin', 'Widower', 'Xaro-Xhoan-Daxos', 'Barsena', 'Belaquo', 'Goghor-the-Giant', 'Irri', 'Spotted-Cat', 'Oznak-zo-Pahl', 'Benerro', 'Moqorro', 'Black-Balaq', 'Caggo', 'Lucifer-Long', 'Meris', 'Camarron', 'Cleon', 'Ghael', 'Kedry', 'Cohollo', 'Haggo', 'Qotho', 'Hero', 'Prendahl-na-Ghezn', 'Sallor', 'Doreah', 'Eroeh', 'Ezzara', 'Lynesse-Hightower', 'Lysono-Maar', 'Marselen', 'Mezzara', 'Miklaz', 'Mirri-Maz-Duur', 'Pono', 'Pyat-Pree', 'Qavo-Nogarys', 'Qezza', 'Quaithe', 'Rhaegel-Targaryen', 'Symon-Stripeback', 'Yurkhaz-zo-Yunzak', 'Denzo-Dhan', 'Dick-Straw', 'Hugh-Hungerford', 'Draqaz', 'Jommo', 'Ogo', 'Rhaego', 'Fogo', 'Grazdan-mo-Eraz', 'Myles-Toyne', 'Marghaz-zo-Loraq', 'Nyessos-Vhassar', 'Malaquo-Maegyr', 'Rodrik-Sparr', 'Trystane-Martell', 'Old-Bill-Bone', 'Rhaella-Targaryen'], 8: ['Stannis-Baratheon', 'Alester-Florent', 'Axell-Florent', 'Cortnay-Penrose', 'Davos-Seaworth', 'Lamprey', 'Renly-Baratheon', 'Selyse-Florent', 'Allard-Seaworth', 'Marya-Seaworth', 'Alys-Karstark', 'Melisandre', 'Sigorn', 'Andrew-Estermont', 'Edric-Storm', 'Gerald-Gower', 'Ardrian-Celtigar', 'Monford-Velaryon', 'Porridge', 'Salladhor-Saan', 'Shireen-Baratheon', 'Azor-Ahai', 'Bartimus', 'Benethon-Scales', 'Narbert-Grandison', 'Dick-Crabb', 'Brus-Buckler', 'Malegorn', 'Bryce-Caron', 'Eldon-Estermont', 'Guyard-Morrigen', 'Colen-of-Greenpools', 'Cayn', 'Corliss-Penny', 'Cressen', 'Patchface', 'Pylos', 'Daeron-I-Targaryen', 'Dalla', 'Devan-Seaworth', 'Garth-(Wolfs-Den)', 'Godric-Borrell', 'Guncer-Sunglass', 'Imry-Florent', 'Khorane-Sathmantes', 'Leona-Woolfield', 'Marlon-Manderly', 'Matthos-Seaworth', 'Sybell-Spicer', 'Eustace-Brune', 'Steffon-Varner', 'Patrek-of-Kings-Mountain', 'Wun-Weg-Wun-Dar-Wun', 'Joramun', 'Richard-Horpe', 'Lorent-Caswell', 'Lyanna-Mormont', 'Steffon-Baratheon', 'Roose-Ryswell', 'Theomore', 'William-Foxglove'], 2: ['Aegon-V-Targaryen', 'Aemon-Targaryen-(Maester-Aemon)', 'Alliser-Thorne', 'Bowen-Marsh', 'Chett', 'Clydas', 'Cotter-Pyke', 'Dareon', 'Denys-Mallister', 'Donal-Noye', 'Eddison-Tollett', 'Gilly', 'Grenn', 'Hobb', 'Janos-Slynt', 'Jeor-Mormont', 'Jon-Snow', 'Maekar-I-Targaryen', 'Mance-Rayder', 'Owen', 'Pypar', 'Samwell-Tarly', 'Wynton-Stout', 'Albett', 'Halder', 'Rast', 'Jojen-Reed', 'Allar-Deem', 'Qhorin-Halfhand', 'Arron', 'Emrick', 'Hareth-(Moles-Town)', 'Satin', 'Arson', 'Benjen-Stark', 'Bael-the-Bard', 'Ygritte', 'Bannen', 'Dywen', 'Bedwyck', 'Craster', 'Jaremy-Rykker', 'Waymar-Royce', 'Jack-Bulwer', 'Big-Boil', 'Blane', 'Cellador', 'Othell-Yarwyck', 'Tormund', 'Wick-Whittlestick', 'Brynden-Rivers', 'Coldhands', 'Meera-Reed', 'Brandon-Norrey', 'Torghen-Flint', 'Bump', 'Varamyr', 'Byam-Flint', 'Lark', 'Small-Paul', 'Clubfoot-Karl', 'Mully', 'Dirk', 'Thoren-Smallwood', 'Dalbridge', 'Ebben', 'Lanna-(Happy-Port)', 'Dick-Follard', 'Mag-Mar-Tun-Doh-Weg', 'Dryn', 'Stonesnake', 'Emmett', 'Watt', 'Hop-Robin', 'Errok', 'Grigg', 'Styr', 'Fulk', 'Gared', 'Will-(prologue)', 'Garth-Greyfeather', 'Hairy-Hal', 'Gendel', 'Gorne', 'Gerrick-Kingsblood', 'Raymun-Redbeard', 'Kojja-Mo', 'Kegs', 'Todder', 'Jarl', 'Haggon', 'Hake', 'Leathers', 'Rory', 'Spare-Boot', 'Harma', 'Rattleshirt', 'Weeper', 'Jafer-Flowers', 'Jarman-Buckwell', 'Ottyn-Wythers', 'Matthar', 'Orell', 'Othor', 'Ryk', 'Soren-Shieldbreaker', 'Tom-Barleycorn', 'Toregg', 'Ulmer', 'Zei', 'Softfoot', 'Mallador-Locke', 'Munda', 'Ragwyle', 'Quhuru-Mo', 'Xhondo', 'Theobald', 'Thistle'], 11: ['Alleras', 'Armen', 'Leo-Tyrell', 'Marwyn', 'Mollander', 'Pate-(novice)', 'Roone', 'Gormon-Tyrell', 'Walgrave', 'Quill', 'Rosey'], 10: ['Aeron-Greyjoy', 'Asha-Greyjoy', 'Baelor-Blacktyde', 'Balon-Greyjoy', 'Dagmer', 'Dunstan-Drumm', 'Emmond', 'Euron-Greyjoy', 'Gormond-Goodbrother', 'Gorold-Goodbrother', 'Greydon-Goodbrother', 'Meldred-Merlyn', 'Rus', 'Victarion-Greyjoy', 'Aggar', 'Gariss', 'Gelmarr', 'Gynir', 'Alysane-Mormont', 'Justin-Massey', 'Clayton-Suggs', 'Godry-Farring', 'Grimtongue', 'Hagen', 'Qarl-the-Maid', 'Red-Oarsman', 'Rodrik-Harlaw', 'Rolfe-the-Dwarf', 'Sybelle-Glover', 'Torgon-Greyiron', 'Tristifer-Botley', 'Tycho-Nestoris', 'Wex-Pyke', 'Cromm', 'Hotho-Harlaw', 'Humfrey-Hewett', 'Nute', 'Quellon-Greyjoy', 'Sawane-Botley', 'Murch', 'Lorren', 'Harl', 'Harras-Harlaw', 'Sigfryd-Harlaw', 'Kerwin', 'Longwater-Pyke', 'Urzen', 'Lucas-Codd', 'Torwold-Browntooth', 'Ralf-Stonehouse', 'Ralf-(Limper)', 'Talbert-Serry', 'Wulfe'], 7: ['Aerys-II-Targaryen', 'Arthur-Dayne', 'Denys-Darklyn', 'Elbert-Arryn', 'Gerold-Hightower', 'Jon-Arryn', 'Owen-Merryweather', 'Rossart', 'Alys-Arryn', 'Elys-Waynwood', 'Lewyn-Martell', 'Smiling-Knight', 'Oswell-Whent', 'Hugh', 'Jonothor-Darry', 'Simon-Toyne'], 0: ['Sansa-Stark', 'Alyn', 'Jory-Cassel', 'Tomard', 'Anya-Waynwood', 'Benedar-Belmore', 'Gilwood-Hunter', 'Harrold-Hardyng', 'Horton-Redfort', 'Petyr-Baelish', 'Symond-Templeton', 'Yohn-Royce', 'Mordane', 'Baelor-I-Targaryen', 'Ballabar', 'Frenken', 'Podrick-Payne', 'Dontos-Hollard', 'Marillion', 'Colemon', 'Mya-Stone', 'Robert-Arryn', 'Cedric-Payne', 'Garlan-Tyrell', 'Willas-Tyrell', 'Gretchel', 'Maddy', 'Lothor-Brune', 'Wyl-(guard)', 'Florian-the-Fool', 'Jonquil', 'Leonette-Fossoway', 'Gerold-Grafton', 'Terrance-Lynderly', 'Grisel', 'Gyles-Grafton', 'Heward', 'Tysha', 'Kella', 'Lyn-Corbray', 'Lyonel-Corbray', 'Myranda-Royce', 'Oswell-Kettleblack']}

In [61]:
bron_kerbosch = [{'Walton', 'Qyburn', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Roose-Bolton'}, {'Walton', 'Nage'}, {'Joanna-Lannister', 'Tywin-Lannister'}, {'Craster', 'Samwell-Tarly', 'Dywen', 'Bedwyck', 'Grenn', 'Eddison-Tollett'}, {'Qyburn', 'Osmund-Kettleblack', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Cersei-Lannister'}, {'Qyburn', 'Osmund-Kettleblack', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Pycelle', 'Cersei-Lannister'}, {'Jocelyn-Swyft', 'Qyburn', 'Cersei-Lannister', 'Osmund-Kettleblack'}, {'Moon-Boy', 'Cersei-Lannister', 'Lancel-Lannister', 'Osmund-Kettleblack'}, {'Osmund-Kettleblack', 'Tyrion-Lannister', 'Lancel-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister'}, {'Osmund-Kettleblack', 'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell'}, {'Osmund-Kettleblack', 'Tommen-Baratheon', 'Sansa-Stark', 'Boros-Blount', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Loras-Tyrell'}, {'Tyrion-Lannister', 'Osmund-Kettleblack', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Boros-Blount', 'Meryn-Trant', 'Cersei-Lannister'}, {'Tyrion-Lannister', 'Osmund-Kettleblack', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Pycelle', 'Cersei-Lannister'}, {'Osmund-Kettleblack', 'Tommen-Baratheon', 'Boros-Blount', 'Osney-Kettleblack', 'Cersei-Lannister'}, {'Osmund-Kettleblack', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Boros-Blount', 'Meryn-Trant', 'Balon-Swann'}, {'Osmund-Kettleblack', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Boros-Blount', 'Loras-Tyrell', 'Balon-Swann'}, {'Osfryd-Kettleblack', 'Osney-Kettleblack', 'Cersei-Lannister', 'Osmund-Kettleblack'}, {'Osmund-Kettleblack', 'Tommen-Baratheon', 'Margaery-Tyrell', 'Osney-Kettleblack', 'Cersei-Lannister'}, {'Puckens', 'Shortear'}, {'Lyle-Crakehall', 'Brynden-Tully', 'Addam-Marbrand', 'Jaime-Lannister'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Robb-Stark', 'Walder-Frey', 'Brynden-Tully', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Theon-Greyjoy', 'Brynden-Tully', 'Arya-Stark', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Walder-Frey', 'Jaime-Lannister', 'Robb-Stark', 'Theon-Greyjoy', 'Brynden-Tully', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Brynden-Tully', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Tywin-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brynden-Tully', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Addam-Marbrand', 'Tywin-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brynden-Tully'}, {'Emmon-Frey', 'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Brynden-Tully'}, {'Emmon-Frey', 'Edmure-Tully', 'Jaime-Lannister', 'Ryman-Frey', 'Brynden-Tully'}, {'Robb-Stark', 'Edmure-Tully', 'Jaime-Lannister', 'Ryman-Frey', 'Brynden-Tully', 'Catelyn-Stark'}, {'Ryman-Frey', 'Edwyn-Frey', 'Brynden-Tully', 'Jaime-Lannister'}, {'Edmure-Tully', 'Jaime-Lannister', 'Robb-Stark', 'Hoster-Tully', 'Brynden-Tully', 'Catelyn-Stark'}, {'Edmure-Tully', 'Jaime-Lannister', 'Hoster-Tully', 'Brynden-Tully', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Emmon-Frey', 'Brynden-Tully', 'Daven-Lannister', 'Jaime-Lannister'}, {'Rickard-Karstark', 'Jaime-Lannister', 'Robb-Stark', 'Brynden-Tully', 'Catelyn-Stark'}, {'Brynden-Tully', 'Nestor-Royce', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Jon-Snow', 'Wick-Whittlestick', 'Bowen-Marsh'}, {'Grey-Worm', 'Daenerys-Targaryen', 'Daario-Naharis', 'Barristan-Selmy', 'Aggo', 'Jorah-Mormont'}, {'Drogo', 'Daenerys-Targaryen', 'Daario-Naharis', 'Jhogo', 'Aggo', 'Rakharo', 'Jorah-Mormont'}, {'Daenerys-Targaryen', 'Daario-Naharis', 'Jhogo', 'Barristan-Selmy', 'Aggo', 'Jorah-Mormont'}, {'Daenerys-Targaryen', 'Jhogo', 'Barristan-Selmy', 'Aggo', 'Jorah-Mormont', 'Belwas'}, {'Illyrio-Mopatis', 'Irri', 'Daenerys-Targaryen', 'Viserys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Irri', 'Doreah', 'Daenerys-Targaryen', 'Viserys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Illyrio-Mopatis', 'Robert-Baratheon', 'Tyrion-Lannister', 'Daenerys-Targaryen', 'Viserys-Targaryen', 'Jorah-Mormont'}, {'Illyrio-Mopatis', 'Robert-Baratheon', 'Daenerys-Targaryen', 'Viserys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Robert-Baratheon', 'Daenerys-Targaryen', 'Viserys-Targaryen', 'Rhaegar-Targaryen', 'Jorah-Mormont'}, {'Irri', 'Daario-Naharis', 'Missandei', 'Daenerys-Targaryen', 'Jhiqui', 'Jorah-Mormont'}, {'Irri', 'Doreah', 'Jhiqui', 'Daenerys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Rakharo', 'Irri', 'Daario-Naharis', 'Jhiqui', 'Daenerys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Belwas', 'Daenerys-Targaryen', 'Jhiqui', 'Jorah-Mormont', 'Arstan'}, {'Daenerys-Targaryen', 'Penny', 'Jorah-Mormont', 'Tyrion-Lannister'}, {'Daenerys-Targaryen', 'Illyrio-Mopatis', 'Arstan', 'Jorah-Mormont', 'Belwas'}, {'Drogo', 'Irri', 'Daario-Naharis', 'Jhogo', 'Daenerys-Targaryen', 'Rakharo', 'Jorah-Mormont'}, {'Qotho', 'Mirri-Maz-Duur', 'Daenerys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Mirri-Maz-Duur', 'Jhogo', 'Daenerys-Targaryen', 'Drogo', 'Jorah-Mormont'}, {'Daenerys-Targaryen', 'Pono', 'Drogo', 'Jorah-Mormont'}, {'Daenerys-Targaryen', 'Mero', 'Arstan', 'Jorah-Mormont'}, {'Daenerys-Targaryen', 'Rhaegar-Targaryen', 'Arstan', 'Jorah-Mormont'}, {'Robert-Baratheon', 'Tyrion-Lannister', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Jorah-Mormont'}, {'Daario-Naharis', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Missandei', 'Jorah-Mormont'}, {'Robert-Baratheon', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Rhaegar-Targaryen', 'Jorah-Mormont'}, {'Grey-Worm', 'Irri', 'Daario-Naharis', 'Daenerys-Targaryen', 'Jorah-Mormont'}, {'Robert-Baratheon', 'Jeor-Mormont', 'Jorah-Mormont', 'Tyrion-Lannister'}, {'Penny', 'Nurse', 'Jorah-Mormont', 'Tyrion-Lannister'}, {'Leo-Lefford', 'Tyrion-Lannister'}, {'Daenerys-Targaryen', 'Pyat-Pree', 'Xaro-Xhoan-Daxos'}, {'Daenerys-Targaryen', 'Xaro-Xhoan-Daxos', 'Barristan-Selmy'}, {'Malegorn', 'Brus-Buckler', 'Narbert-Grandison', 'Selyse-Florent'}, {'Malegorn', 'Jon-Snow', 'Narbert-Grandison', 'Selyse-Florent'}, {'Bonifer-Hasty', 'Jaime-Lannister'}, {'Jommo', 'Ogo', 'Drogo'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Val', 'Jon-Snow', 'Dalla'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Melisandre', 'Jon-Snow', 'Dalla'}, {'Mance-Rayder', 'Melisandre', 'Jon-Snow', 'Dalla', 'Gilly'}, {'Alys-Arryn', 'Elys-Waynwood'}, {'Alys-Arryn', 'Jon-Arryn'}, {'Humfrey-Hewett', 'Victarion-Greyjoy', 'Euron-Greyjoy'}, {'Quentyn-Martell', 'Anders-Yronwood'}, {'Oberyn-Martell', 'Anders-Yronwood'}, {'Franklyn-Flowers', 'Black-Balaq'}, {'Jeyne-Farman', 'Melara-Hetherspoon'}, {'Robb-Stark', 'Rollam-Westerling', 'Raynald-Westerling'}, {'Nyessos-Vhassar', 'Jon-Connington'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Tywin-Lannister', 'Rhaegar-Targaryen', 'Jon-Connington'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Tywin-Lannister', 'Tyrion-Lannister', 'Jon-Connington'}, {'Tywin-Lannister', 'Tyrion-Lannister', 'Varys', 'Robert-Baratheon', 'Jon-Connington'}, {'Robert-Baratheon', 'Tywin-Lannister', 'Rhaegar-Targaryen', 'Jon-Connington'}, {'Rolly-Duckfield', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Tyrion-Lannister', 'Haldon', 'Yandry', 'Jon-Connington', 'Lemore'}, {'Rolly-Duckfield', 'Tyrion-Lannister', 'Haldon', 'Illyrio-Mopatis', 'Jon-Connington'}, {'Daenerys-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Rhaegar-Targaryen', 'Jon-Connington'}, {'Daenerys-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Tyrion-Lannister', 'Jon-Connington'}, {'Robert-Baratheon', 'Tyrion-Lannister', 'Daenerys-Targaryen', 'Illyrio-Mopatis', 'Jon-Connington'}, {'Robert-Baratheon', 'Tyrion-Lannister', 'Varys', 'Daenerys-Targaryen', 'Jon-Connington'}, {'Daenerys-Targaryen', 'Rhaegar-Targaryen', 'Robert-Baratheon', 'Jon-Connington'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Jon-Connington', 'Franklyn-Flowers'}, {'Myles-Toyne', 'Harry-Strickland', 'Jon-Connington'}, {'Harry-Strickland', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Jon-Connington'}, {'Alayaya', 'Cersei-Lannister', 'Bronn', 'Tyrion-Lannister'}, {'Alayaya', 'Dancy'}, {'Gunthor-son-of-Gurn', 'Tyrion-Lannister'}, {'Balon-Greyjoy', 'Aeron-Greyjoy', 'Asha-Greyjoy', 'Theon-Greyjoy', 'Victarion-Greyjoy', 'Euron-Greyjoy'}, {'Balon-Greyjoy', 'Aeron-Greyjoy', 'Asha-Greyjoy', 'Dagmer', 'Theon-Greyjoy'}, {'Balon-Greyjoy', 'Aeron-Greyjoy', 'Victarion-Greyjoy', 'Euron-Greyjoy', 'Baelor-Blacktyde'}, {'Theon-Greyjoy', 'Aeron-Greyjoy', 'Murenmure'}, {'Victarion-Greyjoy', 'Gorold-Goodbrother', 'Aeron-Greyjoy'}, {'Gormond-Goodbrother', 'Aeron-Greyjoy'}, {'Greydon-Goodbrother', 'Aeron-Greyjoy'}, {'Victarion-Greyjoy', 'Aeron-Greyjoy', 'Meldred-Merlyn'}, {'Rus', 'Aeron-Greyjoy'}, {'Victarion-Greyjoy', 'Dunstan-Drumm', 'Aeron-Greyjoy'}, {'Emmond', 'Aeron-Greyjoy'}, {'Alys-Karstark', 'Jon-Snow', 'Sigorn', 'Melisandre'}, {'Jon-Snow', 'Stannis-Baratheon', 'Sigorn', 'Melisandre'}, {'Mollander', 'Pate-(novice)', 'Armen', 'Alleras'}, {'Mollander', 'Alleras', 'Roone', 'Armen'}, {'Randyll-Tarly', 'Stannis-Baratheon', 'Robett-Glover'}, {'Randyll-Tarly', 'Helman-Tallhart', 'Robett-Glover'}, {'Robett-Glover', 'Galbart-Glover'}, {'Stannis-Baratheon', 'Robett-Glover', 'Davos-Seaworth'}, {'Rorge', 'Robett-Glover', 'Vargo-Hoat'}, {'Kegs', 'Spare-Boot', 'Jon-Snow', 'Satin', 'Owen'}, {'Kegs', 'Jon-Snow', 'Grenn', 'Pypar', 'Owen'}, {'Kegs', 'Pypar', 'Jon-Snow', 'Satin', 'Owen'}, {'Aemon-Targaryen-(Maester-Aemon)', 'Jon-Snow', 'Grenn', 'Pypar', 'Owen'}, {'Robert-Baratheon', 'Jon-Snow', 'Aemon-Targaryen-(Maester-Aemon)', 'Owen'}, {'Jon-Snow', 'Aemon-Targaryen-(Maester-Aemon)', 'Hobb', 'Owen'}, {'Spare-Boot', 'Mully', 'Jon-Snow', 'Satin', 'Owen'}, {'Jon-Snow', 'Hobb', 'Owen', 'Mully'}, {'Prendahl-na-Ghezn', 'Daario-Naharis', 'Sallor'}, {'Nyessos-Vhassar', 'Malaquo-Maegyr'}, {'Tycho-Nestoris', 'Stannis-Baratheon', 'Jon-Snow'}, {'Tycho-Nestoris', 'Stannis-Baratheon', 'Asha-Greyjoy'}, {'Shae', 'Tanda-Stokeworth', 'Lollys-Stokeworth', 'Tyrion-Lannister'}, {'Gyles-Rosby', 'Tanda-Stokeworth', 'Lollys-Stokeworth', 'Tyrion-Lannister'}, {'Tanda-Stokeworth', 'Tyrion-Lannister', 'Shae', 'Sansa-Stark', 'Cersei-Lannister'}, {'Tanda-Stokeworth', 'Tyrion-Lannister', 'Sansa-Stark', 'Gyles-Rosby', 'Boros-Blount', 'Cersei-Lannister'}, {'Grazhar', 'Qezza'}, {'Daenerys-Targaryen', 'Hizdahr-zo-Loraq', 'Qezza'}, {'Daenerys-Targaryen', 'Qezza', 'Mezzara'}, {'Daenerys-Targaryen', 'Miklaz', 'Qezza'}, {'Stannis-Baratheon', 'William-Foxglove'}, {'Ryman-Frey', 'Stevron-Frey', 'Robb-Stark', 'Catelyn-Stark'}, {'Ryman-Frey', 'Petyr-Frey'}, {'Robb-Stark', 'Jaime-Lannister', 'Ryman-Frey', 'Roose-Bolton', 'Catelyn-Stark'}, {'Ryman-Frey', 'Robb-Stark', 'Walder-Rivers', 'Jaime-Lannister'}, {'Ryman-Frey', 'Robb-Stark', 'Olyvar-Frey'}, {'Ryman-Frey', 'Edwyn-Frey', 'Walder-Rivers', 'Jaime-Lannister'}, {'Grigg', 'Jon-Snow', 'Errok'}, {'Grigg', 'Jon-Snow', 'Jarl'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Cersei-Lannister', 'Stannis-Baratheon', 'Jaime-Lannister', 'Robert-Baratheon'}, {'Robert-Baratheon', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Catelyn-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Joffrey-Baratheon'}, {'Robert-Baratheon', 'Sansa-Stark', 'Jon-Snow', 'Cersei-Lannister', 'Stannis-Baratheon', 'Janos-Slynt', 'Tyrion-Lannister', 'Eddard-Stark', 'Joffrey-Baratheon'}, {'Stannis-Baratheon', 'Robert-Baratheon', 'Jon-Snow', 'Val'}, {'Stannis-Baratheon', 'Val', 'Jon-Snow', 'Selyse-Florent', 'Shireen-Baratheon'}, {'Stannis-Baratheon', 'Axell-Florent', 'Jon-Snow', 'Selyse-Florent', 'Shireen-Baratheon'}, {'Stannis-Baratheon', 'Joffrey-Baratheon', 'Jon-Snow', 'Selyse-Florent', 'Shireen-Baratheon'}, {'Mance-Rayder', 'Samwell-Tarly', 'Stannis-Baratheon', 'Alliser-Thorne', 'Aemon-Targaryen-(Maester-Aemon)', 'Janos-Slynt', 'Jon-Snow', 'Bowen-Marsh'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Aemon-Targaryen-(Maester-Aemon)', 'Donal-Noye', 'Jon-Snow', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Stannis-Baratheon', 'Aemon-Targaryen-(Maester-Aemon)', 'Cotter-Pyke', 'Jon-Snow', 'Bowen-Marsh'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Alliser-Thorne', 'Janos-Slynt', 'Joffrey-Baratheon', 'Jon-Snow'}, {'Stannis-Baratheon', 'Theon-Greyjoy', 'Jon-Snow', 'Mance-Rayder'}, {'Samwell-Tarly', 'Mance-Rayder', 'Stannis-Baratheon', 'Melisandre', 'Jon-Snow'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Godry-Farring', 'Melisandre', 'Jon-Snow'}, {'Mance-Rayder', 'Stannis-Baratheon', 'Melisandre', 'Joffrey-Baratheon', 'Jon-Snow'}, {'Samwell-Tarly', 'Stannis-Baratheon', 'Aemon-Targaryen-(Maester-Aemon)', 'Clydas', 'Jon-Snow'}, {'Stannis-Baratheon', 'Samwell-Tarly', 'Robb-Stark', 'Jon-Snow'}, {'Stannis-Baratheon', 'Melisandre', 'Axell-Florent', 'Jon-Snow', 'Selyse-Florent'}, {'Stannis-Baratheon', 'Melisandre', 'Joffrey-Baratheon', 'Jon-Snow', 'Selyse-Florent'}, {'Stannis-Baratheon', 'Devan-Seaworth', 'Jon-Snow', 'Melisandre'}, {'Stannis-Baratheon', 'Jory-Cassel', 'Theon-Greyjoy', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Jon-Snow'}, {'Stannis-Baratheon', 'Theon-Greyjoy', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Jon-Snow', 'Catelyn-Stark'}, {'Stannis-Baratheon', 'Alliser-Thorne', 'Janos-Slynt', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Jon-Snow'}, {'Stannis-Baratheon', 'Aemon-Targaryen-(Maester-Aemon)', 'Janos-Slynt', 'Robert-Baratheon', 'Jon-Snow'}, {'Stannis-Baratheon', 'Cotter-Pyke', 'Jon-Snow', 'Selyse-Florent'}, {'Mance-Rayder', 'Rattleshirt', 'Styr', 'Jarl', 'Ygritte', 'Jon-Snow'}, {'Mance-Rayder', 'Rattleshirt', 'Styr', 'Jarl', 'Jon-Snow', 'Harma'}, {'Mance-Rayder', 'Val', 'Jon-Snow', 'Jarl'}, {'Mance-Rayder', 'Rattleshirt', 'Melisandre', 'Ygritte', 'Jon-Snow'}, {'Mance-Rayder', 'Melisandre', 'Joffrey-Baratheon', 'Jon-Snow', 'Arya-Stark'}, {'Mance-Rayder', 'Samwell-Tarly', 'Melisandre', 'Jon-Snow', 'Gilly'}, {'Samwell-Tarly', 'Jon-Snow', 'Hobb', 'Melisandre'}, {'Jon-Snow', 'Gerrick-Kingsblood'}, {'Mag-Mar-Tun-Doh-Weg', 'Jon-Snow', 'Donal-Noye'}, {'Mance-Rayder', 'Rattleshirt', 'Styr', 'Tormund', 'Jon-Snow', 'Ygritte'}, {'Mance-Rayder', 'Rattleshirt', 'Qhorin-Halfhand', 'Jon-Snow', 'Ygritte'}, {'Rattleshirt', 'Tormund', 'Jon-Snow', 'Ygritte', 'Ryk'}, {'Ygritte', 'Jon-Snow', 'Qhorin-Halfhand', 'Stonesnake'}, {'Ygritte', 'Big-Boil', 'Jon-Snow'}, {'Ygritte', 'Jon-Snow', 'Orell'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Janos-Slynt', 'Eddison-Tollett', 'Jon-Snow', 'Hobb', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Clydas', 'Jon-Snow', 'Hobb'}, {'Clydas', 'Jon-Snow', 'Hobb', 'Mully'}, {'Eddison-Tollett', 'Jon-Snow', 'Hobb', 'Mully'}, {'Janos-Slynt', 'Othell-Yarwyck', 'Eddison-Tollett', 'Jon-Snow', 'Hobb', 'Bowen-Marsh'}, {'Kegs', 'Spare-Boot', 'Jon-Snow', 'Hareth-(Moles-Town)', 'Satin'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Janos-Slynt', 'Denys-Mallister', 'Eddison-Tollett', 'Jon-Snow', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Denys-Mallister', 'Pypar', 'Cotter-Pyke', 'Eddison-Tollett', 'Jon-Snow'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Denys-Mallister', 'Cotter-Pyke', 'Eddison-Tollett', 'Jon-Snow', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Jeor-Mormont', 'Aemon-Targaryen-(Maester-Aemon)', 'Grenn', 'Jon-Snow', 'Eddison-Tollett'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Pypar', 'Grenn', 'Jon-Snow', 'Eddison-Tollett'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Jeor-Mormont', 'Jon-Snow', 'Eddison-Tollett', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Eddison-Tollett', 'Jon-Snow', 'Gilly'}, {'Mance-Rayder', 'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Alliser-Thorne', 'Jeor-Mormont', 'Jon-Snow', 'Bowen-Marsh'}, {'Grenn', 'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Alliser-Thorne', 'Jeor-Mormont', 'Jon-Snow'}, {'Mance-Rayder', 'Aemon-Targaryen-(Maester-Aemon)', 'Donal-Noye', 'Jeor-Mormont', 'Jon-Snow', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Jeor-Mormont', 'Jon-Snow', 'Chett'}, {'Robert-Baratheon', 'Jeor-Mormont', 'Jon-Snow', 'Aemon-Targaryen-(Maester-Aemon)'}, {'Mance-Rayder', 'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Jon-Snow', 'Gilly'}, {'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Dareon', 'Jon-Snow', 'Gilly'}, {'Mance-Rayder', 'Samwell-Tarly', 'Aemon-Targaryen-(Maester-Aemon)', 'Alliser-Thorne', 'Pypar', 'Jon-Snow'}, {'Samwell-Tarly', 'Alliser-Thorne', 'Aemon-Targaryen-(Maester-Aemon)', 'Pypar', 'Grenn', 'Jon-Snow'}, {'Jon-Snow', 'Hareth-(Moles-Town)', 'Rory'}, {'Jaremy-Rykker', 'Jeor-Mormont', 'Jon-Snow', 'Thoren-Smallwood'}, {'Jaremy-Rykker', 'Jeor-Mormont', 'Jon-Snow', 'Benjen-Stark'}, {'Robb-Stark', 'Theon-Greyjoy', 'Rickon-Stark', 'Jon-Snow', 'Catelyn-Stark', 'Bran-Stark', 'Luwin', 'Tyrion-Lannister', 'Eddard-Stark', 'Rodrik-Cassel'}, {'Robb-Stark', 'Theon-Greyjoy', 'Rickon-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Bran-Stark', 'Tyrion-Lannister', 'Eddard-Stark', 'Rodrik-Cassel'}, {'Robb-Stark', 'Rickon-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Bran-Stark', 'Tyrion-Lannister', 'Eddard-Stark'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Cersei-Lannister', 'Bran-Stark', 'Robert-Baratheon'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Cersei-Lannister', 'Jaime-Lannister', 'Robert-Baratheon'}, {'Robert-Baratheon', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Bran-Stark', 'Tyrion-Lannister', 'Eddard-Stark', 'Joffrey-Baratheon'}, {'Robert-Baratheon', 'Robb-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Jory-Cassel', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Jon-Snow', 'Arya-Stark', 'Rodrik-Cassel'}, {'Jory-Cassel', 'Theon-Greyjoy', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Jon-Snow', 'Arya-Stark', 'Rodrik-Cassel'}, {'Bran-Stark', 'Tyrion-Lannister', 'Benjen-Stark', 'Robb-Stark', 'Robert-Baratheon', 'Jon-Snow', 'Arya-Stark', 'Cersei-Lannister'}, {'Bran-Stark', 'Tyrion-Lannister', 'Benjen-Stark', 'Robb-Stark', 'Theon-Greyjoy', 'Jon-Snow', 'Arya-Stark'}, {'Robb-Stark', 'Theon-Greyjoy', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Rodrik-Cassel'}, {'Robb-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Bran-Stark', 'Tyrion-Lannister', 'Eddard-Stark', 'Rodrik-Cassel', 'Joffrey-Baratheon'}, {'Robb-Stark', 'Jon-Snow', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Rodrik-Cassel', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Robb-Stark', 'Jon-Snow', 'Arya-Stark', 'Nan'}, {'Bran-Stark', 'Tyrion-Lannister', 'Yoren', 'Robb-Stark', 'Joffrey-Baratheon', 'Jon-Snow', 'Arya-Stark'}, {'Bran-Stark', 'Tyrion-Lannister', 'Yoren', 'Robb-Stark', 'Jon-Snow', 'Arya-Stark', 'Benjen-Stark'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Jon-Snow', 'Nan'}, {'Robb-Stark', 'Samwell-Tarly', 'Jon-Snow', 'Bran-Stark'}, {'Bran-Stark', 'Luwin', 'Tyrion-Lannister', 'Benjen-Stark', 'Robb-Stark', 'Theon-Greyjoy', 'Jon-Snow'}, {'Mully', 'Jon-Snow', 'Fulk'}, {'Mordane', 'Robert-Baratheon', 'Eddard-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Spare-Boot', 'Mully', 'Jon-Snow', 'Hareth-(Moles-Town)', 'Satin'}, {'Jon-Snow', 'Hareth-(Moles-Town)', 'Leathers'}, {'Jon-Snow', 'Hareth-(Moles-Town)', 'Hop-Robin', 'Emmett'}, {'Jon-Snow', 'Styr', 'Errok'}, {'Janos-Slynt', 'Denys-Mallister', 'Othell-Yarwyck', 'Eddison-Tollett', 'Jon-Snow', 'Bowen-Marsh'}, {'Denys-Mallister', 'Othell-Yarwyck', 'Pypar', 'Cotter-Pyke', 'Eddison-Tollett', 'Jon-Snow'}, {'Denys-Mallister', 'Othell-Yarwyck', 'Cotter-Pyke', 'Eddison-Tollett', 'Jon-Snow', 'Bowen-Marsh'}, {'Samwell-Tarly', 'Lark', 'Jeor-Mormont', 'Jon-Snow', 'Chett'}, {'Mance-Rayder', 'Samwell-Tarly', 'Alliser-Thorne', 'Qhorin-Halfhand', 'Jeor-Mormont', 'Jon-Snow'}, {'Mance-Rayder', 'Qhorin-Halfhand', 'Jeor-Mormont', 'Jon-Snow', 'Thoren-Smallwood'}, {'Mance-Rayder', 'Qhorin-Halfhand', 'Donal-Noye', 'Jeor-Mormont', 'Jon-Snow'}, {'Samwell-Tarly', 'Qhorin-Halfhand', 'Jeor-Mormont', 'Jon-Snow', 'Chett'}, {'Dalbridge', 'Jon-Snow', 'Qhorin-Halfhand', 'Ebben'}, {'Ebben', 'Jon-Snow', 'Qhorin-Halfhand', 'Stonesnake'}, {'Eddison-Tollett', 'Jon-Snow', 'Satin', 'Mully'}, {'Craster', 'Samwell-Tarly', 'Dywen', 'Jeor-Mormont', 'Grenn', 'Jon-Snow', 'Eddison-Tollett'}, {'Othell-Yarwyck', 'Jeor-Mormont', 'Jon-Snow', 'Eddison-Tollett', 'Bowen-Marsh'}, {'Eddison-Tollett', 'Jon-Snow', 'Satin', 'Pypar'}, {'Eddison-Tollett', 'Jon-Snow', 'Satin', 'Bowen-Marsh'}, {'Craster', 'Samwell-Tarly', 'Eddison-Tollett', 'Jon-Snow', 'Gilly'}, {'Eddison-Tollett', 'Jon-Snow', 'Janos-Slynt', 'Emmett'}, {'Halder', 'Pypar', 'Grenn', 'Jon-Snow', 'Todder'}, {'Tormund', 'Toregg', 'Jon-Snow', 'Leathers'}, {'Jon-Snow', 'Leathers', 'Wun-Weg-Wun-Dar-Wun'}, {'Jon-Snow', 'Cellador', 'Bowen-Marsh', 'Othell-Yarwyck'}, {'Mance-Rayder', 'Tormund', 'Jon-Snow', 'Bowen-Marsh'}, {'Craster', 'Mance-Rayder', 'Samwell-Tarly', 'Jon-Snow', 'Gilly'}, {'Mance-Rayder', 'Rattleshirt', 'Styr', 'Tormund', 'Jon-Snow', 'Harma'}, {'Mance-Rayder', 'Craster', 'Samwell-Tarly', 'Jeor-Mormont', 'Jon-Snow'}, {'Mance-Rayder', 'Craster', 'Jeor-Mormont', 'Jon-Snow', 'Thoren-Smallwood'}, {'Mance-Rayder', 'Jeor-Mormont', 'Jon-Snow', 'Benjen-Stark'}, {'Mance-Rayder', 'Tormund', 'Jon-Snow', 'Val'}, {'Mance-Rayder', 'Benjen-Stark', 'Theon-Greyjoy', 'Jon-Snow', 'Arya-Stark'}, {'Mance-Rayder', 'Varamyr', 'Jon-Snow', 'Harma'}, {'Mance-Rayder', 'Tormund', 'Jon-Snow', 'Craster'}, {'Samwell-Tarly', 'Jeor-Mormont', 'Jon-Snow', 'Ottyn-Wythers'}, {'Samwell-Tarly', 'Alliser-Thorne', 'Halder', 'Pypar', 'Grenn', 'Jon-Snow'}, {'Samwell-Tarly', 'Rast', 'Alliser-Thorne', 'Halder', 'Pypar', 'Jon-Snow'}, {'Samwell-Tarly', 'Jeor-Mormont', 'Jon-Snow', 'Bran-Stark'}, {'Bran-Stark', 'Tyrion-Lannister', 'Benjen-Stark', 'Robert-Baratheon', 'Jeor-Mormont', 'Jon-Snow'}, {'Bran-Stark', 'Yoren', 'Benjen-Stark', 'Tyrion-Lannister', 'Jeor-Mormont', 'Jon-Snow'}, {'Narbert-Grandison', 'Patrek-of-Kings-Mountain', 'Jon-Snow', 'Selyse-Florent'}, {'Mordane', 'Jory-Cassel', 'Robert-Baratheon', 'Eddard-Stark', 'Sansa-Stark', 'Jon-Snow', 'Arya-Stark', 'Cersei-Lannister'}, {'Ulmer', 'Jon-Snow'}, {'Rattleshirt', 'Harma', 'Tormund', 'Jon-Snow', 'Weeper'}, {'Jon-Snow', 'Albett', 'Rast', 'Halder'}, {'Othor', 'Jon-Snow'}, {'Ottyn-Wythers', 'Jeor-Mormont', 'Jon-Snow', 'Thoren-Smallwood'}, {'Jeor-Mormont', 'Jon-Snow', 'Thoren-Smallwood', 'Jarman-Buckwell'}, {'Jon-Snow', 'Dick-Follard', 'Satin'}, {'Jon-Snow', 'Weeper', 'Othell-Yarwyck'}, {'Jon-Snow', 'Arya-Stark', 'Dareon'}, {'Patrek-of-Kings-Mountain', 'Jon-Snow', 'Selyse-Florent', 'Wun-Weg-Wun-Dar-Wun'}, {'Jeor-Mormont', 'Jon-Snow', 'Alliser-Thorne', 'Tyrion-Lannister'}, {'Jack-Bulwer', 'Hairy-Hal', 'Jon-Snow', 'Garth-Greyfeather'}, {'Soren-Shieldbreaker', 'Jon-Snow'}, {'Jon-Snow', 'Selyse-Florent', 'Satin'}, {'Jon-Snow', 'Gorne', 'Gendel'}, {'Jon-Snow', 'Matthar'}, {'Brandon-Norrey', 'Jon-Snow'}, {'Tom-Barleycorn', 'Jon-Snow'}, {'Jon-Snow', 'Zei'}, {'Jon-Snow', 'Cellador', 'Clydas'}, {'Hake', 'Jon-Snow'}, {'Jon-Snow', 'Arson'}, {'Hot-Pie', 'Arya-Stark', 'Weasel', 'Lommy-Greenhands'}, {'Hot-Pie', 'Yoren', 'Gendry', 'Woth', 'Arya-Stark', 'Lommy-Greenhands'}, {'Catelyn-Stark', 'Mychel-Redfort'}, {'Belaquo', 'Goghor-the-Giant', 'Spotted-Cat', 'Hizdahr-zo-Loraq'}, {'Belaquo', 'Irri', 'Hizdahr-zo-Loraq'}, {'Martyn-Rivers', 'Robb-Stark'}, {'Stannis-Baratheon', 'Dontos-Hollard', 'Tyrion-Lannister', 'Brienne-of-Tarth', 'Joffrey-Baratheon', 'Sansa-Stark'}, {'Stannis-Baratheon', 'Dontos-Hollard', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Petyr-Baelish'}, {'Sansa-Stark', 'Willas-Tyrell', 'Dontos-Hollard', 'Tyrion-Lannister'}, {'Lothor-Brune', 'Sansa-Stark', 'Petyr-Baelish', 'Dontos-Hollard'}, {'Moon-Boy', 'Dontos-Hollard'}, {'Eddard-Stark', 'Tywin-Lannister', 'Robb-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Tywin-Lannister', 'Sansa-Stark', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Barristan-Selmy', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Robb-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Bran-Stark', 'Tyrion-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Barristan-Selmy', 'Varys', 'Joffrey-Baratheon'}, {'Renly-Baratheon', 'Eddard-Stark', 'Cayn'}, {'Renly-Baratheon', 'Stannis-Baratheon', 'Davos-Seaworth', 'Robb-Stark', 'Eddard-Stark', 'Joffrey-Baratheon'}, {'Ramsay-Snow', 'Wyman-Manderly', 'Stannis-Baratheon', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Roose-Bolton'}, {'Tywin-Lannister', 'Stannis-Baratheon', 'Balon-Greyjoy', 'Robert-Baratheon', 'Robb-Stark', 'Eddard-Stark', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Stannis-Baratheon', 'Balon-Greyjoy', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Catelyn-Stark'}, {'Eddard-Stark', 'Tywin-Lannister', 'Sansa-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Lysa-Arryn'}, {'Tywin-Lannister', 'Robb-Stark', 'Theon-Greyjoy', 'Catelyn-Stark', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton'}, {'Tywin-Lannister', 'Robb-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Stannis-Baratheon', 'Jaime-Lannister', 'Robert-Baratheon'}, {'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Stannis-Baratheon', 'Janos-Slynt', 'Tyrion-Lannister', 'Varys', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Jon-Arryn', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Lysa-Arryn'}, {'Eddard-Stark', 'Jon-Arryn', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Varys'}, {'Eddard-Stark', 'Jon-Arryn', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Varys'}, {'Eddard-Stark', 'Stannis-Baratheon', 'Aegon-I-Targaryen', 'Tyrion-Lannister'}, {'Wyman-Manderly', 'Stannis-Baratheon', 'Robb-Stark', 'Eddard-Stark', 'Davos-Seaworth'}, {'Eddard-Stark', 'Arya-Stark', 'Syrio-Forel'}, {'Robb-Stark', 'Eddard-Stark', 'Torrhen-Karstark'}, {'Bran-Stark', 'Ramsay-Snow', 'Wyman-Manderly', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Rodrik-Cassel'}, {'Bran-Stark', 'Ramsay-Snow', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Arya-Stark', 'Rodrik-Cassel'}, {'Ramsay-Snow', 'Wyman-Manderly', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Roose-Bolton', 'Rodrik-Cassel'}, {'Ramsay-Snow', 'Theon-Greyjoy', 'Robb-Stark', 'Eddard-Stark', 'Roose-Bolton', 'Arya-Stark', 'Rodrik-Cassel'}, {'Eddard-Stark', 'Tywin-Lannister', 'Robb-Stark', 'Sansa-Stark', 'Arya-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Tywin-Lannister', 'Robb-Stark', 'Sansa-Stark', 'Arya-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Robb-Stark', 'Arya-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton'}, {'Tywin-Lannister', 'Robb-Stark', 'Theon-Greyjoy', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton'}, {'Tywin-Lannister', 'Edmure-Tully', 'Robb-Stark', 'Eddard-Stark', 'Marq-Piper', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Petyr-Baelish', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Gregor-Clegane', 'Marq-Piper'}, {'Robb-Stark', 'Eddard-Stark', 'Rickon-Stark', 'Brandon-Stark'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Arya-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Bran-Stark', 'Robert-Baratheon'}, {'Eddard-Stark', 'Jory-Cassel', 'Robb-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Cersei-Lannister', 'Jaime-Lannister', 'Robert-Baratheon'}, {'Eddard-Stark', 'Robb-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Bran-Stark', 'Tyrion-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Jory-Cassel', 'Petyr-Baelish', 'Robb-Stark', 'Eddard-Stark', 'Arya-Stark', 'Rodrik-Cassel'}, {'Jory-Cassel', 'Petyr-Baelish', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Arya-Stark', 'Rodrik-Cassel'}, {'Robb-Stark', 'Theon-Greyjoy', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton', 'Rodrik-Cassel'}, {'Robb-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Roose-Bolton', 'Rodrik-Cassel'}, {'Robb-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Catelyn-Stark', 'Bran-Stark', 'Tyrion-Lannister', 'Eddard-Stark', 'Rodrik-Cassel', 'Joffrey-Baratheon'}, {'Robb-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Catelyn-Stark', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Rodrik-Cassel', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Rickard-Karstark', 'Luwin', 'Robb-Stark', 'Eddard-Stark', 'Catelyn-Stark'}, {'Tyrion-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Cleos-Frey', 'Catelyn-Stark', 'Cersei-Lannister'}, {'Rickard-Karstark', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Roose-Bolton', 'Catelyn-Stark'}, {'Rickard-Karstark', 'Jaime-Lannister', 'Robb-Stark', 'Eddard-Stark', 'Roose-Bolton', 'Galbart-Glover'}, {'Brandon-Stark', 'Robert-Baratheon', 'Robb-Stark', 'Eddard-Stark', 'Petyr-Baelish'}, {'Sandor-Clegane', 'Eddard-Stark', 'Sansa-Stark', 'Mycah', 'Arya-Stark', 'Joffrey-Baratheon'}, {'Tomard', 'Arya-Stark', 'Jory-Cassel', 'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish'}, {'Mordane', 'Arya-Stark', 'Jory-Cassel', 'Robert-Baratheon', 'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister'}, {'Mordane', 'Arya-Stark', 'Robert-Baratheon', 'Eddard-Stark', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Tywin-Lannister', 'Sansa-Stark', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Eddard-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Sandor-Clegane', 'Robert-Baratheon', 'Sansa-Stark', 'Petyr-Baelish', 'Arya-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Gregor-Clegane', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Robert-Baratheon', 'Sansa-Stark', 'Petyr-Baelish', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Gregor-Clegane', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Jon-Arryn', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister', 'Tyrion-Lannister', 'Barristan-Selmy', 'Jaime-Lannister', 'Robert-Baratheon', 'Varys'}, {'Loras-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Rodrik-Cassel', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Jon-Arryn', 'Robert-Baratheon', 'Tyrion-Lannister', 'Eddard-Stark', 'Petyr-Baelish', 'Catelyn-Stark', 'Cersei-Lannister'}, {'Beric-Dondarrion', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Sansa-Stark', 'Arya-Stark', 'Catelyn-Stark', 'Joffrey-Baratheon'}, {'Beric-Dondarrion', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Sansa-Stark', 'Catelyn-Stark', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Aerys-II-Targaryen', 'Tywin-Lannister', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Aerys-II-Targaryen', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Barristan-Selmy', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Sansa-Stark', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Daenerys-Targaryen', 'Robert-Baratheon', 'Barristan-Selmy', 'Eddard-Stark', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Lyanna-Stark', 'Robert-Baratheon', 'Eddard-Stark', 'Sansa-Stark', 'Rhaegar-Targaryen'}, {'Robert-Baratheon', 'Barristan-Selmy', 'Jaime-Lannister', 'Eddard-Stark', 'Sansa-Stark', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Bran-Stark', 'Eddard-Stark', 'Howland-Reed'}, {'Eddard-Stark', 'Jory-Cassel', 'Wyl-(guard)', 'Jaime-Lannister'}, {'Torrhen-Stark', 'Eddard-Stark', 'Aegon-I-Targaryen'}, {'Aerys-II-Targaryen', 'Robert-Baratheon', 'Jaime-Lannister', 'Eddard-Stark', 'Barristan-Selmy', 'Gerold-Hightower'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Arya-Stark', 'Beric-Dondarrion', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sandor-Clegane', 'Sansa-Stark', 'Beric-Dondarrion', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Aerys-II-Targaryen', 'Tywin-Lannister', 'Robert-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Aerys-II-Targaryen', 'Robert-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Barristan-Selmy', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Sansa-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Tyrion-Lannister', 'Varys', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'High-Septon-(fat_one)', 'Eddard-Stark', 'Tyrion-Lannister', 'Joffrey-Baratheon'}, {'Eddard-Stark', 'Maron-Greyjoy'}, {'Aerys-II-Targaryen', 'Eddard-Stark', 'Brandon-Stark', 'Robert-Baratheon'}, {'Aerys-II-Targaryen', 'Jon-Arryn', 'Robert-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Eddard-Stark', 'Barristan-Selmy', 'Cersei-Lannister'}, {'Eddard-Stark', 'Ilyn-Payne', 'Edmure-Tully', 'Jaime-Lannister'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Gendry', 'Eddard-Stark', 'Arya-Stark', 'Harwin'}, {'Eddard-Stark', 'Arya-Stark', 'Jory-Cassel', 'Harwin'}, {'Eddard-Stark', 'Alyn', 'Jory-Cassel', 'Harwin'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Robert-Baratheon', 'Gendry', 'Eddard-Stark', 'Arya-Stark'}, {'Lyanna-Stark', 'Eddard-Stark', 'Brandon-Stark', 'Robert-Baratheon'}, {'Tomard', 'Eddard-Stark', 'Jory-Cassel', 'Alyn'}, {'Eddard-Stark', 'Robert-Baratheon', 'Jory-Cassel', 'Alyn'}, {'Vayon-Poole', 'Robert-Baratheon', 'Jory-Cassel', 'Eddard-Stark', 'Arya-Stark', 'Cersei-Lannister'}, {'Eddard-Stark', 'Tregar'}, {'Daenerys-Targaryen', 'Robert-Baratheon', 'Tyrion-Lannister', 'Varys', 'Eddard-Stark', 'Barristan-Selmy', 'Cersei-Lannister'}, {'Tomard', 'Eddard-Stark', 'Cayn'}, {'Eddard-Stark', 'Desmond', 'Arya-Stark'}, {'Eddard-Stark', 'Willam-Dustin', 'Robert-Baratheon'}, {'Joss', 'Eddard-Stark'}, {'Wylla', 'Eddard-Stark'}, {'Porther', 'Eddard-Stark'}, {'Torrhen-Karstark', 'Eddard-Stark', 'Daryn-Hornwood'}, {'Eddard-Stark', 'Daenerys-Targaryen', 'Aegon-I-Targaryen', 'Tyrion-Lannister'}, {'Eddard-Stark', 'Varly'}, {'Jacks', 'Eddard-Stark'}, {'Eddard-Stark', 'Robert-Baratheon', 'Raymun-Darry'}, {'Eddard-Stark', 'Tobho-Mott'}, {'Rolly-Duckfield', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Tyrion-Lannister', 'Haldon', 'Yandry', 'Ysilla', 'Lemore'}, {'Qotho', 'Mirri-Maz-Duur', 'Daenerys-Targaryen', 'Haggo', 'Drogo'}, {'Qotho', 'Cohollo', 'Daenerys-Targaryen', 'Haggo', 'Drogo'}, {'Drogo', 'Daenerys-Targaryen', 'Haggo', 'Rakharo'}, {'Camarron', 'Spotted-Cat'}, {'Martyn-Lannister', 'Tywin-Lannister'}, {'Martyn-Lannister', 'Willem-Lannister'}, {'Karyl-Vance', 'Clement-Piper', 'Jaime-Lannister'}, {'Karyl-Vance', 'Marq-Piper', 'Edmure-Tully'}, {'Karyl-Vance', 'Edmure-Tully', 'Jaime-Lannister'}, {'Creighton-Longbough', 'Shadrich'}, {'Shadrich', 'Jaime-Lannister'}, {'Vardis-Egen', 'Eon-Hunter', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Grey-Worm', 'Irri', 'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq'}, {'Irri', 'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Missandei', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq', 'Jhiqui'}, {'Victarion-Greyjoy', 'Talbert-Serry'}, {'Melara-Hetherspoon', 'Cersei-Lannister', 'Maggy'}, {'Melara-Hetherspoon', 'Cersei-Lannister', 'Jaime-Lannister'}, {'Theon-Greyjoy', 'Rodrik-Cassel', 'Dagmer'}, {'Arya-Stark', 'Ravella-Swann'}, {'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Reznak-mo-Reznak', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq', 'Galazza-Galare'}, {'Tattered-Prince', 'Quentyn-Martell', 'Gerris-Drinkwater', 'Caggo', 'Meris'}, {'Lucifer-Long', 'Caggo'}, {'Theon-Greyjoy', 'Squint', 'Lorren'}, {'Theon-Greyjoy', 'Squint', 'Drennan'}, {'Falyse-Stokeworth', 'Bronn', 'Cersei-Lannister', 'Balman-Byrch'}, {'Falyse-Stokeworth', 'Qyburn', 'Cersei-Lannister'}, {'Brienne-of-Tarth', 'Willow-Heddle', 'Gendry'}, {'Jon-Umber-(Greatjon)', 'Robb-Stark', 'Wendel-Manderly', 'Robin-Flint'}, {'Sandor-Clegane', 'Amory-Lorch', 'Arya-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Rafford', 'Dunsen', 'Polliver', 'Chiswyck', 'Gregor-Clegane', 'Meryn-Trant', 'Tickler', 'Joffrey-Baratheon'}, {'Polliver', 'Dunsen', 'Chiswyck', 'Weese', 'Gregor-Clegane', 'Arya-Stark'}, {'Khrazz', 'Goghor-the-Giant', 'Spotted-Cat', 'Hizdahr-zo-Loraq'}, {'Quentyn-Martell', 'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Missandei', 'Daenerys-Targaryen', 'Barristan-Selmy'}, {'Quentyn-Martell', 'Gerris-Drinkwater', 'Hizdahr-zo-Loraq', 'Daenerys-Targaryen', 'Barristan-Selmy'}, {'Quentyn-Martell', 'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Belwas'}, {'Daenerys-Targaryen', 'Bloodbeard', 'Hizdahr-zo-Loraq', 'Barristan-Selmy'}, {'Daenerys-Targaryen', 'Spotted-Cat', 'Hizdahr-zo-Loraq'}, {'Hizdahr-zo-Loraq', 'Marghaz-zo-Loraq'}, {'Grey-Worm', 'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq'}, {'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Missandei', 'Daenerys-Targaryen', 'Barristan-Selmy', 'Skahaz-mo-Kandaq'}, {'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq', 'Belwas'}, {'Hizdahr-zo-Loraq', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq', 'Jhiqui', 'Belwas'}, {'Ben-Plumm', 'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Belwas'}, {'Ben-Plumm', 'Daario-Naharis', 'Hizdahr-zo-Loraq', 'Barristan-Selmy', 'Daenerys-Targaryen'}, {'Khrazz', 'Steelskin', 'Hizdahr-zo-Loraq', 'Barristan-Selmy'}, {'Brienne-of-Tarth', 'Selwyn-Tarth'}, {'Renly-Baratheon', 'Lorent-Caswell'}, {'Imry-Florent', 'Davos-Seaworth'}, {'Dick-Crabb', 'Renly-Baratheon', 'Jaime-Lannister', 'Brienne-of-Tarth'}, {'Dick-Crabb', 'Podrick-Payne', 'Brienne-of-Tarth'}, {'Dick-Crabb', 'Eustace-Brune'}, {'Daario-Naharis', 'Jhogo', 'Groleo', 'Hero', 'Daenerys-Targaryen'}, {'Daario-Naharis', 'Groleo', 'Barristan-Selmy', 'Daenerys-Targaryen', 'Skahaz-mo-Kandaq'}, {'Daario-Naharis', 'Jhogo', 'Groleo', 'Daenerys-Targaryen', 'Barristan-Selmy'}, {'Frenken', 'Ballabar'}, {'Ballabar', 'Podrick-Payne', 'Tyrion-Lannister'}, {'Theon-Greyjoy', 'Kyra'}, {'Porridge', 'Axell-Florent', 'Lamprey', 'Davos-Seaworth'}, {'Robar-Royce', 'Emmon-Cuy', 'Brienne-of-Tarth'}, {'Robar-Royce', 'Catelyn-Stark', 'Brienne-of-Tarth'}, {'Terrance-Lynderly', 'Robert-Arryn', 'Gyles-Grafton'}, {'Jon-Umber-(Greatjon)', 'Rickard-Karstark', 'Robb-Stark', 'Roose-Bolton', 'Galbart-Glover'}, {'Asha-Greyjoy', 'Galbart-Glover'}, {'Aegon-Frey-(son-of-Stevron)', 'Stevron-Frey', 'Robb-Stark', 'Walder-Frey', 'Catelyn-Stark'}, {'Renly-Baratheon', 'Alester-Florent', 'Stannis-Baratheon', 'Cortnay-Penrose', 'Davos-Seaworth'}, {'Alester-Florent', 'Stannis-Baratheon', 'Axell-Florent', 'Selyse-Florent', 'Davos-Seaworth'}, {'Axell-Florent', 'Alester-Florent', 'Lamprey', 'Davos-Seaworth'}, {'Mance-Rayder', 'Theon-Greyjoy', 'Rowan', 'Squirrel'}, {'Theon-Greyjoy', 'Jeyne-Poole', 'Rowan', 'Squirrel'}, {'Elia-Martell', 'Ashara-Dayne'}, {'Kella', 'Sansa-Stark'}, {'Bran-Stark', 'Robb-Stark', 'Stafford-Lannister', 'Catelyn-Stark'}, {'Robb-Stark', 'Stafford-Lannister', 'Catelyn-Stark', 'Jaime-Lannister'}, {'Bran-Stark', 'Meera-Reed', 'Luwin', 'Rickon-Stark', 'Hodor', 'Osha', 'Jojen-Reed'}, {'Craster', 'Samwell-Tarly', 'Gilly', 'Jojen-Reed'}, {'Samwell-Tarly', 'Meera-Reed', 'Gilly', 'Jojen-Reed'}, {'Bran-Stark', 'Meera-Reed', 'Samwell-Tarly', 'Jojen-Reed'}, {'Bran-Stark', 'Alebelly', 'Jojen-Reed'}, {'Cromm', 'Hagen', 'Grimtongue'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)'}, {'Mordane', 'Myrcella-Baratheon', 'Robert-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Mordane', 'Robert-Baratheon', 'Jeyne-Poole', 'Arya-Stark', 'Sansa-Stark'}, {'Renly-Baratheon', 'Robert-Baratheon', 'Cressen', 'Stannis-Baratheon'}, {'Stannis-Baratheon', 'Robert-Baratheon', 'Balon-Greyjoy', 'Asha-Greyjoy'}, {'Tyrion-Lannister', 'Lancel-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Sansa-Stark', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Varys'}, {'Tywin-Lannister', 'Tommen-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Robb-Stark', 'Sansa-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Sansa-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Lysa-Arryn'}, {'Sandor-Clegane', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Meryn-Trant', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Tommen-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Myrcella-Baratheon', 'Tyrion-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Myrcella-Baratheon', 'Tyrion-Lannister', 'Robb-Stark', 'Robert-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Tommen-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Joffrey-Baratheon'}, {'Bran-Stark', 'Tyrion-Lannister', 'Tommen-Baratheon', 'Robert-Baratheon', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Jeyne-Poole', 'Ilyn-Payne', 'Robert-Baratheon', 'Sansa-Stark', 'Arya-Stark'}, {'Sansa-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Sansa-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Sansa-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Robert-Baratheon', 'Sansa-Stark', 'Catelyn-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Varys'}, {'Myrcella-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Robb-Stark', 'Robert-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Myrcella-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Tommen-Baratheon', 'Robert-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Sansa-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robert-Baratheon', 'Barristan-Selmy', 'Meryn-Trant', 'Joffrey-Baratheon'}, {'Sandor-Clegane', 'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Loras-Tyrell', 'Joffrey-Baratheon'}, {'Jeyne-Poole', 'Ilyn-Payne', 'Robert-Baratheon', 'Sansa-Stark', 'Loras-Tyrell'}, {'Aerys-II-Targaryen', 'Robert-Baratheon', 'Viserys-Targaryen', 'Rhaegar-Targaryen'}, {'Aerys-II-Targaryen', 'Robert-Baratheon', 'Viserys-Targaryen', 'Tyrion-Lannister'}, {'Doran-Martell', 'Daenerys-Targaryen', 'Tyrion-Lannister', 'Robert-Baratheon', 'Viserys-Targaryen'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Jaime-Lannister', 'Robert-Baratheon', 'Arya-Stark', 'Thoros-of-Myr'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Gendry', 'Robert-Baratheon', 'Arya-Stark', 'Thoros-of-Myr'}, {'Doran-Martell', 'Myrcella-Baratheon', 'Tyrion-Lannister', 'Robert-Baratheon', 'Cersei-Lannister'}, {'Edric-Storm', 'Robert-Baratheon', 'Stannis-Baratheon'}, {'Aerys-II-Targaryen', 'Tywin-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Jaime-Lannister', 'Robb-Stark', 'Robert-Baratheon', 'Hoster-Tully', 'Catelyn-Stark'}, {'Jaime-Lannister', 'Robert-Baratheon', 'Hoster-Tully', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Taena-of-Myr', 'Tyrion-Lannister', 'Jaime-Lannister', 'Tommen-Baratheon', 'Robert-Baratheon', 'Margaery-Tyrell', 'Cersei-Lannister'}, {'Doran-Martell', 'Tywin-Lannister', 'Tyrion-Lannister', 'Robert-Baratheon', 'Gregor-Clegane', 'Cersei-Lannister'}, {'Doran-Martell', 'Tywin-Lannister', 'Tyrion-Lannister', 'Varys', 'Robert-Baratheon', 'Cersei-Lannister'}, {'Doran-Martell', 'Daenerys-Targaryen', 'Tyrion-Lannister', 'Varys', 'Robert-Baratheon', 'Cersei-Lannister'}, {'Brienne-of-Tarth', 'Robert-Baratheon', 'Sandor-Clegane', 'Gendry'}, {'Robert-Baratheon', 'Aerys-I-Targaryen'}, {'Alerie-Hightower', 'Margaery-Tyrell', 'Mace-Tyrell'}, {'Guncer-Sunglass', 'Davos-Seaworth', 'Melisandre'}, {'Morrec', 'Tyrion-Lannister', 'Jyck'}, {'Lollys-Stokeworth', 'Tyrion-Lannister', 'Varys', 'Bronn', 'Shae'}, {'Shae', 'Lollys-Stokeworth', 'Tyrion-Lannister', 'Symon-Silver-Tongue'}, {'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Bronn', 'Shae', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Shae', 'Tyrion-Lannister', 'Chella', 'Varys'}, {'Shae', 'Sansa-Stark', 'Brella'}, {'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Shae', 'Sansa-Stark', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Shae', 'Sansa-Stark', 'Tysha', 'Tyrion-Lannister'}, {'Ternesio-Terys', 'Arya-Stark', 'Denyo-Terys'}, {'Mya-Stone', 'Robert-Arryn', 'Maddy'}, {'Mya-Stone', 'Robert-Arryn', 'Myranda-Royce'}, {'Mya-Stone', 'Robert-Arryn', 'Catelyn-Stark'}, {'Mya-Stone', 'Robert-Arryn', 'Lothor-Brune'}, {'Raymun-Redbeard', 'Gerrick-Kingsblood'}, {'Robb-Stark', 'Tristifer-IV-Mudd'}, {'Roger-Ryswell', 'Theon-Greyjoy', 'Barbrey-Dustin'}, {'Ramsay-Snow', 'Jeyne-Poole', 'Roose-Bolton', 'Theon-Greyjoy', 'Barbrey-Dustin'}, {'Ramsay-Snow', 'Stannis-Baratheon', 'Roose-Bolton', 'Theon-Greyjoy', 'Barbrey-Dustin'}, {'Walder-Frey', 'Danwell-Frey'}, {'Hosteen-Frey', 'Danwell-Frey', 'Jared-Frey'}, {'Renly-Baratheon', 'Stannis-Baratheon', 'Melisandre', 'Devan-Seaworth', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Devan-Seaworth', 'Pylos', 'Davos-Seaworth'}, {'Meris', 'Denzo-Dhan'}, {'Sawane-Botley', 'Euron-Greyjoy'}, {'Renly-Baratheon', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Cortnay-Penrose'}, {'Cortnay-Penrose', 'Renly-Baratheon', 'Stannis-Baratheon', 'Bryce-Caron'}, {'Robb-Stark', 'Raynald-Westerling', 'Catelyn-Stark'}, {'Victarion-Greyjoy', 'Kerwin'}, {'Ragwyle', 'Qhorin-Halfhand'}, {'Watt', 'Eddison-Tollett'}, {'Elinor-Tyrell', 'Alla-Tyrell', 'Margaery-Tyrell', 'Megga-Tyrell', 'Sansa-Stark'}, {'Ramsay-Snow', 'Palla', 'Theon-Greyjoy'}, {'Meera-Reed', 'Coldhands', 'Gilly'}, {'Samwell-Tarly', 'Small-Paul', 'Gilly'}, {'Samwell-Tarly', 'Kojja-Mo', 'Gilly'}, {'Arianne-Martell', 'Doran-Martell', 'Obara-Sand', 'Tyene-Sand'}, {'Nymeria-Sand', 'Doran-Martell', 'Obara-Sand', 'Tyene-Sand'}, {'Sarella-Sand', 'Tyene-Sand'}, {'Arianne-Martell', 'Andrey-Dalt', 'Garin-(orphan)', 'Sylva-Santagar', 'Tyene-Sand'}, {'Arianne-Martell', 'Doran-Martell', 'Oberyn-Martell', 'Tyene-Sand'}, {'Garin-(orphan)', 'Nymeria-Sand', 'Tyene-Sand'}, {'Hot-Pie', 'Sharna', 'Tom-of-Sevenstreams', 'Lem'}, {'Hot-Pie', 'Gendry', 'Lem', 'Arya-Stark', 'Tom-of-Sevenstreams'}, {'Greenbeard', 'Lem', 'Anguy', 'Arya-Stark', 'Tom-of-Sevenstreams'}, {'Lem', 'Gendry', 'Greenbeard', 'Arya-Stark', 'Tom-of-Sevenstreams'}, {'Beric-Dondarrion', 'Lem', 'Anguy', 'Arya-Stark', 'Tom-of-Sevenstreams'}, {'Beric-Dondarrion', 'Gendry', 'Lem', 'Arya-Stark', 'Tom-of-Sevenstreams'}, {'Anguy', 'Sharna', 'Tom-of-Sevenstreams', 'Lem'}, {'Tansy', 'Tom-of-Sevenstreams', 'Lem'}, {'Wyman-Manderly', 'Rhaegar-Frey'}, {'Rhaegar-Frey', 'Jared-Frey'}, {'Perwyn-Frey', 'Robb-Stark', 'Wendel-Manderly'}, {'Narbert-Grandison', 'Benethon-Scales', 'Selyse-Florent'}, {'Elinor-Tyrell', 'Tallad', 'Margaery-Tyrell'}, {'Lambert-Turnberry', 'Jalabhar-Xho', 'Tallad'}, {'Lambert-Turnberry', 'Blue-Bard', 'Tallad'}, {'Margaery-Tyrell', 'Tallad', 'Cersei-Lannister', 'Blue-Bard'}, {'Jalabhar-Xho', 'Tallad', 'Cersei-Lannister'}, {'Bran-Stark', 'Theon-Greyjoy', 'Mikken', 'Osha'}, {'Bran-Stark', 'Mikken', 'Chayle'}, {'Bran-Stark', 'Mikken', 'Alebelly'}, {'Godric-Borrell', 'Davos-Seaworth'}, {'Renly-Baratheon', 'Cressen', 'Stannis-Baratheon', 'Melisandre', 'Davos-Seaworth'}, {'Cressen', 'Stannis-Baratheon', 'Selyse-Florent', 'Shireen-Baratheon', 'Davos-Seaworth'}, {'Cressen', 'Stannis-Baratheon', 'Melisandre', 'Selyse-Florent', 'Davos-Seaworth'}, {'Cressen', 'Stannis-Baratheon', 'Shireen-Baratheon', 'Pylos', 'Davos-Seaworth'}, {'Robb-Stark', 'Wyman-Manderly', 'Sybell-Spicer', 'Davos-Seaworth'}, {'Sybell-Spicer', 'Garth-(Wolfs-Den)', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Salladhor-Saan', 'Melisandre', 'Axell-Florent', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Salladhor-Saan', 'Khorane-Sathmantes', 'Davos-Seaworth'}, {'Allard-Seaworth', 'Stannis-Baratheon', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Axell-Florent', 'Selyse-Florent', 'Shireen-Baratheon', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Edric-Storm', 'Shireen-Baratheon', 'Pylos', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Joffrey-Baratheon', 'Selyse-Florent', 'Shireen-Baratheon', 'Davos-Seaworth'}, {'Renly-Baratheon', 'Stannis-Baratheon', 'Melisandre', 'Joffrey-Baratheon', 'Davos-Seaworth'}, {'Monford-Velaryon', 'Stannis-Baratheon', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Melisandre', 'Axell-Florent', 'Selyse-Florent', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Melisandre', 'Joffrey-Baratheon', 'Selyse-Florent', 'Davos-Seaworth'}, {'Stannis-Baratheon', 'Marlon-Manderly', 'Davos-Seaworth'}, {'Edric-Storm', 'Andrew-Estermont', 'Davos-Seaworth'}, {'Bartimus', 'Davos-Seaworth'}, {'Wyman-Manderly', 'Leona-Woolfield', 'Davos-Seaworth'}, {'Matthos-Seaworth', 'Davos-Seaworth'}, {'Arianne-Martell', 'Myrcella-Baratheon', 'Gerold-Dayne', 'Balon-Swann'}, {'Arianne-Martell', 'Myrcella-Baratheon', 'Gerold-Dayne', 'Garin-(orphan)', 'Arys-Oakheart'}, {'Brienne-of-Tarth', 'Meribald', 'Podrick-Payne', 'Hyle-Hunt'}, {'Tywin-Lannister', 'Podrick-Payne', 'Tyrion-Lannister', 'Brienne-of-Tarth', 'Sansa-Stark'}, {'Tywin-Lannister', 'Podrick-Payne', 'Tyrion-Lannister', 'Sansa-Stark', 'Petyr-Baelish'}, {'Bronn', 'Tywin-Lannister', 'Podrick-Payne', 'Tyrion-Lannister'}, {'Brienne-of-Tarth', 'Hyle-Hunt', 'Podrick-Payne', 'Sansa-Stark'}, {'Cedric-Payne', 'Podrick-Payne'}, {'Mance-Rayder', 'Joramun', 'Melisandre'}, {'Theon-Greyjoy', 'Willow-Witch-eye'}, {'Olenna-Redwyne', 'Butterbumps', 'Sansa-Stark'}, {'Edmure-Tully', 'Robb-Stark', 'Marq-Piper', 'Catelyn-Stark', 'Lothar-Frey'}, {'Smiling-Knight', 'Simon-Toyne'}, {'Maegor-I-Targaryen', 'Jaehaerys-I-Targaryen'}, {'Bran-Stark', 'Hayhead', 'Alebelly'}, {'Ramsay-Snow', 'Theon-Greyjoy', 'Sour-Alyn', 'Damon-Dance-for-Me', 'Skinner'}, {'Skinner', 'Luton'}, {'Miklaz', 'Draqaz'}, {'Lyle-Crakehall', 'Ilyn-Payne', 'Jaime-Lannister'}, {'Lyle-Crakehall', 'Amerei-Frey', 'Jaime-Lannister'}, {'Bran-Stark', 'Hodor', 'Coldhands', 'Meera-Reed'}, {'Jaime-Lannister', 'Hoster-Blackwood'}, {'Sansa-Stark', 'Baelor-I-Targaryen', 'Cersei-Lannister', 'Tyrion-Lannister'}, {'Tyrion-Lannister', 'Lancel-Lannister', 'Jaime-Lannister', 'Kevan-Lannister', 'Sansa-Stark', 'Varys', 'Cersei-Lannister', 'Joffrey-Baratheon'}, {'Sansa-Stark', 'Robert-Arryn', 'Myranda-Royce'}, {'Brienne-of-Tarth', 'Sansa-Stark', 'Hyle-Hunt', 'Sandor-Clegane'}, {'Brienne-of-Tarth', 'Sansa-Stark', 'Hyle-Hunt', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Sansa-Stark', 'Pycelle', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon'}, {'Sansa-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Stannis-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Loras-Tyrell'}, {'Sansa-Stark', 'Cersei-Lannister', 'Renly-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell'}, {'Sansa-Stark', 'Myrcella-Baratheon', 'Robert-Arryn', 'Tyrion-Lannister'}, {'Myrcella-Baratheon', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Arys-Oakheart'}, {'Tywin-Lannister', 'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Kevan-Lannister', 'Joffrey-Baratheon'}, {'Tywin-Lannister', 'Sansa-Stark', 'Pycelle', 'Cersei-Lannister', 'Stannis-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Kevan-Lannister', 'Joffrey-Baratheon'}, {'Jaime-Lannister', 'Tywin-Lannister', 'Sansa-Stark', 'Pycelle', 'Cersei-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Varys', 'Kevan-Lannister', 'Joffrey-Baratheon'}, {'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Stannis-Baratheon', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Loras-Tyrell'}, {'Sansa-Stark', 'Petyr-Baelish', 'Oswell-Kettleblack'}, {'Petyr-Baelish', 'Sansa-Stark', 'Catelyn-Stark', 'Robert-Arryn', 'Nestor-Royce', 'Lysa-Arryn'}, {'Petyr-Baelish', 'Sansa-Stark', 'Robert-Arryn', 'Nestor-Royce', 'Yohn-Royce'}, {'Sansa-Stark', 'Petyr-Baelish', 'Lothor-Brune', 'Robert-Arryn'}, {'Marillion', 'Petyr-Baelish', 'Tyrion-Lannister', 'Sansa-Stark', 'Robert-Arryn', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Gretchel', 'Sansa-Stark', 'Robert-Arryn', 'Maddy'}, {'Mace-Tyrell', 'Tywin-Lannister', 'Oberyn-Martell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister'}, {'Tommen-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell'}, {'Mace-Tyrell', 'Margaery-Tyrell', 'Tyrion-Lannister', 'Tommen-Baratheon', 'Joffrey-Baratheon', 'Sansa-Stark', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Olenna-Redwyne', 'Sansa-Stark', 'Cersei-Lannister', 'Mace-Tyrell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Loras-Tyrell'}, {'Mace-Tyrell', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Gyles-Rosby', 'Pycelle', 'Cersei-Lannister'}, {'Mace-Tyrell', 'Willas-Tyrell', 'Margaery-Tyrell', 'Tyrion-Lannister', 'Loras-Tyrell', 'Sansa-Stark', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Gregor-Clegane', 'Oberyn-Martell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister'}, {'Sandor-Clegane', 'Sansa-Stark', 'Arya-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Polliver', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Meryn-Trant'}, {'Tyrion-Lannister', 'Tommen-Baratheon', 'Joffrey-Baratheon', 'Sansa-Stark', 'Gyles-Rosby', 'Boros-Blount', 'Cersei-Lannister'}, {'Tyrion-Lannister', 'Jaime-Lannister', 'Barristan-Selmy', 'Joffrey-Baratheon', 'Sansa-Stark', 'Boros-Blount', 'Meryn-Trant', 'Cersei-Lannister'}, {'Sandor-Clegane', 'Mandon-Moore', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister'}, {'Mandon-Moore', 'Tyrion-Lannister', 'Varys', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister'}, {'Garlan-Tyrell', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Sansa-Stark', 'Cersei-Lannister', 'Loras-Tyrell'}, {'Sansa-Stark', 'Tysha', 'Tyrion-Lannister', 'Jaime-Lannister'}, {'Maegor-I-Targaryen', 'Sansa-Stark', 'Cersei-Lannister', 'Varys'}, {'Maegor-I-Targaryen', 'Sansa-Stark', 'Cersei-Lannister', 'Tommen-Baratheon'}, {'Garlan-Tyrell', 'Willas-Tyrell', 'Loras-Tyrell', 'Tyrion-Lannister', 'Sansa-Stark', 'Cersei-Lannister'}, {'Florian-the-Fool', 'Sansa-Stark'}, {'Brienne-of-Tarth', 'Elder-Brother', 'Meribald', 'Narbert'}, {'Drogo', 'Qotho', 'Daenerys-Targaryen', 'Quaro'}, {'Rhaenys-Targaryen', 'Visenya-Targaryen', 'Aegon-I-Targaryen'}, {'Benedar-Belmore', 'Gilwood-Hunter', 'Anya-Waynwood', 'Symond-Templeton'}, {'Benedar-Belmore', 'Petyr-Baelish', 'Anya-Waynwood', 'Symond-Templeton'}, {'Petyr-Baelish', 'Anya-Waynwood', 'Symond-Templeton', 'Yohn-Royce'}, {'Gilwood-Hunter', 'Lysa-Arryn'}, {'Lyn-Corbray', 'Petyr-Baelish', 'Lysa-Arryn', 'Robert-Arryn'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Edmure-Tully', 'Tansy', 'Hoster-Tully', 'Catelyn-Stark', 'Lysa-Arryn'}, {'Stannis-Baratheon', 'Mathis-Rowan', 'Lysa-Arryn', 'Catelyn-Stark'}, {'Tyrion-Lannister', 'Jon-Arryn', 'Catelyn-Stark', 'Petyr-Baelish', 'Lysa-Arryn', 'Robert-Arryn'}, {'Vardis-Egen', 'Catelyn-Stark', 'Lysa-Arryn', 'Tyrion-Lannister'}, {'Shrouded-Lord', 'Tyrion-Lannister'}, {'Anya-Waynwood', 'Yohn-Royce', 'Harrold-Hardyng'}, {'Anya-Waynwood', 'Yohn-Royce', 'Horton-Redfort'}, {'Robert-Arryn', 'Yohn-Royce', 'Harrold-Hardyng'}, {'Galyeon-of-Cuy', 'Tyrion-Lannister'}, {'Theobald', 'Samwell-Tarly'}, {'Larraq', 'Barristan-Selmy'}, {'Hot-Pie', 'Amory-Lorch', 'Yoren', 'Gendry', 'Arya-Stark'}, {'Jaqen-Hghar', 'Yoren', 'Biter', 'Rorge', 'Arya-Stark'}, {'Gendry', 'Jaqen-Hghar', 'Arya-Stark', 'Yoren'}, {'Joffrey-Baratheon', 'Arya-Stark', 'Yoren', 'Amory-Lorch'}, {'Arya-Stark', 'Syrio-Forel', 'Yoren'}, {'Koss', 'Arya-Stark', 'Yoren'}, {'Theon-Greyjoy', 'Roose-Bolton', 'Walda-Frey-(daughter-of-Merrett)'}, {'Haldon', 'Pono'}, {'Wyman-Manderly', 'Theomore', 'Stannis-Baratheon'}, {'Victarion-Greyjoy', 'Wulfe'}, {'Ramsay-Snow', 'Ben-Bones', 'Theon-Greyjoy'}, {'Ramsay-Snow', 'Hother-Umber', 'Wyman-Manderly'}, {'Ramsay-Snow', 'Theon-Greyjoy', 'Sour-Alyn', 'Yellow-Dick'}, {'Mance-Rayder', 'Ramsay-Snow', 'Arya-Stark', 'Theon-Greyjoy'}, {'Ramsay-Snow', 'Roose-Bolton', 'Theon-Greyjoy', 'Jeyne-Poole', 'Arya-Stark'}, {'Mance-Rayder', 'Ramsay-Snow', 'Stannis-Baratheon', 'Theon-Greyjoy'}, {'Bran-Stark', 'Ramsay-Snow', 'Theon-Greyjoy', 'Rodrik-Cassel', 'Walder-Frey-(son-of-Merrett)'}, {'Bran-Stark', 'Ramsay-Snow', 'Donella-Hornwood', 'Rodrik-Cassel'}, {'Bran-Stark', 'Ramsay-Snow', 'Reek'}, {'Ramsay-Snow', 'Aggar'}, {'Vardis-Egen', 'Colemon', 'Catelyn-Stark'}, {'Robert-Arryn', 'Colemon', 'Catelyn-Stark', 'Petyr-Baelish'}, {'Gretchel', 'Colemon', 'Robert-Arryn', 'Maddy'}, {'Tywin-Lannister', 'Amory-Lorch', 'Oberyn-Martell', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Cersei-Lannister'}, {'Doran-Martell', 'Tywin-Lannister', 'Oberyn-Martell', 'Tyrion-Lannister', 'Gregor-Clegane', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Elia-Martell', 'Oberyn-Martell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Gregor-Clegane', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Oberyn-Martell', 'Tyrion-Lannister', 'Bronn', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Cersei-Lannister'}, {'Addam-Marbrand', 'Tywin-Lannister', 'Oberyn-Martell', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Cersei-Lannister'}, {'Doran-Martell', 'Viserys-Targaryen', 'Oberyn-Martell', 'Tyrion-Lannister'}, {'Arianne-Martell', 'Doran-Martell', 'Oberyn-Martell', 'Ellaria-Sand', 'Areo-Hotah'}, {'Doran-Martell', 'Oberyn-Martell', 'Tyrion-Lannister', 'Ellaria-Sand', 'Cersei-Lannister'}, {'Doran-Martell', 'Oberyn-Martell', 'Daemon-Sand'}, {'Daenerys-Targaryen', 'Tattered-Prince', 'Bloodbeard'}, {'Aggar', 'Gynir'}, {'Wex-Pyke', 'Gynir'}, {'Moelle', 'Scolera', 'Cersei-Lannister', 'Unella'}, {'Craster', 'Samwell-Tarly', 'Dirk'}, {'Amerei-Frey', 'Lancel-Lannister', 'Jaime-Lannister'}, {'Clubfoot-Karl', 'Samwell-Tarly', 'Craster'}, {'Tattered-Prince', 'Dick-Straw'}, {'Dick-Straw', 'Hugh-Hungerford'}, {'Brea', 'Arya-Stark', 'Brusco'}, {'Brea', 'Arya-Stark', 'Talea'}, {'Pate-(novice)', 'Walgrave'}, {'Gormon-Tyrell', 'Walgrave'}, {'Bran-Stark', 'Luwin', 'Theon-Greyjoy', 'Rickon-Stark', 'Rodrik-Cassel', 'Walder-Frey-(son-of-Merrett)'}, {'Bran-Stark', 'Theon-Greyjoy', 'Walder-Frey-(son-of-Jammos)', 'Walder-Frey-(son-of-Merrett)'}, {'Theon-Greyjoy', 'Wex-Pyke', 'Walder-Frey-(son-of-Merrett)'}, {'Samwell-Tarly', 'Dywen', 'Bannen'}, {'Ulf-son-of-Umar', 'Shagga'}, {'Alysane-Mormont', 'Stannis-Baratheon', 'Asha-Greyjoy', 'Justin-Massey'}, {'Anguy', 'Husband', 'Lem'}, {'Waymar-Royce', 'Gared', 'Will-(prologue)'}, {'Jeor-Mormont', 'Gared'}, {'Kevan-Lannister', 'Jaime-Lannister', 'Ronnet-Connington'}, {'Randyll-Tarly', 'Arwyn-Oakheart'}, {'Randyll-Tarly', 'Hyle-Hunt', 'Catelyn-Stark', 'Brienne-of-Tarth'}, {'Randyll-Tarly', 'Mace-Tyrell', 'Robert-Strong'}, {'Randyll-Tarly', 'Renly-Baratheon', 'Stannis-Baratheon', 'Brienne-of-Tarth', 'Catelyn-Stark'}, {'Randyll-Tarly', 'Renly-Baratheon', 'Stannis-Baratheon', 'Mathis-Rowan', 'Catelyn-Stark'}, {'Randyll-Tarly', 'Renly-Baratheon', 'Stannis-Baratheon', 'Mace-Tyrell', 'Mathis-Rowan'}, {'Randyll-Tarly', 'Stannis-Baratheon', 'Samwell-Tarly'}, {'Randyll-Tarly', 'Stannis-Baratheon', 'Mace-Tyrell', 'Kevan-Lannister'}, {'Tyrion-Lannister', 'Lancel-Lannister', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Jacelyn-Bywater', 'Varys'}, {'Lem', 'Gendry', 'Greenbeard', 'Arya-Stark', 'Thoros-of-Myr'}, {'Lem', 'Gendry', 'Greenbeard', 'Arya-Stark', 'Harwin'}, {'Greenbeard', 'Lem', 'Anguy', 'Arya-Stark', 'Harwin'}, {'Shireen-Baratheon', 'Daeron-I-Targaryen'}, {'Cutjack', 'Kurz'}, {'Cutjack', 'Tarber'}, {'Cutjack', 'Gendry'}, {'Daenerys-Targaryen', 'Daeron-II-Targaryen'}, {'Daeron-II-Targaryen', 'Aemon-Targaryen-(Maester-Aemon)'}, {'Senelle', 'Margaery-Tyrell', 'Cersei-Lannister'}, {'High-Sparrow', 'Taena-of-Myr', 'Tommen-Baratheon', 'Margaery-Tyrell', 'Osney-Kettleblack', 'Cersei-Lannister'}, {'Taena-of-Myr', 'Dorcas', 'Margaery-Tyrell', 'Osney-Kettleblack', 'Cersei-Lannister'}, {'Tommen-Baratheon', 'Stannis-Baratheon', 'Osney-Kettleblack', 'Cersei-Lannister'}, {'Margaery-Tyrell', 'Osney-Kettleblack', 'Alla-Tyrell'}, {'Eldon-Estermont', 'Steffon-Varner'}, {'Ardrian-Celtigar', 'Monford-Velaryon'}, {'Coratt', 'Conn'}, {'Conn', 'Shagga', 'Tyrion-Lannister'}, {'Timett', 'Shagga', 'Bronn', 'Tyrion-Lannister'}, {'Timett', 'Bronn', 'Tyrion-Lannister', 'Varys'}, {'Aemon-Targaryen-(Dragonknight)', 'Jaime-Lannister'}, {'Daenerys-Targaryen', 'Kraznys-mo-Nakloz', 'Arstan'}, {'Cressen', 'Stannis-Baratheon', 'Selyse-Florent', 'Shireen-Baratheon', 'Patchface'}, {'Tommen-Baratheon', 'Joffrey-Baratheon', 'Stannis-Baratheon', 'Shireen-Baratheon'}, {'Theon-Greyjoy', 'Stygg', 'Benfred-Tallhart'}, {'Helman-Tallhart', 'Benfred-Tallhart'}, {'Benfred-Tallhart', 'Leobald-Tallhart'}, {'Jory-Cassel', 'Heward', 'Wyl-(guard)'}, {'Weasel', 'Weese', 'Arya-Stark'}, {'Weese', 'Jaqen-Hghar', 'Arya-Stark'}, {'Weese', 'Gregor-Clegane', 'Tywin-Lannister', 'Arya-Stark'}, {'Arianne-Martell', 'Garin-(orphan)', 'Cedra'}, {'Marwyn', 'Armen', 'Alleras'}, {'Leo-Tyrell', 'Alleras', 'Pate-(novice)', 'Armen'}, {'Leo-Tyrell', 'Alleras', 'Armen', 'Roone'}, {'Walder-Frey', 'Petyr-Frey'}, {'Brandon-Stark', 'Rickard-Stark'}, {'Bran-Stark', 'Jory-Cassel', 'Robb-Stark', 'Hullen', 'Rodrik-Cassel'}, {'Hullen', 'Jory-Cassel', 'Harwin'}, {'Samwell-Tarly', 'Jeor-Mormont', 'Grenn', 'Small-Paul'}, {'Samwell-Tarly', 'Lark', 'Small-Paul', 'Jeor-Mormont', 'Chett'}, {'Arianne-Martell', 'Timoth'}, {'Captain-Myraham', 'Theon-Greyjoy'}, {'Urswyck', 'Jaime-Lannister'}, {'Stannis-Baratheon', 'Theon-Greyjoy', 'Balon-Greyjoy', 'Asha-Greyjoy'}, {'Benedar-Belmore', 'Gilwood-Hunter', 'Anya-Waynwood', 'Horton-Redfort'}, {'Rakharo', 'Daenerys-Targaryen', 'Aggo', 'Drogo', 'Quaro'}, {'Viserys-Targaryen', 'Rhaella-Targaryen', 'Rhaegar-Targaryen'}, {'Daenerys-Targaryen', 'Viserys-Targaryen', 'Rhaegar-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)'}, {'Daenerys-Targaryen', 'Viserys-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Tyrion-Lannister'}, {'Robb-Stark', 'Desmond-Grell', 'Catelyn-Stark', 'Edmure-Tully'}, {'Robb-Stark', 'Desmond-Grell', 'Catelyn-Stark', 'Utherydes-Wayn'}, {'Damon-Dance-for-Me', 'Theon-Greyjoy', 'Grunt', 'Sour-Alyn'}, {'Red-Oarsman', 'Lucas-Codd'}, {'Lucas-Codd', 'Torwold-Browntooth'}, {'Victarion-Greyjoy', 'Longwater-Pyke'}, {'Yezzan-zo-Qaggaz', 'Scar', 'Tyrion-Lannister'}, {'Gerald-Gower', 'Andrew-Estermont'}, {'Zollo', 'Shagwell', 'Rorge', 'Jaime-Lannister'}, {'Brienne-of-Tarth', 'Shagwell', 'Jaime-Lannister'}, {'Pyg', 'Shagwell', 'Timeon', 'Brienne-of-Tarth'}, {'Shadd', 'Wendel-Manderly'}, {'Janos-Slynt', 'Tyrion-Lannister', 'Varys', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Jacelyn-Bywater'}, {'Allar-Deem', 'Janos-Slynt', 'Tyrion-Lannister'}, {'Mad-Huntsman', 'Arya-Stark'}, {'Tyrion-Lannister', 'Chataya'}, {'Bran-Stark', 'Hali'}, {'Lyn-Corbray', 'Petyr-Baelish', 'Lyonel-Corbray'}, {'Dorcas', 'Margaery-Tyrell', 'Qyburn', 'Cersei-Lannister'}, {'Dorcas', 'Qyburn', 'Cersei-Lannister', 'Jocelyn-Swyft'}, {'Stannis-Baratheon', 'Qyburn', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Roose-Bolton'}, {'Margaery-Tyrell', 'Blue-Bard', 'Qyburn', 'Cersei-Lannister'}, {'Robert-Strong', 'Qyburn', 'Cersei-Lannister'}, {'Stannis-Baratheon', 'Qyburn', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Pycelle', 'Cersei-Lannister'}, {'Qyburn', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Pycelle', 'Cersei-Lannister'}, {'Qyburn', 'Joffrey-Baratheon', 'Gyles-Rosby', 'Pycelle', 'Cersei-Lannister'}, {'Stannis-Baratheon', 'Qyburn', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Joffrey-Baratheon', 'Cersei-Lannister'}, {'Qyburn', 'Joffrey-Baratheon', 'Margaery-Tyrell', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Pia', 'Qyburn', 'Jaime-Lannister'}, {'Lorren', 'Sybelle-Glover'}, {'Asha-Greyjoy', 'Sybelle-Glover'}, {'Emrick', 'Arron', 'Hareth-(Moles-Town)', 'Satin'}, {'Robb-Stark', 'Walder-Rivers', 'Lothar-Frey'}, {'Mathis-Rowan', 'Jon-Fossoway'}, {'Doran-Martell', 'Gregor-Clegane', 'Caleotte'}, {'Doran-Martell', 'Areo-Hotah', 'Obara-Sand', 'Caleotte'}, {'Addam-Marbrand', 'Tywin-Lannister', 'Tyrion-Lannister', 'Jaime-Lannister', 'Kevan-Lannister', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Varys'}, {'Addam-Marbrand', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Addam-Marbrand', 'Jalabhar-Xho', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Petyr-Baelish', 'Merrett-Frey'}, {'Lothar-Frey', 'Merrett-Frey'}, {'Lanna-(Happy-Port)', 'Dareon'}, {'Ben-Plumm', 'Kasporio', 'Tybero-Istarion', 'Tyrion-Lannister'}, {'Hotho-Harlaw', 'Sigfryd-Harlaw', 'Rodrik-Harlaw'}, {'Holly', 'Theon-Greyjoy', 'Jeyne-Poole', 'Rowan'}, {'Aenys-Frey', 'Theon-Greyjoy', 'Arya-Stark'}, {'Theon-Greyjoy', 'Stygg', 'Werlag'}, {'Tywin-Lannister', 'Walder-Frey', 'Jaime-Lannister', 'Robb-Stark', 'Theon-Greyjoy', 'Roose-Bolton', 'Catelyn-Stark'}, {'Jon-Umber-(Greatjon)', 'Tywin-Lannister', 'Walder-Frey', 'Robb-Stark', 'Theon-Greyjoy', 'Roose-Bolton', 'Catelyn-Stark'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Theon-Greyjoy', 'Rickon-Stark', 'Osha'}, {'Bran-Stark', 'Jon-Umber-(Greatjon)', 'Robb-Stark', 'Theon-Greyjoy', 'Catelyn-Stark'}, {'Bran-Stark', 'Luwin', 'Hallis-Mollen', 'Robb-Stark', 'Theon-Greyjoy', 'Catelyn-Stark'}, {'Theon-Greyjoy', 'Jason-Mallister', 'Catelyn-Stark'}, {'Bran-Stark', 'Theon-Greyjoy', 'Beth-Cassel', 'Rodrik-Cassel'}, {'Harwood-Stout', 'Theon-Greyjoy'}, {'Aenys-Frey', 'Theon-Greyjoy', 'Wyman-Manderly'}, {'Holly', 'Theon-Greyjoy', 'Frenya'}, {'Theon-Greyjoy', 'Farlen'}, {'Theon-Greyjoy', 'Asha-Greyjoy', 'Wex-Pyke'}, {'Theon-Greyjoy', 'Lorren', 'Urzen'}, {'Theon-Greyjoy', 'Lorren', 'Wex-Pyke'}, {'Theon-Greyjoy', 'Captain-Myraham-Daughter'}, {'Jonothor-Darry', 'Lewyn-Martell', 'Oswell-Whent'}, {'Arthur-Dayne', 'Lewyn-Martell', 'Gerold-Hightower'}, {'Oswell-Whent', 'Lewyn-Martell', 'Gerold-Hightower'}, {'Mace-Tyrell', 'Robert-Strong', 'Cersei-Lannister'}, {'Halys-Hornwood', 'Rodrik-Cassel'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Gendry', 'Lem', 'Arya-Stark', 'Thoros-of-Myr'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Gendry', 'Lem', 'Arya-Stark', 'Harwin'}, {'Lem', 'Sandor-Clegane', 'Beric-Dondarrion', 'Jaime-Lannister', 'Arya-Stark', 'Thoros-of-Myr'}, {'Sandor-Clegane', 'Beric-Dondarrion', 'Lem', 'Anguy', 'Arya-Stark', 'Harwin'}, {'Jaime-Lannister', 'Beric-Dondarrion', 'Mariya-Darry'}, {'Jack-Bulwer', 'Beric-Dondarrion', 'Thoros-of-Myr'}, {'Sandor-Clegane', 'Amory-Lorch', 'Arya-Stark', 'Cersei-Lannister', 'Ilyn-Payne', 'Rafford', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Meryn-Trant'}, {'Bran-Stark', 'Cley-Cerwyn', 'Donella-Hornwood', 'Rodrik-Cassel'}, {'Bran-Stark', 'Luwin', 'Donella-Hornwood', 'Rodrik-Cassel'}, {'Mors-Umber', 'Donella-Hornwood'}, {'Rodrik-Cassel', 'Donella-Hornwood', 'Leobald-Tallhart'}, {'Aemon-Targaryen-(Maester-Aemon)', 'Wynton-Stout'}, {'Corliss-Penny', 'Stannis-Baratheon'}, {'Moreo-Tumitis', 'Catelyn-Stark'}, {'Craster', 'Byam-Flint'}, {'Victarion-Greyjoy', 'Dunstan-Drumm', 'Rodrik-Harlaw'}, {'Arya-Stark', 'Syrio-Forel', 'Meryn-Trant'}, {'Arthur-Dayne', 'Smiling-Knight'}, {'Praed', 'Arya-Stark'}, {'Amabel', 'Arya-Stark', 'Harra'}, {'Zollo', 'Rorge', 'Vargo-Hoat', 'Jaime-Lannister'}, {'Tywin-Lannister', 'Amory-Lorch', 'Jaime-Lannister', 'Roose-Bolton', 'Arya-Stark', 'Vargo-Hoat'}, {'Tywin-Lannister', 'Jaime-Lannister', 'Brienne-of-Tarth', 'Roose-Bolton', 'Vargo-Hoat'}, {'Rorge', 'Arya-Stark', 'Vargo-Hoat', 'Jaime-Lannister'}, {'Pyg', 'Vargo-Hoat', 'Brienne-of-Tarth'}, {'Florian-the-Fool', 'Jonquil'}, {'Robb-Stark', 'Dacey-Mormont', 'Maege-Mormont', 'Catelyn-Stark'}, {'Robb-Stark', 'Sybell-Spicer', 'Jaime-Lannister'}, {'Robb-Stark', 'Rolph-Spicer'}, {'Elmar-Frey', 'Tywin-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Roose-Bolton', 'Arya-Stark', 'Catelyn-Stark'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Hodor', 'Nan'}, {'Jeyne-Westerling', 'Robb-Stark', 'Walder-Frey', 'Catelyn-Stark', 'Lothar-Frey'}, {'Tywin-Lannister', 'Stannis-Baratheon', 'Tyrion-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Roose-Bolton', 'Catelyn-Stark', 'Brienne-of-Tarth'}, {'Elmar-Frey', 'Tywin-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Roose-Bolton', 'Catelyn-Stark', 'Brienne-of-Tarth'}, {'Elmar-Frey', 'Tywin-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Walder-Frey', 'Roose-Bolton', 'Catelyn-Stark'}, {'Tywin-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Walder-Frey', 'Catelyn-Stark', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Robb-Stark', 'Elmar-Frey', 'Catelyn-Stark', 'Brienne-of-Tarth'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Robb-Stark', 'Walder-Frey', 'Elmar-Frey', 'Catelyn-Stark'}, {'Tyrion-Lannister', 'Jaime-Lannister', 'Robb-Stark', 'Cleos-Frey', 'Catelyn-Stark', 'Cersei-Lannister', 'Brienne-of-Tarth'}, {'Bran-Stark', 'Wyman-Manderly', 'Robb-Stark', 'Hodor', 'Rodrik-Cassel'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Hodor', 'Rickon-Stark', 'Rodrik-Cassel'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Hodor', 'Rickon-Stark', 'Osha'}, {'Robb-Stark', 'Walder-Frey', 'Roose-Bolton', 'Catelyn-Stark', 'Lothar-Frey'}, {'Lothar-Frey', 'Edmure-Tully', 'Robb-Stark', 'Walder-Frey', 'Catelyn-Stark', 'Roslin-Frey'}, {'Robb-Stark', 'Jon-Umber-(Smalljon)', 'Jon-Umber-(Greatjon)'}, {'Bran-Stark', 'Jon-Umber-(Greatjon)', 'Rickard-Karstark', 'Robb-Stark', 'Catelyn-Stark'}, {'Jon-Umber-(Greatjon)', 'Rickard-Karstark', 'Robb-Stark', 'Roose-Bolton', 'Catelyn-Stark'}, {'Jon-Umber-(Greatjon)', 'Rickard-Karstark', 'Robb-Stark', 'Maege-Mormont', 'Catelyn-Stark'}, {'Robb-Stark', 'Wendel-Manderly', 'Catelyn-Stark', 'Jon-Umber-(Greatjon)'}, {'Robb-Stark', 'Wendel-Manderly', 'Catelyn-Stark', 'Hallis-Mollen'}, {'Edmure-Tully', 'Robb-Stark', 'Hoster-Tully', 'Catelyn-Stark', 'Lothar-Frey'}, {'Bran-Stark', 'Luwin', 'Robb-Stark', 'Hullen', 'Rodrik-Cassel'}, {'Robb-Stark', 'Bran-Stark', 'Stiv'}, {'Robb-Stark', 'Willem-Lannister'}, {'Aerys-II-Targaryen', 'Rhaegar-Targaryen', 'Arthur-Dayne'}, {'Tywin-Lannister', 'Jaime-Lannister', 'Elia-Martell', 'Rhaegar-Targaryen', 'Cersei-Lannister'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Tywin-Lannister', 'Rhaegar-Targaryen', 'Elia-Martell'}, {'Daenerys-Targaryen', 'Mace-Tyrell', 'Cersei-Lannister', 'Tyrion-Lannister'}, {'Joffrey-Baratheon', 'Mace-Tyrell', 'Paxter-Redwyne', 'Tyrion-Lannister'}, {'Paxter-Redwyne', 'Mace-Tyrell', 'Mathis-Rowan'}, {'Donnel-Waynwood', 'Catelyn-Stark'}, {'Eldon-Estermont', 'Bryce-Caron'}, {'Arianne-Martell', 'Myrcella-Baratheon', 'Andrey-Dalt', 'Garin-(orphan)', 'Sylva-Santagar'}, {'Hot-Pie', 'Mebble'}, {'Waymar-Royce', 'Benjen-Stark'}, {'Morgo', 'Tyrion-Lannister'}, {'Pate-(novice)', 'Rosey'}, {'Kindly-Man', 'Arya-Stark', 'Waif'}, {'Sandor-Clegane', 'Mandon-Moore', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Balon-Swann'}, {'Mandon-Moore', 'Tyrion-Lannister', 'Varys', 'Bronn', 'Joffrey-Baratheon', 'Cersei-Lannister'}, {'Dorcas', 'Taena-of-Myr', 'Jocelyn-Swyft', 'Cersei-Lannister'}, {'Tyrion-Lannister', 'Bronn', 'Catelyn-Stark', 'Willis-Wode', 'Marillion', 'Rodrik-Cassel'}, {'Tyrion-Lannister', 'Bronn', 'Joffrey-Baratheon', 'Catelyn-Stark', 'Rodrik-Cassel'}, {'Tyrion-Lannister', 'Catelyn-Stark', 'Petyr-Baelish', 'Marillion', 'Rodrik-Cassel'}, {'Daenerys-Targaryen', 'Quaithe'}, {'Brienne-of-Tarth', 'Creighton-Longbough', 'Illifer'}, {'Gregor-Clegane', 'Helman-Tallhart'}, {'Rhaegel-Targaryen', 'Daenerys-Targaryen'}, {'Jack-Be-Lucky', 'Lem'}, {'Aenys-Frey', 'Wyman-Manderly', 'Hosteen-Frey'}, {'Samwell-Tarly', 'Alleras', 'Leo-Tyrell'}, {'Samwell-Tarly', 'Marwyn', 'Alleras'}, {'Samwell-Tarly', 'Alleras', 'Aemon-Targaryen-(Maester-Aemon)'}, {'Hobber-Redwyne', 'Horas-Redwyne'}, {'Brienne-of-Tarth', 'Clarence-Crabb'}, {'Jeyne-Westerling', 'Gawen-Westerling'}, {'Sybell-Spicer', 'Gawen-Westerling'}, {'Jaime-Lannister', 'Tywin-Lannister', 'Wylis-Manderly'}, {'Stannis-Baratheon', 'Orton-Merryweather', 'Tommen-Baratheon', 'Aurane-Waters', 'Harys-Swyft', 'Cersei-Lannister'}, {'Orton-Merryweather', 'Tommen-Baratheon', 'Aurane-Waters', 'Harys-Swyft', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Orton-Merryweather', 'Tommen-Baratheon', 'Aurane-Waters', 'Margaery-Tyrell', 'Gyles-Rosby', 'Cersei-Lannister'}, {'Stannis-Baratheon', 'Tommen-Baratheon', 'Aurane-Waters', 'Cersei-Lannister', 'Loras-Tyrell'}, {'Tommen-Baratheon', 'Aurane-Waters', 'Margaery-Tyrell', 'Cersei-Lannister', 'Loras-Tyrell'}, {'Aurane-Waters', 'Paxter-Redwyne'}, {'Genna-Lannister', 'Tywin-Lannister', 'Emmon-Frey', 'Jaime-Lannister'}, {'Haldon', 'Qavo-Nogarys', 'Tyrion-Lannister'}, {'Daenerys-Targaryen', 'Qavo-Nogarys', 'Tyrion-Lannister'}, {'Quentyn-Martell', 'Old-Bill-Bone'}, {'Tytos-Blackwood', 'Jonos-Bracken', 'Jaime-Lannister'}, {'Brienne-of-Tarth', 'Sandor-Clegane', 'Timeon'}, {'Cersei-Lannister', 'Tyrion-Lannister', 'Preston-Greenfield'}, {'Arianne-Martell', 'Doran-Martell', 'Quentyn-Martell'}, {'Arianne-Martell', 'Doran-Martell', 'Myrcella-Baratheon', 'Areo-Hotah', 'Balon-Swann'}, {'Arianne-Martell', 'Doran-Martell', 'Myrcella-Baratheon', 'Areo-Hotah', 'Arys-Oakheart'}, {'Arianne-Martell', 'Doran-Martell', 'Obara-Sand', 'Areo-Hotah'}, {'Arianne-Martell', 'Franklyn-Fowler'}, {'Arianne-Martell', 'Mellei', 'Morra'}, {'Arianne-Martell', 'Ricasso'}, {'Tommen-Baratheon', 'Luwin', 'Tyrion-Lannister', 'Bran-Stark'}, {'Bran-Stark', 'Luwin', 'Joseth'}, {'Drogo', 'Rhaego'}, {'Jaime-Lannister', 'Joy-Hill'}, {'Quill', 'Pate-(novice)'}, {'Penny', 'Sweets', 'Yezzan-zo-Qaggaz', 'Tyrion-Lannister'}, {'Kevan-Lannister', 'Willem-Lannister'}, {'Tion-Frey', 'Willem-Lannister'}, {'Tommen-Baratheon', 'High-Sparrow', 'Maegor-I-Targaryen', 'Cersei-Lannister'}, {'Tommen-Baratheon', 'High-Sparrow', 'Cersei-Lannister', 'Kevan-Lannister'}, {'Tywin-Lannister', 'Stannis-Baratheon', 'Bronn', 'Tyrion-Lannister', 'Tommen-Baratheon', 'Joffrey-Baratheon', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Edmure-Tully', 'Jaime-Lannister', 'Tommen-Baratheon', 'Brienne-of-Tarth'}, {'Bronn', 'Tyrion-Lannister', 'Tommen-Baratheon', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Jacelyn-Bywater'}, {'Orton-Merryweather', 'Taena-of-Myr', 'Tommen-Baratheon', 'Harys-Swyft', 'Cersei-Lannister'}, {'Orton-Merryweather', 'Taena-of-Myr', 'Tommen-Baratheon', 'Margaery-Tyrell', 'Cersei-Lannister'}, {'Stannis-Baratheon', 'Jaime-Lannister', 'Tommen-Baratheon', 'Harys-Swyft', 'Cersei-Lannister'}, {'Tyrion-Lannister', 'Tommen-Baratheon', 'Joffrey-Baratheon', 'Gyles-Rosby', 'Cersei-Lannister', 'Jacelyn-Bywater'}, {'Taena-of-Myr', 'Jaime-Lannister', 'Tommen-Baratheon', 'Harys-Swyft', 'Cersei-Lannister'}, {'Tommen-Baratheon', 'Pate-(whipping-boy)'}, {'Aggar', 'Gelmarr'}, {'Aggar', 'Gariss'}, {'Cletus-Yronwood', 'Archibald-Yronwood', 'Gerris-Drinkwater', 'Quentyn-Martell'}, {'Tattered-Prince', 'Archibald-Yronwood', 'Gerris-Drinkwater', 'Quentyn-Martell'}, {'Aegon-Targaryen-(son-of-Rhaegar)', 'Tywin-Lannister', 'Tyrion-Lannister', 'Elia-Martell'}, {'Pia', 'Josmyn-Peckledon', 'Jaime-Lannister'}, {'Pia', 'Tyrion-Lannister', 'Jaime-Lannister'}, {'Victarion-Greyjoy', 'Euron-Greyjoy', 'Hotho-Harlaw', 'Rodrik-Harlaw'}, {'Cersei-Lannister', 'Garth-Tyrell'}, {'High-Sparrow', 'Cersei-Lannister', 'Unella'}, {'Yurkhaz-zo-Yunzak', 'Daenerys-Targaryen'}, {'Ygritte', 'Bael-the-Bard'}, {'Stannis-Baratheon', 'Orton-Merryweather', 'Petyr-Baelish', 'Pycelle', 'Cersei-Lannister'}, {'Stannis-Baratheon', 'Orton-Merryweather', 'Harys-Swyft', 'Pycelle', 'Cersei-Lannister'}, {'Orton-Merryweather', 'Harys-Swyft', 'Gyles-Rosby', 'Pycelle', 'Cersei-Lannister'}, {'Mors-Umber', 'Stannis-Baratheon', 'Arnolf-Karstark'}, {'Mors-Umber', 'Hother-Umber'}, {'Bronn', 'Chiggen', 'Willis-Wode', 'Tyrion-Lannister'}, {'Aemon-Targaryen-(Maester-Aemon)', 'Maekar-I-Targaryen'}, {'Aemon-Targaryen-(Maester-Aemon)', 'Aegon-V-Targaryen'}, {'Rickard-Karstark', 'Arnolf-Karstark'}, {'Stannis-Baratheon', 'Asha-Greyjoy', 'Arnolf-Karstark'}, {'Arthor-Karstark', 'Arnolf-Karstark'}, {'Hallyne', 'Joffrey-Baratheon', 'Tyrion-Lannister'}, {'Stannis-Baratheon', 'Tywin-Lannister', 'Tyrion-Lannister', 'Varys', 'Bronn', 'Joffrey-Baratheon', 'Catelyn-Stark', 'Cersei-Lannister'}, {'Tyrion-Lannister', 'Varys', 'Bronn', 'Joffrey-Baratheon', 'Cersei-Lannister', 'Jacelyn-Bywater'}, {'Tywin-Lannister', 'Amory-Lorch', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Arya-Stark', 'Cersei-Lannister'}, {'Renly-Baratheon', 'Sandor-Clegane', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Loras-Tyrell', 'Balon-Swann'}, {'Myrcella-Baratheon', 'Sandor-Clegane', 'Tyrion-Lannister', 'Joffrey-Baratheon', 'Balon-Swann'}, {'Sandor-Clegane', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Meryn-Trant', 'Balon-Swann'}, {'Sandor-Clegane', 'Tyrion-Lannister', 'Jaime-Lannister', 'Joffrey-Baratheon', 'Gregor-Clegane', 'Loras-Tyrell', 'Balon-Swann'}, {'Joffrey-Baratheon', 'Horas-Redwyne'}, {'Tyrion-Lannister', 'Joffrey-Baratheon', 'High-Septon-(Tyrions)', 'Pycelle', 'Cersei-Lannister'}, {'Doran-Martell', 'Gregor-Clegane', 'Tyrion-Lannister', 'Balon-Swann'}, {'Samwell-Tarly', 'Paxter-Redwyne'}, {'Brienne-of-Tarth', 'Vyman', 'Catelyn-Stark'}, {'Hoster-Tully', 'Vyman', 'Catelyn-Stark'}, {'Brandon-Norrey', 'Torghen-Flint'}, {'Bump', 'Varamyr'}, {'Haggon', 'Varamyr'}, {'Thistle', 'Varamyr'}, {'Torbert', 'Raynard'}, {'Raynard', 'Cersei-Lannister'}, {'Lewys-Piper', 'Jaime-Lannister'}, {'Hagen', 'Grimtongue', 'Lorren'}, {'Harl', 'Lorren'}, {'Arya-Stark', 'Yorko-Terys'}, {'Meralyn', 'Arya-Stark'}, {'Arya-Stark', 'Nan', 'Harren-Hoare'}, {'Pate-(Old)', 'Arya-Stark'}, {'Dobber', 'Arya-Stark'}, {'Sandor-Clegane', 'Arya-Stark', 'Gendry', 'Amory-Lorch'}, {'Arya-Stark', 'Bellegere-Otherys'}, {'Umma', 'Arya-Stark'}, {'Daenerys-Targaryen', 'Lynesse-Hightower'}, {'Ogo', 'Fogo'}, {'Ricasso', 'Manfrey-Martell'}, {'Daenerys-Targaryen', 'Cletus-Yronwood', 'Quentyn-Martell', 'Gerris-Drinkwater'}, {'Cletus-Yronwood', 'Quentyn-Martell', 'Kedry'}, {'Hugo-Vance', 'Jaime-Lannister'}, {'Robin-Ryger', 'Catelyn-Stark', 'Jaime-Lannister'}, {'Stannis-Baratheon', 'Jaime-Lannister', 'Harys-Swyft', 'Pycelle', 'Cersei-Lannister'}, {'Tywin-Lannister', 'Jaime-Lannister', 'Walder-Frey', 'Pycelle', 'Cersei-Lannister'}, {'Tytos-Blackwood', 'Edmure-Tully', 'Catelyn-Stark', 'Jaime-Lannister'}, {'Shitmouth', 'Jaime-Lannister'}, {'Garrett-Paege', 'Jaime-Lannister'}, {'Rennifer-Longwaters', 'Jaime-Lannister'}, {'Puckens', 'Pycelle'}, {'Terrance-Lynderly', 'Gerold-Grafton'}, {'Aerys-II-Targaryen', 'Owen-Merryweather'}, {'Jon-Arryn', 'Hugh'}, {'Vardis-Egen', 'Catelyn-Stark', 'Bronn', 'Tyrion-Lannister'}, {'Clayton-Suggs', 'Stannis-Baratheon', 'Godry-Farring', 'Asha-Greyjoy'}, {'Stannis-Baratheon', 'Godry-Farring', 'Asha-Greyjoy', 'Justin-Massey'}, {'Barristan-Selmy', 'Widower'}, {'Merrit', 'Gendry'}, {'Ezzara', 'Daenerys-Targaryen'}, {'Tattered-Prince', 'Quentyn-Martell', 'Gerris-Drinkwater', 'Daenerys-Targaryen', 'Meris'}, {'Daenerys-Targaryen', 'Ghael', 'Cleon'}, {'Daenerys-Targaryen', 'Cleon', 'Quentyn-Martell'}, {'Daenerys-Targaryen', 'Eroeh'}, {'Daenerys-Targaryen', 'Victarion-Greyjoy', 'Moqorro'}, {'Daenerys-Targaryen', 'Moqorro', 'Tyrion-Lannister', 'Benerro'}, {'Oznak-zo-Pahl', 'Daenerys-Targaryen', 'Belwas'}, {'Lysono-Maar', 'Daenerys-Targaryen'}, {'Daenerys-Targaryen', 'Quentyn-Martell', 'Doran-Martell'}, {'Ben-Plumm', 'Daenerys-Targaryen', 'Penny', 'Tyrion-Lannister'}, {'Ben-Plumm', 'Daenerys-Targaryen', 'Tyrion-Lannister', 'Barristan-Selmy'}, {'Daenerys-Targaryen', 'Barsena'}, {'Daenerys-Targaryen', 'Symon-Stripeback', 'Marselen'}, {'Doran-Martell', 'Myrcella-Baratheon', 'Cersei-Lannister', 'Arys-Oakheart'}, {'Doran-Martell', 'Myrcella-Baratheon', 'Tyrion-Lannister', 'Balon-Swann'}, {'Dolf', 'Shagga'}, {'Grazhar', 'Galazza-Galare', 'Barristan-Selmy'}, {'Brienne-of-Tarth', 'Renly-Baratheon', 'Emmon-Cuy'}, {'Brienne-of-Tarth', 'Jeyne-Heddle'}, {'Brienne-of-Tarth', 'Biter'}, {'Mero', 'Grazdan-mo-Eraz'}, {'Murch', 'Gariss'}, {'Yezzan-zo-Qaggaz', 'Nurse', 'Penny', 'Tyrion-Lannister'}, {'Renly-Baratheon', 'Guyard-Morrigen', 'Bryce-Caron'}, {'Cersei-Lannister', 'Mark-Mullendore'}, {'Bran-Stark', 'Meera-Reed', 'Leaf'}, {'Haldon', 'Tyrion-Lannister', 'Benerro'}, {'Cersei-Lannister', 'Tyrion-Lannister', 'Vylarr'}, {'Red-Oarsman', 'Euron-Greyjoy', 'Asha-Greyjoy'}, {'Victarion-Greyjoy', 'Harras-Harlaw'}, {'Grisel', 'Petyr-Baelish'}, {'Tytos-Blackwood', 'Jason-Mallister', 'Catelyn-Stark'}, {'Renly-Baratheon', 'Colen-of-Greenpools', 'Catelyn-Stark'}, {'Catelyn-Stark', 'Brenett'}, {'Masha-Heddle', 'Catelyn-Stark'}, {'Aerys-II-Targaryen', 'Gerold-Hightower', 'Arthur-Dayne'}, {'Aerys-II-Targaryen', 'Denys-Darklyn'}, {'Tyrion-Lannister', 'Mord'}, {'Quentyn-Martell', 'Trystane-Martell'}, {'Softfoot', 'Lark'}, {'Samwell-Tarly', 'Quhuru-Mo', 'Xhondo'}, {'Hagen', 'Qarl-the-Maid', 'Asha-Greyjoy', 'Tristifer-Botley'}, {'Hagen', 'Grimtongue', 'Asha-Greyjoy'}, {'Forley-Prester', 'Emmon-Frey', 'Edmure-Tully'}, {'Mallador-Locke', 'Thoren-Smallwood'}, {'Shagga', 'Pycelle', 'Tyrion-Lannister'}, {'Tormund', 'Munda'}, {'Allard-Seaworth', 'Stannis-Baratheon', 'Marya-Seaworth'}, {'Aron-Santagar', 'Sandor-Clegane'}, {'Myrcella-Baratheon', 'Trystane-Martell'}, {'Tormund', 'Dryn'}, {'Torgon-Greyiron', 'Asha-Greyjoy'}, {'Jon-Umber-(Greatjon)', 'Hother-Umber'}, {'Stannis-Baratheon', 'Roose-Ryswell'}, {'Kurleket', 'Tyrion-Lannister'}, {'Rolfe-the-Dwarf', 'Asha-Greyjoy'}, {'Victarion-Greyjoy', 'Euron-Greyjoy', 'Asha-Greyjoy', 'Rodrik-Harlaw'}, {'Stannis-Baratheon', 'Richard-Horpe', 'Justin-Massey'}, {'Stannis-Baratheon', 'Azor-Ahai'}, {'Victarion-Greyjoy', 'Euron-Greyjoy', 'Nute', 'Rodrik-Harlaw'}, {'Victarion-Greyjoy', 'Euron-Greyjoy', 'Quellon-Greyjoy'}, {'Bran-Stark', 'Brynden-Rivers', 'Meera-Reed'}, {'Bran-Stark', 'Rickard-Stark'}, {'Bran-Stark', 'Poxy-Tym'}, {'Victarion-Greyjoy', 'Moqorro', 'Rodrik-Sparr'}, {'Ralf-Stonehouse', 'Victarion-Greyjoy', 'Nute'}, {'Victarion-Greyjoy', 'Ralf-(Limper)'}, {'Renly-Baratheon', 'Steffon-Baratheon', 'Stannis-Baratheon'}, {'Stannis-Baratheon', 'Lyanna-Mormont'}, {'Aerys-II-Targaryen', 'Elbert-Arryn'}, {'Rossart', 'Aerys-II-Targaryen'}, {'Rugen', 'Rennifer-Longwaters'}, {'Aegon-I-Targaryen', 'Nan', 'Harren-Hoare'}, {'Blane', 'Jeor-Mormont'}, {'Jafer-Flowers', 'Jaremy-Rykker'}, {'Garlan-Tyrell', 'Leonette-Fossoway'}]

In [62]:
hierarchical_clustering = [['Harry-Strickland', 'Myles-Toyne', 'Cedra', 'Elder-Brother', 'Meribald', 'Narbert', 'Addam-Marbrand', 'Brynden-Tully', 'Lyle-Crakehall', 'Tyrion-Lannister', 'Gyles-Rosby', 'Joffrey-Baratheon', 'Olenna-Redwyne', 'Mace-Tyrell', 'Jaime-Lannister', 'Oberyn-Martell', 'Sansa-Stark', 'Kevan-Lannister', 'Barristan-Selmy', 'Pycelle', 'Boros-Blount', 'Cersei-Lannister', 'Tanda-Stokeworth', 'Mandon-Moore', 'Varys', 'Lancel-Lannister', 'Jacelyn-Bywater', 'Janos-Slynt', 'Stannis-Baratheon', 'Shae', 'Bronn', 'Harys-Swyft', 'Tommen-Baratheon', 'Orton-Merryweather', 'Aurane-Waters', 'Margaery-Tyrell', 'Loras-Tyrell', 'Taena-of-Myr', 'Osney-Kettleblack', 'Osmund-Kettleblack', 'Qyburn', 'Brienne-of-Tarth', 'Robert-Baratheon', 'Tywin-Lannister', 'Gregor-Clegane', 'Petyr-Baelish', 'Sandor-Clegane', 'Balon-Swann', 'Renly-Baratheon', 'Meryn-Trant', 'Chiswyck', 'Ilyn-Payne', 'Amory-Lorch', 'Arya-Stark', 'Rodrik-Cassel', 'Roose-Bolton', 'Polliver', 'Doran-Martell', 'Myrcella-Baratheon', 'Robb-Stark', 'Jory-Cassel', 'Theon-Greyjoy', 'Walder-Frey', 'Elmar-Frey', 'Edmure-Tully', 'Catelyn-Stark', 'Beric-Dondarrion', 'Eddard-Stark', 'Lysa-Arryn', 'Gendry', 'Marq-Piper', 'Jon-Snow', 'Mordane', 'Jeyne-Poole', 'Bran-Stark', 'Yoren', 'Benjen-Stark', 'Jon-Arryn', 'Aerys-II-Targaryen', 'Jon-Umber-(Greatjon)', 'Lothar-Frey', 'Mathis-Rowan', 'Daenerys-Targaryen', 'Hoster-Tully', 'Rickard-Karstark', 'Galbart-Glover', 'Rhaegar-Targaryen', 'Gerold-Hightower', 'Dunsen', 'Wyman-Manderly', 'Ramsay-Snow', 'Mance-Rayder', 'Randyll-Tarly', 'Elia-Martell', 'Rafford', 'Viserys-Targaryen', 'Tickler', 'Luwin', 'Hullen', 'Harwin', 'Hyle-Hunt', 'Weese', 'Ryman-Frey', 'Willas-Tyrell', 'Dorcas', 'Arys-Oakheart', 'Dontos-Hollard', 'Stafford-Lannister', 'Jeor-Mormont', 'Garlan-Tyrell', 'Robert-Arryn', 'Marillion', 'Walder-Frey-(son-of-Merrett)', 'Rickon-Stark', 'Brandon-Stark', 'Robert-Strong', 'Barbrey-Dustin', 'Jocelyn-Swyft', 'Vayon-Poole', 'Maegor-I-Targaryen', 'High-Sparrow', 'Balon-Greyjoy', 'Shireen-Baratheon', 'Cortnay-Penrose', 'Davos-Seaworth', 'Sybell-Spicer', 'Robett-Glover', 'Vargo-Hoat', 'Willis-Wode', 'Stevron-Frey', 'Walder-Rivers', 'Nestor-Royce', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Tomard', 'Paxter-Redwyne', 'Hodor', 'Samwell-Tarly', 'Melisandre', 'Godry-Farring', 'Alliser-Thorne', 'Bowen-Marsh', 'Hobb', 'Aemon-Targaryen-(Maester-Aemon)', 'Donal-Noye', 'Eddison-Tollett', 'Denys-Mallister', 'Othell-Yarwyck', 'Cotter-Pyke', 'Pypar', 'Owen', 'Satin', 'Mully', 'Selyse-Florent', 'Cressen', 'Spare-Boot', 'Kegs', 'Grenn', 'Hareth-(Moles-Town)', 'Alester-Florent', 'Axell-Florent', 'Ottyn-Wythers', 'Tansy', 'Lem', 'Yohn-Royce', 'Ellaria-Sand', 'Areo-Hotah', 'Arianne-Martell', 'Alyn', 'Aegon-Frey-(son-of-Stevron)', 'Emmon-Frey', 'Jon-Connington', 'Rorge', 'Jaqen-Hghar', 'Devan-Seaworth', 'Donella-Hornwood', 'Clydas', 'Lollys-Stokeworth', 'Woth', 'Helman-Tallhart', 'Jalabhar-Xho', 'Tallad', 'Cellador', 'High-Septon-(Tyrions)', 'Lark', 'Lyanna-Stark', 'Aegon-I-Targaryen', 'Nan', 'Rast', 'Halder', 'Tysha', 'Cleos-Frey', 'Blue-Bard', 'Lambert-Turnberry', 'Edric-Storm', 'Albett', 'Zollo', 'Shagwell', 'Hallis-Mollen', 'Wendel-Manderly', 'Robin-Flint', 'Asha-Greyjoy', 'Dagmer', 'Small-Paul', 'Gilly', 'Meera-Reed', 'Coldhands', 'Dalla', 'Val', 'Anguy', 'Greenbeard', 'Tom-of-Sevenstreams', 'Hot-Pie', 'Sharna', 'Thoros-of-Myr', 'Sigorn', 'Bedwyck', 'Dywen', 'Craster', 'Jojen-Reed', 'Osha', 'Thoren-Smallwood', 'Qhorin-Halfhand', 'Chett', 'Jaremy-Rykker', 'Beth-Cassel', 'Karyl-Vance', 'Wyl-(guard)', 'Lothor-Brune', 'Jorah-Mormont', 'Alayaya', 'Arnolf-Karstark', 'Bryce-Caron', 'Symon-Silver-Tongue', 'Roslin-Frey', 'Walton', 'Podrick-Payne', 'Dick-Crabb', 'Baelor-I-Targaryen', 'Patchface', 'Cley-Cerwyn', 'Jack-Bulwer', 'Biter', 'Emmett', 'Edwyn-Frey', 'Maege-Mormont', 'Dacey-Mormont', 'Pia', 'Aenys-Frey', 'Tyene-Sand', 'Garin-(orphan)', 'Gerold-Dayne', 'Sylva-Santagar', 'Andrey-Dalt', 'Daven-Lannister', 'Tytos-Blackwood', 'Jason-Mallister', 'High-Septon-(fat_one)', 'Lyn-Corbray', 'Dareon', 'Syrio-Forel', 'Elinor-Tyrell', 'Alla-Tyrell', 'Hother-Umber', 'Mors-Umber', 'Pylos', 'Doreah', 'Pyg', 'Timeon', 'Hop-Robin', 'Tormund', 'Leathers', 'Ygritte', 'Weeper', 'Rattleshirt', 'Harma', 'Jarl', 'Styr', 'Genna-Lannister', 'Squirrel', 'Rowan', 'Holly', 'Mikken', 'Torrhen-Stark', 'Daeron-II-Targaryen', 'Alerie-Hightower', 'Monford-Velaryon', 'Emrick', 'Arron', 'Bloodbeard', 'Big-Boil', 'Howland-Reed', 'Eon-Hunter', 'Vardis-Egen', 'Colemon', 'Gretchel', 'Maddy', 'Utherydes-Wayn', 'Preston-Greenfield', 'Daemon-Sand', 'Forley-Prester', 'Wylis-Manderly', 'Jarman-Buckwell', 'Willem-Lannister', 'Merrett-Frey', 'Martyn-Lannister', 'Palla', 'Steffon-Baratheon', 'Rhaella-Targaryen', 'Theomore', 'Lamprey', 'Porridge', 'Chiggen', 'Arthur-Dayne', 'Lewyn-Martell', 'Oswell-Whent', 'Jonothor-Darry', 'Alebelly', 'Cayn', 'Wex-Pyke', 'Mya-Stone', 'Myranda-Royce', 'Victarion-Greyjoy', 'Ryk', 'Chella', 'Mycah', 'Clayton-Suggs', 'Drogo', 'Pono', 'Illyrio-Mopatis', 'Irri', 'Haldon', 'Rakharo', 'Daario-Naharis', 'Jhogo', 'Aggo', 'Grey-Worm', 'Quaro', 'Hizdahr-zo-Loraq', 'Jhiqui', 'Belwas', 'Skahaz-mo-Kandaq', 'Ben-Plumm', 'Missandei', 'Quentyn-Martell', 'Groleo', 'Arstan', 'Benerro', 'Moqorro', 'Hero', 'Penny', 'Steelskin', 'Belaquo', 'Spotted-Cat', 'Khrazz', 'Goghor-the-Giant', 'Kasporio', 'Tybero-Istarion', 'Tattered-Prince', 'Gerris-Drinkwater', 'Caggo', 'Meris', 'Cletus-Yronwood', 'Kedry', 'Mirri-Maz-Duur', 'Haggo', 'Qotho', 'Nurse', 'Shagga', 'Timett', 'Desmond-Grell', 'Yandry', 'Lemore', 'Ysilla', 'Rolly-Duckfield', 'Harren-Hoare', 'Varamyr', 'Franklyn-Flowers', 'Baelor-Blacktyde', 'Aeron-Greyjoy', 'Euron-Greyjoy', 'Rodrik-Harlaw', 'Dunstan-Drumm', 'Hotho-Harlaw', 'Allar-Deem', 'Anders-Yronwood', 'Husband', 'Harrold-Hardyng', 'Desmond', 'Chayle', 'Joseth', 'Reek', 'Rickard-Stark', 'Stiv', 'Clement-Piper', 'Garth-(Wolfs-Den)', 'Guncer-Sunglass', 'Khorane-Sathmantes', 'Salladhor-Saan', 'Leona-Woolfield', 'Willam-Dustin', 'Errok', 'Grigg', 'Humfrey-Hewett', 'Quellon-Greyjoy', 'Garth-Greyfeather', 'Hairy-Hal', 'Sigfryd-Harlaw', 'Ronnet-Connington', 'Lyonel-Corbray', 'Walda-Frey-(daughter-of-Merrett)', 'Stonesnake', 'Archibald-Yronwood', 'Jeyne-Westerling', 'Gawen-Westerling', 'Qavo-Nogarys', 'Cohollo', 'Tycho-Nestoris', 'Vyman', 'Nymeria-Sand', 'Falyse-Stokeworth', 'Toregg', 'Lommy-Greenhands', 'Weasel', 'Allard-Seaworth', 'Marya-Seaworth', 'Koss', 'Leobald-Tallhart', 'Benfred-Tallhart', 'Stygg', 'Werlag', 'Willow-Heddle', 'Melara-Hetherspoon', 'Maggy', 'Torrhen-Karstark', 'Daryn-Hornwood', 'Marlon-Manderly', 'Dick-Follard', 'Hallyne', 'Heward', 'Amerei-Frey', 'Nute', 'Ralf-Stonehouse', 'Hayhead', 'Justin-Massey', 'Andrew-Estermont', 'Red-Oarsman', 'Roger-Ryswell', 'Colen-of-Greenpools', 'Vylarr', 'Clubfoot-Karl', 'Qezza', 'Mezzara', 'Miklaz', 'Grazhar', 'Galazza-Galare', 'Reznak-mo-Reznak', 'Mag-Mar-Tun-Doh-Weg', 'Frenya', 'Orell', 'Perwyn-Frey', 'Petyr-Frey', 'Moon-Boy', 'Caleotte', 'Obara-Sand', 'Osfryd-Kettleblack', 'Meldred-Merlyn', 'Robar-Royce', 'Emmon-Cuy', 'Richard-Horpe', 'Ebben', 'Dalbridge', 'Anya-Waynwood', 'Symond-Templeton', 'Gilwood-Hunter', 'Benedar-Belmore', 'Horton-Redfort', 'Narbert-Grandison', 'Malegorn', 'Patrek-of-Kings-Mountain', 'Wun-Weg-Wun-Dar-Wun', 'Brus-Buckler', 'Benethon-Scales', 'Alysane-Mormont', 'Mero', 'Ben-Bones', 'Skinner', 'Damon-Dance-for-Me', 'Sour-Alyn', 'Grunt', 'Yellow-Dick', 'Fulk', 'Jon-Umber-(Smalljon)', 'Oswell-Kettleblack', 'Robin-Ryger', 'Senelle', 'Jonos-Bracken', 'Alys-Karstark', 'Gorold-Goodbrother', 'Walder-Frey-(son-of-Jammos)', 'Todder', 'Murenmure', 'Butterbumps', 'Dirk', 'Rory', 'Joramun', 'Cleon', 'Brynden-Rivers', 'Trystane-Martell', 'Wick-Whittlestick', 'Megga-Tyrell', 'Kraznys-mo-Nakloz', 'Ballabar', 'Lorren', 'Hagen', 'Grimtongue', 'Sybelle-Glover', 'Squint', 'Drennan', 'Urzen', 'Cromm', 'Qarl-the-Maid', 'Tristifer-Botley', 'Rodrik-Sparr', 'Sweets', 'Yezzan-zo-Qaggaz', 'Balman-Byrch', 'Alleras', 'Marwyn', 'Armen', 'Leo-Tyrell', 'Roone', 'Mollander', 'Pate-(novice)', 'Oznak-zo-Pahl', 'Brella', 'Raymun-Darry', 'Scar', 'Xaro-Xhoan-Daxos', 'Hosteen-Frey', 'Danwell-Frey', 'Jared-Frey', 'Rhaegar-Frey', 'Mariya-Darry', 'Leaf', 'Guyard-Morrigen', 'Visenya-Targaryen', 'Rhaenys-Targaryen', 'Aemon-Targaryen-(Dragonknight)', 'Maekar-I-Targaryen', 'Wynton-Stout', 'Aerys-I-Targaryen', 'Elbert-Arryn', 'Owen-Merryweather', 'Dancy', 'Alys-Arryn', 'Harra', 'Amabel', 'Gerald-Gower', 'Franklyn-Fowler', 'Mellei', 'Morra', 'Ricasso', 'Timoth', 'Arthor-Karstark', 'Aron-Santagar', 'Arson', 'Arwyn-Oakheart', 'Bellegere-Otherys', 'Praed', 'Rolfe-the-Dwarf', 'Torgon-Greyiron', 'Ashara-Dayne', 'Larraq', 'Widower', 'Waymar-Royce', 'Black-Balaq', 'Blane', 'Hali', 'Poxy-Tym', 'Brandon-Norrey', 'Torghen-Flint', 'Brenett', 'Eldon-Estermont', 'Byam-Flint', 'Lucifer-Long', 'Camarron', 'Captain-Myraham-Daughter', 'Donnel-Waynwood', 'Moreo-Tumitis', 'Mychel-Redfort', 'Cedric-Payne', 'Garth-Tyrell', 'Mark-Mullendore', 'Raynard', 'Corliss-Penny', 'Shadrich', 'Creighton-Longbough', 'Cutjack', 'Tarber', 'Prendahl-na-Ghezn', 'Sallor', 'Ezzara', 'Lynesse-Hightower', 'Lysono-Maar', 'Marselen', 'Symon-Stripeback', 'Daeron-I-Targaryen', 'Imry-Florent', 'Matthos-Seaworth', 'Denzo-Dhan', 'Eustace-Brune', 'Draqaz', 'Jommo', 'Ogo', 'Dryn', 'Maron-Greyjoy', 'Wylla', 'Watt', 'Steffon-Varner', 'Sawane-Botley', 'Florian-the-Fool', 'Galyeon-of-Cuy', 'Leonette-Fossoway', 'Garrett-Paege', 'Gendel', 'Gorne', 'Merrit', 'Grazdan-mo-Eraz', 'Grisel', 'Gyles-Grafton', 'Terrance-Lynderly', 'Gerold-Grafton', 'Halys-Hornwood', 'Harl', 'Harras-Harlaw', 'Horas-Redwyne', 'Hugh', 'Hugo-Vance', 'Jaehaerys-I-Targaryen', 'Jafer-Flowers', 'Joy-Hill', 'Lewys-Piper', 'Rennifer-Longwaters', 'Shitmouth', 'Nyessos-Vhassar', 'Jon-Fossoway', 'Matthar', 'Soren-Shieldbreaker', 'Ulmer', 'Zei', 'Kella', 'Softfoot', 'Longwater-Pyke', 'Lorent-Caswell', 'Lucas-Codd', 'Torwold-Browntooth', 'Luton', 'Lyanna-Mormont', 'Martyn-Rivers', 'Nage', 'Old-Bill-Bone', 'Olyvar-Frey', 'Puckens', 'Shortear', 'Ragwyle', 'Rugen', 'Tristifer-IV-Mudd', 'Roose-Ryswell', 'Sarella-Sand', 'Shadd', 'William-Foxglove', 'Wulfe', 'Aggar', 'Gariss', 'Gelmarr', 'Gynir', 'Raynald-Westerling', 'Rollam-Westerling', 'Gared', 'Denyo-Terys', 'Ternesio-Terys', 'Ghael', 'Bannen', 'Conn', 'Kojja-Mo', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Emmond', 'Greydon-Goodbrother', 'Denys-Darklyn', 'Rossart', 'Elys-Waynwood', 'Smiling-Knight', 'Simon-Toyne', 'Dobber', 'Mad-Huntsman', 'Pate-(Old)', 'Talea', 'Brea', 'Brusco', 'Frenken', 'Barsena', 'Bartimus', 'Bonifer-Hasty', 'Clarence-Crabb', 'Jeyne-Heddle', 'Selwyn-Tarth', 'Captain-Myraham', 'Moelle', 'Unella', 'Scolera', 'Kurz', 'Yurkhaz-zo-Yunzak', 'Lanna-(Happy-Port)', 'Dick-Straw', 'Hugh-Hungerford', 'Jacks', 'Porther', 'Tregar', 'Varly', 'Fogo', 'Hake', 'Hoster-Blackwood', 'Mebble', 'Jeyne-Farman', 'Joanna-Lannister', 'Tom-Barleycorn', 'Kyra', 'Leo-Lefford', 'Mallador-Locke', 'Manfrey-Martell', 'Morgo', 'Munda', 'Quill', 'Pate-(whipping-boy)', 'Quhuru-Mo', 'Xhondo', 'Torbert', 'Ulf-son-of-Umar', 'Shrouded-Lord', 'Willow-Witch-eye', 'Josmyn-Peckledon', 'Rus', 'Ardrian-Celtigar', 'Bael-the-Bard', 'Bump', 'Masha-Heddle', 'Coratt', 'Rhaegel-Targaryen', 'Rhaego', 'Joss', 'Murch', 'Gerrick-Kingsblood', 'Raymun-Redbeard', 'Harwood-Stout', 'Marghaz-zo-Loraq', 'Jack-Be-Lucky', 'Othor', 'Morrec', 'Jyck', 'Kurleket', 'Rolph-Spicer', 'Illifer', 'Pyat-Pree', 'Aegon-V-Targaryen', 'Gormond-Goodbrother', 'Umma', 'Yorko-Terys', 'Azor-Ahai', 'Farlen', 'Tion-Frey', 'Ravella-Swann', 'Tobho-Mott', 'Gunthor-son-of-Gurn', 'Hobber-Redwyne', 'Malaquo-Maegyr', 'Ralf-(Limper)', 'Theobald', 'Thistle', 'Will-(prologue)', 'Eroeh', 'Jonquil', 'Haggon', 'Meralyn', 'Godric-Borrell', 'Rosey', 'Chataya'], ['Kindly-Man'], ['Waif'], ['Quaithe'], ['Dolf'], ['Gormon-Tyrell', 'Walgrave'], ['Urswyck'], ['Kerwin'], ['Mord'], ['Talbert-Serry']]

In [63]:
spectral_clustering = {9: ['Addam-Marbrand', 'Daenerys-Targaryen', 'Theon-Greyjoy', 'Balon-Greyjoy', 'Barristan-Selmy', 'Brienne-of-Tarth', 'Jon-Arryn', 'Ramsay-Snow', 'Alayaya', 'Cortnay-Penrose', 'Lancel-Lannister', 'Chiswyck', 'Edric-Storm', 'Arys-Oakheart', 'Doran-Martell', 'Ellaria-Sand', 'Benjen-Stark', 'Mordane', 'Rodrik-Cassel', 'Vayon-Poole', 'Harys-Swyft', 'Baelor-I-Targaryen', 'Tanda-Stokeworth', 'Cleos-Frey', 'Lysa-Arryn', 'Garlan-Tyrell', 'Garth-Tyrell', 'High-Septon-(Tyrions)', 'Maegor-I-Targaryen', 'Mark-Mullendore', 'Preston-Greenfield', 'Raynard', 'Vylarr', 'High-Septon-(fat_one)', 'Hallyne'], 0: ['Brynden-Tully', 'Lyle-Crakehall', 'Varys', 'Aegon-Frey-(son-of-Stevron)', 'Stevron-Frey', 'Harren-Hoare', 'Rhaenys-Targaryen', 'Visenya-Targaryen', 'Aegon-Targaryen-(son-of-Rhaegar)', 'Franklyn-Flowers', 'Haldon', 'Harry-Strickland', 'Lemore', 'Rhaenys-Targaryen-(daughter-of-Rhaegar)', 'Rolly-Duckfield', 'Viserys-Targaryen', 'Yandry', 'Ysilla', 'Aegon-V-Targaryen', 'Alleras', 'Daeron-II-Targaryen', 'Eddison-Tollett', 'Gilly', 'Grenn', 'Hobb', 'Maekar-I-Targaryen', 'Mance-Rayder', 'Pypar', 'Wynton-Stout', 'Aenys-Frey', 'Hosteen-Frey', 'Wyman-Manderly', 'Aeron-Greyjoy', 'Baelor-Blacktyde', 'Dagmer', 'Dunstan-Drumm', 'Emmond', 'Euron-Greyjoy', 'Gormond-Goodbrother', 'Gorold-Goodbrother', 'Greydon-Goodbrother', 'Meldred-Merlyn', 'Murenmure', 'Rus', 'Victarion-Greyjoy', 'Arthur-Dayne', 'Denys-Darklyn', 'Elbert-Arryn', 'Owen-Merryweather', 'Rossart', 'Aggar', 'Gariss', 'Gelmarr', 'Gynir', 'Aggo', 'Belwas', 'Daario-Naharis', 'Grey-Worm', 'Jhogo', 'Jorah-Mormont', 'Quaro', 'Rakharo', 'Dancy', 'Halder', 'Rast', 'Alebelly', 'Hayhead', 'Jojen-Reed', 'Mikken', 'Alerie-Hightower', 'Lamprey', 'Armen', 'Leo-Tyrell', 'Marwyn', 'Mollander', 'Pate-(novice)', 'Roone', 'Qhorin-Halfhand', 'Harwin', 'Alys-Arryn', 'Elys-Waynwood', 'Alys-Karstark', 'Justin-Massey', 'Amabel', 'Harra', 'Dunsen', 'Hot-Pie', 'Sandor-Clegane', 'Tickler', 'Vargo-Hoat', 'Anders-Yronwood', 'Quentyn-Martell', 'Andrew-Estermont', 'Gerald-Gower', 'Andrey-Dalt', 'Arianne-Martell', 'Garin-(orphan)', 'Sylva-Santagar', 'Tyene-Sand', 'Anguy', 'Greenbeard', 'Husband', 'Sharna', 'Tom-of-Sevenstreams', 'Anya-Waynwood', 'Benedar-Belmore', 'Gilwood-Hunter', 'Harrold-Hardyng', 'Horton-Redfort', 'Symond-Templeton', 'Archibald-Yronwood', 'Cletus-Yronwood', 'Gerris-Drinkwater', 'Tattered-Prince', 'Ardrian-Celtigar', 'Areo-Hotah', 'Balon-Swann', 'Caleotte', 'Obara-Sand', 'Cedra', 'Franklyn-Fowler', 'Gerold-Dayne', 'Mellei', 'Morra', 'Ricasso', 'Timoth', 'Arthor-Karstark', 'Aron-Santagar', 'Arron', 'Emrick', 'Hareth-(Moles-Town)', 'Satin', 'Arstan', 'Jhiqui', 'Kraznys-mo-Nakloz', 'Mero', 'Lewyn-Martell', 'Smiling-Knight', 'Arwyn-Oakheart', 'Bellegere-Otherys', 'Biter', 'Brea', 'Brusco', 'Denyo-Terys', 'Dobber', 'Jaqen-Hghar', 'Kindly-Man', 'Koss', 'Lommy-Greenhands', 'Mad-Huntsman', 'Meralyn', 'Pate-(Old)', 'Praed', 'Ravella-Swann', 'Syrio-Forel', 'Talea', 'Ternesio-Terys', 'Tommen-Baratheon', 'Umma', 'Waif', 'Weasel', 'Weese', 'Woth', 'Yorko-Terys', 'Grimtongue', 'Hagen', 'Qarl-the-Maid', 'Red-Oarsman', 'Rodrik-Harlaw', 'Rolfe-the-Dwarf', 'Sybelle-Glover', 'Torgon-Greyiron', 'Tristifer-Botley', 'Wex-Pyke', 'Ashara-Dayne', 'Porridge', 'Salladhor-Saan', 'Bael-the-Bard', 'Ygritte', 'Ballabar', 'Frenken', 'Bannen', 'Roger-Ryswell', 'Bloodbeard', 'Galazza-Galare', 'Grazhar', 'Groleo', 'Hizdahr-zo-Loraq', 'Khrazz', 'Larraq', 'Missandei', 'Pycelle', 'Reznak-mo-Reznak', 'Skahaz-mo-Kandaq', 'Steelskin', 'Widower', 'Xaro-Xhoan-Daxos', 'Barsena', 'Bartimus', 'Bedwyck', 'Craster', 'Belaquo', 'Goghor-the-Giant', 'Irri', 'Spotted-Cat', 'Oznak-zo-Pahl', 'Ben-Bones', 'Penny', 'Tybero-Istarion', 'Benerro', 'Moqorro', 'Benethon-Scales', 'Benfred-Tallhart', 'Helman-Tallhart', 'Leobald-Tallhart', 'Stygg', 'Waymar-Royce', 'Beth-Cassel', 'Black-Balaq', 'Blane', 'Lambert-Turnberry', 'Othell-Yarwyck', 'Tormund', 'Brynden-Rivers', 'Chayle', 'Cley-Cerwyn', 'Coldhands', 'Donella-Hornwood', 'Hali', 'Hallis-Mollen', 'Hodor', 'Hullen', 'Jon-Umber-(Greatjon)', 'Joseth', 'Leaf', 'Meera-Reed', 'Osha', 'Poxy-Tym', 'Reek', 'Rickard-Stark', 'Stiv', 'Walder-Frey-(son-of-Jammos)', 'Walder-Frey-(son-of-Merrett)', 'Torghen-Flint', 'Brella', 'Clarence-Crabb', 'Creighton-Longbough', 'Edmure-Tully', 'Elder-Brother', 'Emmon-Cuy', 'Illifer', 'Jeyne-Heddle', 'Meribald', 'Narbert', 'Pyg', 'Robar-Royce', 'Selwyn-Tarth', 'Timeon', 'Vyman', 'Walton', 'Willow-Heddle', 'Lollys-Stokeworth', 'Shagga', 'Brus-Buckler', 'Eldon-Estermont', 'Guyard-Morrigen', 'Bump', 'Butterbumps', 'Byam-Flint', 'Caggo', 'Lucifer-Long', 'Meris', 'Camarron', 'Captain-Myraham', 'Captain-Myraham-Daughter', 'Colemon', 'Colen-of-Greenpools', 'Eon-Hunter', 'Jason-Mallister', 'Jeyne-Westerling', 'Lothar-Frey', 'Maege-Mormont', 'Masha-Heddle', 'Mya-Stone', 'Raynald-Westerling', 'Robert-Arryn', 'Tansy', 'Cayn', 'Cedric-Payne', 'Dorcas', 'High-Sparrow', 'Jocelyn-Swyft', 'Osfryd-Kettleblack', 'Senelle', 'Unella', 'Small-Paul', 'Cleon', 'Ghael', 'Kedry', 'Clubfoot-Karl', 'Mully', 'Cohollo', 'Haggo', 'Qotho', 'Coratt', 'Dirk', 'Cressen', 'Pylos', 'Cromm', 'Cutjack', 'Kurz', 'Tarber', 'Hero', 'Prendahl-na-Ghezn', 'Sallor', 'Daemon-Sand', 'Doreah', 'Eroeh', 'Ezzara', 'Lynesse-Hightower', 'Lysono-Maar', 'Marselen', 'Mezzara', 'Miklaz', 'Mirri-Maz-Duur', 'Pono', 'Pyat-Pree', 'Qezza', 'Quaithe', 'Rhaegel-Targaryen', 'Symon-Stripeback', 'Yurkhaz-zo-Yunzak', 'Daeron-I-Targaryen', 'Ebben', 'Damon-Dance-for-Me', 'Grunt', 'Skinner', 'Sour-Alyn', 'Danwell-Frey', 'Jared-Frey', 'Lanna-(Happy-Port)', 'Garth-(Wolfs-Den)', 'Godric-Borrell', 'Guncer-Sunglass', 'Imry-Florent', 'Leona-Woolfield', 'Matthos-Seaworth', 'Denzo-Dhan', 'Eustace-Brune', 'Dick-Straw', 'Hugh-Hungerford', 'Dolf', 'Nymeria-Sand', 'Draqaz', 'Drennan', 'Squint', 'Jommo', 'Ogo', 'Rhaego', 'Dryn', 'Stonesnake', 'Tobho-Mott', 'Emmett', 'Watt', 'Forley-Prester', 'Steffon-Varner', 'Styr', 'Hotho-Harlaw', 'Humfrey-Hewett', 'Nute', 'Quellon-Greyjoy', 'Sawane-Botley', 'Farlen', 'Jonquil', 'Fogo', 'Frenya', 'Holly', 'Gared', 'Will-(prologue)', 'Murch', 'Leonette-Fossoway', 'Gawen-Westerling', 'Merrit', 'Gerold-Grafton', 'Terrance-Lynderly', 'Oswell-Whent', 'Raymun-Redbeard', 'Kojja-Mo', 'Gormon-Tyrell', 'Walgrave', 'Grazdan-mo-Eraz', 'Todder', 'Jarl', 'Lorren', 'Grisel', 'Gyles-Grafton', 'Haggon', 'Halys-Hornwood', 'Leathers', 'Harl', 'Harma', 'Rattleshirt', 'Harras-Harlaw', 'Myles-Toyne', 'Harwood-Stout', 'Heward', 'Marghaz-zo-Loraq', 'Hobber-Redwyne', 'Rowan', 'Mebble', 'Hother-Umber', 'Sigfryd-Harlaw', 'Hugh', 'Jack-Be-Lucky', 'Jaehaerys-I-Targaryen', 'Jafer-Flowers', 'Josmyn-Peckledon', 'Rhaegar-Frey', 'Jeyne-Farman', 'Squirrel', 'Nyessos-Vhassar', 'Jon-Fossoway', 'Patrek-of-Kings-Mountain', 'Ryk', 'Wun-Weg-Wun-Dar-Wun', 'Jon-Umber-(Smalljon)', 'Jonothor-Darry', 'Joramun', 'Kerwin', 'Willem-Lannister', 'Kyra', 'Softfoot', 'Symon-Silver-Tongue', 'Longwater-Pyke', 'Lorent-Caswell', 'Urzen', 'Merrett-Frey', 'Lucas-Codd', 'Torwold-Browntooth', 'Luton', 'Lyn-Corbray', 'Lyonel-Corbray', 'Malaquo-Maegyr', 'Mallador-Locke', 'Manfrey-Martell', 'Rodrik-Sparr', 'Mord', 'Munda', 'Trystane-Martell', 'Nage', 'Yezzan-zo-Qaggaz', 'Ralf-Stonehouse', 'Old-Bill-Bone', 'Olyvar-Frey', 'Palla', 'Quill', 'Rosey', 'Pate-(whipping-boy)', 'Perwyn-Frey', 'Petyr-Frey', 'Puckens', 'Shortear', 'Ragwyle', 'Quhuru-Mo', 'Xhondo', 'Ralf-(Limper)', 'Yellow-Dick', 'Rollam-Westerling', 'Torbert', 'Rugen', 'Rhaella-Targaryen', 'Rolph-Spicer', 'Walda-Frey-(daughter-of-Merrett)', 'Theobald', 'Sarella-Sand', 'Shadd', 'Ulf-son-of-Umar', 'Simon-Toyne', 'Werlag', 'Talbert-Serry', 'Willow-Witch-eye', 'Thistle', 'Tion-Frey', 'Wulfe'], 5: ['Cersei-Lannister', 'Joffrey-Baratheon', 'Tyrion-Lannister', 'Eddard-Stark', 'Robert-Baratheon', 'Sansa-Stark'], 7: ['Gyles-Rosby', 'Oberyn-Martell', 'Walder-Frey', 'Elia-Martell', 'Rhaegar-Targaryen', 'Aemon-Targaryen-(Dragonknight)', 'Gerold-Hightower', 'Amory-Lorch', 'Gregor-Clegane', 'Rafford', 'Beric-Dondarrion', 'Elmar-Frey', 'Galbart-Glover', 'Mandon-Moore', 'Stafford-Lannister', 'Dick-Crabb', 'Hoster-Tully', 'Olenna-Redwyne', 'Melara-Hetherspoon', 'Taena-of-Myr', 'Willas-Tyrell', 'Clement-Piper', 'Shadrich', 'Sybell-Spicer', 'Wyl-(guard)', 'Walder-Rivers', 'Garrett-Paege', 'Hugo-Vance', 'Joy-Hill', 'Lewys-Piper', 'Pia', 'Rennifer-Longwaters', 'Shitmouth', 'Tysha', 'Wylis-Manderly', 'Zollo'], 2: ['Jaime-Lannister'], 1: ['Jalabhar-Xho', 'Torrhen-Stark', 'Jon-Connington', 'Aemon-Targaryen-(Maester-Aemon)', 'Jeor-Mormont', 'Owen', 'Samwell-Tarly', 'Asha-Greyjoy', 'Aerys-I-Targaryen', 'Aerys-II-Targaryen', 'Brandon-Stark', 'Drogo', 'Mace-Tyrell', 'Margaery-Tyrell', 'Davos-Seaworth', 'Renly-Baratheon', 'Selyse-Florent', 'Alla-Tyrell', 'Elinor-Tyrell', 'Megga-Tyrell', 'Allar-Deem', 'Alyn', 'Tomard', 'Melisandre', 'Amerei-Frey', 'Gendry', 'Ilyn-Payne', 'Meryn-Trant', 'Polliver', 'Lem', 'Petyr-Baelish', 'Yohn-Royce', 'Rickard-Karstark', 'Illyrio-Mopatis', 'Desmond', 'Jeyne-Poole', 'Mycah', 'Rickon-Stark', 'Rorge', 'Thoros-of-Myr', 'Paxter-Redwyne', 'Podrick-Payne', 'Balman-Byrch', 'Falyse-Stokeworth', 'Boros-Blount', 'Osmund-Kettleblack', 'Ben-Plumm', 'Kasporio', 'Mariya-Darry', 'Blue-Bard', 'Tallad', 'Bonifer-Hasty', 'Howland-Reed', 'Lyanna-Stark', 'Brenett', 'Hyle-Hunt', 'Shagwell', 'Chiggen', 'Jacelyn-Bywater', 'Marillion', 'Timett', 'Vardis-Egen', 'Willis-Wode', 'Daven-Lannister', 'Edwyn-Frey', 'Emmon-Frey', 'Nestor-Royce', 'Ryman-Frey', 'Dacey-Mormont', 'Desmond-Grell', 'Donnel-Waynwood', 'Marq-Piper', 'Moreo-Tumitis', 'Mychel-Redfort', 'Robin-Ryger', 'Roslin-Frey', 'Tytos-Blackwood', 'Utherydes-Wayn', 'Wendel-Manderly', 'Maggy', 'Moelle', 'Moon-Boy', 'Robert-Strong', 'Scolera', 'Chataya', 'Chella', 'Karyl-Vance', 'Gretchel', 'Maddy', 'Conn', 'Qavo-Nogarys', 'Val', 'Daryn-Hornwood', 'Torrhen-Karstark', 'Lothor-Brune', 'Jacks', 'Joss', 'Maron-Greyjoy', 'Porther', 'Raymun-Darry', 'Tregar', 'Varly', 'Willam-Dustin', 'Wylla', 'Genna-Lannister', 'Florian-the-Fool', 'Galyeon-of-Cuy', 'Gunthor-son-of-Gurn', 'Horas-Redwyne', 'Hoster-Blackwood', 'Jonos-Bracken', 'Ronnet-Connington', 'Urswyck', 'Joanna-Lannister', 'Robin-Flint', 'Nurse', 'Jyck', 'Morrec', 'Kella', 'Kurleket', 'Leo-Lefford', 'Martyn-Lannister', 'Martyn-Rivers', 'Morgo', 'Myranda-Royce', 'Oswell-Kettleblack', 'Sweets', 'Tristifer-IV-Mudd', 'Scar', 'Shrouded-Lord'], 4: ['Kevan-Lannister', 'Aegon-I-Targaryen', 'Alliser-Thorne', 'Bowen-Marsh', 'Clydas', 'Cotter-Pyke', 'Donal-Noye', 'Janos-Slynt', 'Bronn', 'Bran-Stark', 'Osney-Kettleblack', 'Allard-Seaworth', 'Marya-Seaworth', 'Jory-Cassel', 'Sigorn', 'Myrcella-Baratheon', 'Monford-Velaryon', 'Randyll-Tarly', 'Godry-Farring', 'Aurane-Waters', 'Loras-Tyrell', 'Orton-Merryweather', 'Shireen-Baratheon', 'Barbrey-Dustin', 'Qyburn', 'Shae', 'Dontos-Hollard', 'Bryce-Caron', 'Mathis-Rowan', 'Corliss-Penny', 'Dalla', 'Khorane-Sathmantes', 'Marlon-Manderly', 'Robett-Glover', 'Richard-Horpe', 'Lyanna-Mormont', 'Steffon-Baratheon', 'Roose-Ryswell', 'Theomore', 'William-Foxglove'], 6: ['Tywin-Lannister', 'Catelyn-Stark', 'Robb-Stark', 'Nan', 'Chett', 'Dareon', 'Denys-Mallister', 'Arya-Stark', 'Albett', 'Alester-Florent', 'Axell-Florent', 'Alysane-Mormont', 'Roose-Bolton', 'Yoren', 'Arnolf-Karstark', 'Mors-Umber', 'Arson', 'Clayton-Suggs', 'Tycho-Nestoris', 'Azor-Ahai', 'Dywen', 'Narbert-Grandison', 'Jaremy-Rykker', 'Luwin', 'Jack-Bulwer', 'Big-Boil', 'Cellador', 'Wick-Whittlestick', 'Brandon-Norrey', 'Malegorn', 'Varamyr', 'Lark', 'Thoren-Smallwood', 'Patchface', 'Dalbridge', 'Devan-Seaworth', 'Dick-Follard', 'Mag-Mar-Tun-Doh-Weg', 'Hop-Robin', 'Errok', 'Grigg', 'Fulk', 'Garth-Greyfeather', 'Hairy-Hal', 'Gendel', 'Gorne', 'Gerrick-Kingsblood', 'Kegs', 'Hake', 'Rory', 'Spare-Boot', 'Weeper', 'Jarman-Buckwell', 'Ottyn-Wythers', 'Matthar', 'Orell', 'Othor', 'Soren-Shieldbreaker', 'Tom-Barleycorn', 'Toregg', 'Ulmer', 'Zei'], 3: ['Stannis-Baratheon'], 8: ['Jon-Snow']}

## Edge Roles

In [82]:
results = fast_greedy
edge_role_fast_greedy = []
for item in fast_greedy:
  if(len(item)<=3):
    edge_role_fast_greedy.extend(item)

edge_role_fast_greedy = sorted(set(edge_role_fast_greedy))

with open("edge_role_fast_greedy.txt", "w") as f:
  f.write(str(edge_role_fast_greedy))

In [84]:
# no girvan_newman and louvain because no small communities
# no bron-kerbosch because a lot of small communities

In [85]:
results = hierarchical_clustering
edge_role_hierarchical_clustering = []
for item in hierarchical_clustering:
  if(len(item)<=3):
    edge_role_hierarchical_clustering.extend(item)

edge_role_hierarchical_clustering = sorted(set(edge_role_hierarchical_clustering))

with open("edge_role_hierarchical_clustering.txt", "w") as f:
  f.write(str(edge_role_hierarchical_clustering))

In [90]:
results = spectral_clustering
edge_role_spectral_clustering = []
for item in spectral_clustering.values():
  if(len(item)<=3):
    edge_role_spectral_clustering.extend(item)

edge_role_spectral_clustering = sorted(set(edge_role_spectral_clustering))

with open("edge_role_spectral_clustering.txt", "w") as f:
  f.write(str(edge_role_spectral_clustering))

In [92]:
# Analyze
print('size of {} is {}'.format('edge_role_fast_greedy',len(edge_role_fast_greedy)))
print('size of {} is {}'.format('edge_role_hierarchical_clustering',len(edge_role_hierarchical_clustering)))
print('size of {} is {}'.format('edge_role_spectral_clustering',len(edge_role_spectral_clustering)))

size of edge_role_fast_greedy is 107
size of edge_role_hierarchical_clustering is 10
size of edge_role_spectral_clustering is 3


In [94]:
edge_role_spectral_clustering in edge_role_hierarchical_clustering 

False

In [95]:
edge_role_hierarchical_clustering in edge_role_fast_greedy

False

In [96]:
edge_role_spectral_clustering in edge_role_fast_greedy 

False

In [102]:
similar_items = len(set(edge_role_hierarchical_clustering) & set(edge_role_fast_greedy))
similarity = similar_items / len(edge_role_fast_greedy)
similarity

0.09345794392523364

## Main Roles

In [103]:
# get roles with highest interactions
import pandas as pd

df = pd.read_csv('asoiaf-all-edges.csv')
sender_counts = df.groupby('Source')['weight'].sum()
receiver_counts = df.groupby('Target')['weight'].sum()
counts = sender_counts.add(receiver_counts, fill_value=0).sort_values(ascending=False)

print(counts)

Tyrion-Lannister      2873.0
Jon-Snow              2757.0
Cersei-Lannister      2232.0
Joffrey-Baratheon     1762.0
Eddard-Stark          1649.0
                       ...  
Garth-Tyrell             3.0
Hugh                     3.0
Old-Bill-Bone            3.0
Owen-Merryweather        3.0
Torwold-Browntooth       3.0
Name: weight, Length: 796, dtype: float64
